**1 Collecting Data**


## Improve thing
  1) Chowlin model to expand data frequency
  2) A. สร้าง Environment และ State Vector (ตามหัวข้อ 5. Method ใน Paper)แผนของคุณระบุว่าจะใช้ Reinforcement Learning (RL) ซึ่งต้องการ "สภาพแวดล้อม (Environment)" ในการตัดสินใจ คุณต้องเขียน Code เพื่อรวมข้อมูลจากสองส่วนเข้าด้วยกันเป็น $S_t$:Macro Signal ($m_t$): เอาผล Forecast หรือค่า ECT/Coefficients จากโมเดล ARDL-ECM ที่คุณทำเสร็จแล้วมาเป็น FeatureTechnical Signal ($b_t$): เอาความน่าจะเป็น ($P_{bull}, P_{bear}, P_{neutral}$) หรือสัญญาณจากโมเดล LSTM/GRU มาเป็น FeaturePortfolio State: สถานะปัจจุบัน (ถือเงินสด, ถือหุ้น, ต้นทุน)สิ่งที่ต้องทำ: สร้าง Class Environment (เช่นสืบทอดจาก gym.Env) ที่รับค่าเหล่านี้เข้าไปในแต่ละ Step
  3) B. พัฒนาส่วน Reinforcement Learning (PPO Algorithm)ใน Code ปัจจุบันยังไม่พบส่วนที่เป็น PPO (Proximal Policy Optimization) หรือ Agent ที่ตัดสินใจ "ซื้อ/ขาย/ถือ"Action: คุณต้องกำหนด Action Space เป็น {Buy, Hold, Sell}Reward Function: เขียนสมการ Reward ตามแผน: $\text{Reward} = \text{Portfolio Value} - (\lambda \times \text{Drawdown Penalty})$Implementation: ใช้ Library เช่น stable-baselines3 หรือ rllib เพื่อสร้าง PPO Agent แล้วสั่ง Train ด้วย Environment ที่สร้างในข้อ A
  4) C. เพิ่มโมเดล CNN (Optional ตามแผน)
ในแผนระบุว่าจะใช้ CNN (Convolutional Neural Networks) ร่วมกับ LSTM เพื่อจับ Pattern กราฟ แต่ใน Code พบแค่ LSTM/GRU


5) D. การรวมระบบและ Backtesting (Phase 4)
Rolling Window Training: ต้องเขียน Loop เพื่อให้โมเดล Train ข้อมูลปี 2010-2024 และ Test ปี 2025 (Out-of-sample)

Performance Metrics: Code ปัจจุบันอาจจะวัดแค่ความแม่นยำ (Accuracy/MSE) ของการทำนายราคา แต่ตามแผนคุณต้องวัด Trading Performance ของบอท ได้แก่:

Cumulative Return (กำไรสะสม)

Sharpe Ratio (ผลตอบแทนเทียบความเสี่ยง)

Maximum Drawdown (การขาดทุนสูงสุด)

In [ ]:
# !pip install fredapi
import os
os.chdir()

| Component               | Description                                    | Paper                 |
| ----------------------- | ---------------------------------------------- | --------------------- |
| Proxy selection         | Dynamic Factor / PCA on macro dataset          | Stock & Watson (2002) |
| Temporal disaggregation | Chow-Lin linear regression                     | Chow & Lin (1971)     |
| Extrapolation           | AR(1) residual correction (Fernandez model)    | Fernandez (1981)      |
| Ratio preservation      | Denton (1971) for % variables (e.g., Debt/GDP) | Denton (1971)         |

📘 Academic Framework (สำหรับ citation ใน report)

Chow & Lin (1971) – Best Linear Unbiased Interpolation by Related Series

Stock & Watson (2002) – Macroeconomic Forecasting Using Diffusion Indexes

Mariano & Murasawa (2003) – A New Coincident Index of Business Cycles Based on Monthly and Quarterly Series


GDP Data
(https://)https://lookerstudio.google.com/u/0/reporting/52a283c8-91f9-4b35-8157-2b2b42312602/page/p_l7cxsbayvc

In [ ]:
# ================== Setup ==================
# pip install yfinance fredapi pandas pandas-datareader

import pandas as pd
import numpy as np
import yfinance as yf
from fredapi import Fred
from pandas_datareader import wb
from datetime import date
import io, requests
from pathlib import Path

# ----- Parameters -----
START = "2013-01-01"
END   = date.today().isoformat()         # วันนี้
FRED_API_KEY = "c948956426006ca126a2dd3bd1f07cee"

# FRED client (กันอินเทอร์เน็ตล่ม)
try:
    fred = Fred(api_key=FRED_API_KEY)
except Exception:
    fred = None

# ================== Helper Functions ==================
def get_fred_series(series_id: str, rename_to: str = None) -> pd.Series:
    """ดึงซีรีส์จาก FRED"""
    if fred is None:
        return pd.Series(dtype="float64", name=rename_to or series_id)
    s = fred.get_series(series_id)
    s.name = rename_to or series_id
    s.index = pd.to_datetime(s.index)
    return s.sort_index()

def expand_period_to_target_index(target_index: pd.DatetimeIndex,
                                  lowfreq_series: pd.Series,
                                  freq: str) -> pd.Series:
    """ขยายค่าความถี่ต่ำไปที่ target_index"""
    if lowfreq_series is None or lowfreq_series.empty:
        return pd.Series(index=target_index, dtype="float64", name=getattr(lowfreq_series, "name", None))
    s = lowfreq_series.dropna().copy()
    s.index = pd.PeriodIndex(s.index, freq=freq)
    df = pd.DataFrame(index=target_index)
    df["period"] = df.index.to_period(freq)
    out = df.join(s.rename(s.name), on="period")[s.name]
    out.name = s.name
    return out

# ================== 2) Macro: ใช้ Macro_data.xlsx ==================
macro_candidates = [Path("/mnt/data/Macro_data.xlsx"), Path("Macro_data.xlsx")]
macro_path = next((str(p) for p in macro_candidates if p.exists()), "Macro_data.xlsx")

macro = pd.read_excel(macro_path, sheet_name=0)

def _pct_to_float(col: pd.Series) -> pd.Series:
    """แปลงคอลัมน์ % ให้เป็นตัวเลข"""
    return (
        col.astype(str)
           .str.strip()
           .str.replace(",", "", regex=False)
           .str.replace("%", "", regex=False)
           .str.replace("(", "-", regex=False)
           .str.replace(")", "", regex=False)
           .replace({"": None})
           .pipe(pd.to_numeric, errors="coerce")
    )

macro["Date"] = pd.to_datetime(macro["Date"], dayfirst=True, errors="coerce")

for col in ["CPI_Actual", "CPI_Forecast", "IPI_Actual", "IPI_Forecast"]:
    if col in macro.columns:
        macro[col] = _pct_to_float(macro[col])

macro = (macro.rename(columns={
            "M1M2_Actual": "BroadMoney_M1M2_Level",
            "CPI_Actual": "Inflation",
            "CPI_Forecast": "Inflation_Forecast",
            "THOR_1M": "THOR_1M",
            "THOR_3M": "THOR_3M",
            "THOR_6M": "THOR_6M",
            "IPI_Actual": "IPI",
            "IPI_Forecast": "IPI_Forecast",
            "Bond_1Y": "Bond_Yield_1Y",
            "Bond_5Y": "Bond_Yield_5Y",
            "Bond_10Y": "Bond_Yield_10Y",
            "Bond_spread_10Y_5Y": "Yield_Spread_10Y_5Y",
            "Bond_spread_5Y_1Y": "Yield_Spread_5Y_1Y",
            "Bond_spread_10Y_1Y": "Yield_Spread_10Y_1Y"
        })
        .sort_values("Date")
        .set_index("Date"))

# ================== 1) Stock Prices ==================
SECTORS = {
    "Banking": ["BBL","KBANK","KKP","KTB","TCAP","TISCO","TTB"],
    "Finance_Securities": ["KTC","AEONTS","JMT","SAWAD"],
    # "Energy_Utilities": ["BANPU","BCP","EGCO","GPSC","GULF","OR","PTT","PTTEP","RATCH","TOP","BGRIM","BCPG","EA","GUNKUL","IRPC","SPRC","WHAUP","CKP"],
    # "Food_Beverage": ["CBG","CPF","OSP","TU","ITC","BTG","ICHI","M","TFG","COCOCO","SAPPE","SNNP"],
    # "Commerce": ["BJC","COM7","CPALL","CRC","HMPRO","GLOBAL","DOHOME","MEGA","MOSHI"],
    # "Property_Development": ["AWC","CPN","LH","WHA","AMATA","AP","MBK","QH","SIRI","SPALI","ROJNA"],
    # "Health_Care_Services": ["BDMS","BH","BCH","CHG"],       =>
    # "Electronic_Components": ["CCET","DELTA","HANA","KCE"],
    # "Construction_Materials": ["SCC","TASCO","TOA"],
    # "ICT": ["ADVANC","TRUE","JAS","JMART","JTS","SKY"],
    # "Transportation_Logistics": ["AOT","BEM","BTS","AAV","BA","RCL"],     =>
}

df_stocks = pd.DataFrame()
for sector, tickers in SECTORS.items():
    for ticker in tickers:
        try:
            ticker_yf = ticker + ".BK"
            data = yf.download(ticker_yf, start=START, end=END, interval="1mo")['Close']
            if isinstance(data, pd.Series):
                df_stocks[f"Close_{ticker}"] = data
            else:
                df_stocks[f"Close_{ticker}"] = data[ticker_yf]
        except Exception:
            df_stocks[f"Close_{ticker}"] = pd.Series(dtype="float64")

df_stocks = df_stocks.dropna(how="all")

if not df_stocks.empty:
    quarterly_index = df_stocks.index
else:
    quarterly_index = pd.date_range(start=pd.to_datetime(START), end=pd.to_datetime(END), freq="QS")

# ================== 2) Align Macro ==================
if not macro.index.is_unique:
    macro = macro[~macro.index.duplicated(keep="last")]

macro_q = macro.reindex(quarterly_index, method="ffill")

# FX THB/USD
fx_thb_usd = yf.download("THB=X", start=START, end=END, interval="1mo")["Close"]
fx_thb_usd.name = "THB_per_USD"

# FX THB/CNY
fx_thb_cny = yf.download("THBCNY=X", start=START, end=END, interval="1mo")["Close"]
fx_thb_cny.name = "THB_per_CNY"

# Brent Oil (USD per barrel)
brent_data = yf.download("BZ=F", start=START, end=END, interval="1mo")["Close"]
brent_data.name = "Brent_Oil_USD_per_bbl"

# CRB Index
crb_data = yf.download("^CRB", start=START, end=END, interval="1mo")["Close"]
crb_data.name = "CRB_Index"

# S&P 500 Index
sp500_data = yf.download("^GSPC", start=START, end=END, interval="1mo")["Close"]
sp500_data.name = "SP500_Index"

# SET Index (Thailand)
set_daily = yf.download("^SET.BK", start=START, end=END, interval="1d")["Close"]

# --- เลือกราคาปิดของ "วันแรกของแต่ละเดือน" ---
set_monthly = set_daily.resample("MS").first()
set_monthly.name = "SET_Index"

# ================== 4) Combine หุ้น + Macro ==================
df = df_stocks.copy()
if df.empty:
    df = pd.DataFrame(index=quarterly_index)

# Macro Variables
if "Inflation" in macro_q.columns:
    df["Inflation"] = macro_q["Inflation"] / 100.0
if "Inflation_Forecast" in macro_q.columns:
    df["Inflation_Forecast"] = macro_q["Inflation_Forecast"] / 100.0
if "IPI" in macro_q.columns:
    df["IPI"] = macro_q["IPI"] / 100.0
if "IPI_Forecast" in macro_q.columns:
    df["IPI_Forecast"] = macro_q["IPI_Forecast"] / 100.0

# ✅ BroadMoney: ใช้สูตร ln(S_t / S_{t-1}) และให้แถวแรกเป็น 0
if "BroadMoney_M1M2_Level" in macro_q.columns:
    money_series = macro_q["BroadMoney_M1M2_Level"]
    money_growth = np.log(money_series / money_series.shift(1))
    money_growth.iloc[0] = 0.0  # แถวแรกให้เป็น 0%
    df["BroadMoney_M1M2_Growth"] = money_growth

# Interest Rates
for col in ["THOR_1M", "THOR_3M", "THOR_6M",
            "Bond_Yield_1Y", "Bond_Yield_5Y", "Bond_Yield_10Y",
            "Yield_Spread_10Y_5Y", "Yield_Spread_5Y_1Y", "Yield_Spread_10Y_1Y"]:
    if col in macro_q.columns:
        df[col] = macro_q[col]

# Combine Global Indicators
df["THB_per_USD"] = fx_thb_usd
df["THB_per_CNY"] = fx_thb_cny
# df["Brent_Oil_USD_per_bbl"] = brent_data
df["CRB_Index"] = crb_data
df["SP500_Index"] = sp500_data
df["SET_Index"] = set_monthly

df['IPI_Surprise'] = df['IPI'] - df['IPI_Forecast']
df['Inflation_Surprise'] = df['Inflation'] - df['Inflation_Forecast']
df['5Y_1Y_Bond_Spread'] = df["Bond_Yield_5Y"] - df["Bond_Yield_1Y"]
df['10Y_5Y_Bond_Spread'] = df["Bond_Yield_10Y"] - df["Bond_Yield_5Y"]
df['3M_1M_THOR_Spread'] = df["THOR_3M"] - df["THOR_1M"]
df['6M_1M_THOR_Spread'] = df["THOR_6M"] - df["THOR_1M"]



# ================== 5) ตรวจสอบผลลัพธ์ ==================
df = df.iloc[25:]
df.drop(["CRB_Index"], axis=1, inplace=True)

print(df)
print("\n=== HEAD ===")
print(df.head(10))
print("\n=== TAIL ===")
print(df.tail(10))
print("\n=== COLUMNS ===")
print(df.columns.tolist())
print("\n=== NA COUNTS ===")
print(df.isna().sum())

# (Optional) Save to CSV
# df.to_csv("th_stock_macro_2013_today_quarterly.csv", encoding="utf-8-sig")


/tmp/ipython-input-4088842435.py:112: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_yf, start=START, end=END, interval="1mo")['Close']
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-4088842435.py:112: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_yf, start=START, end=END, interval="1mo")['Close']
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-4088842435.py:112: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_yf, start=START, end=END, interval="1mo")['Close']
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-4088842435.py:112: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_yf, start=START, end=END, interval="1mo")['Close']
[************

             Close_BBL  Close_KBANK  Close_KKP  Close_KTB  Close_TCAP  \
Date                                                                    
2015-02-01  120.324242   151.045563  18.905380  13.843618   16.486799   
2015-03-01  121.310509   159.398285  18.784966  13.843618   16.369032   
2015-04-01  121.639305   146.173111  17.821629  12.143525   16.015741   
2015-05-01  122.538742   137.793167  17.965874  11.427542   16.611540   
2015-06-01  119.845604   133.553375  17.591583  10.796185   16.126530   
...                ...          ...        ...        ...         ...   
2025-09-01  147.045898   165.540939  57.287346  24.412216   48.975368   
2025-10-01  158.500000   186.500000  65.750000  26.823908   51.000000   
2025-11-01  158.000000   186.500000  64.250000  27.070000   54.500000   
2025-12-01  169.500000   194.500000  68.000000  28.250000   58.250000   
2026-01-01  170.500000   191.500000  66.250000  28.750000   57.750000   

            Close_TISCO  Close_TTB  Close_KTC  Clo

In [ ]:
# ============================================================
# 📊 BLOCK 1.5: Chow-Lin Temporal Disaggregation (GDP Quarterly → Monthly)
#    Reference: Chow & Lin (1971) - Best Linear Unbiased Interpolation
#    Insert this block AFTER Block 1 (Data Collection) and BEFORE Block 2
# ============================================================

import numpy as np
import pandas as pd
from scipy import linalg

# ================== 1) Chow-Lin Function ==================
def chow_lin_disaggregate(y_low: pd.Series, X_high: pd.DataFrame,
                          agg_method: str = 'sum', rho: float = None) -> tuple:
    """
    Chow-Lin temporal disaggregation: แปลง Low-frequency → High-frequency

    Parameters:
    -----------
    y_low : pd.Series
        Low-frequency series (e.g., Quarterly GDP) with DatetimeIndex
    X_high : pd.DataFrame
        High-frequency indicator(s) (e.g., Monthly IPI) with DatetimeIndex
    agg_method : str
        'sum' for flow variables (GDP), 'mean' for stock/average variables
    rho : float or None
        AR(1) autocorrelation parameter. If None, estimate from data.

    Returns:
    --------
    tuple: (pd.Series of monthly data, beta coefficients, estimated rho)
    """

    # 1) Align data
    y_low = y_low.dropna().copy()
    X_high = X_high.dropna().copy()

    # Determine frequency ratio (3 for Q→M)
    n_high_per_low = 3  # Monthly to Quarterly

    # 2) Create date mapping
    quarters = y_low.index
    months = X_high.index

    # Filter to overlapping period
    min_date = max(quarters.min(), months.min().to_period('Q').to_timestamp())
    max_date = min(quarters.max(), months.max().to_period('Q').to_timestamp())

    y_low = y_low[(y_low.index >= min_date) & (y_low.index <= max_date)]

    # Get monthly data for the period
    month_start = y_low.index.min()
    month_end = (y_low.index.max() + pd.offsets.QuarterEnd()).to_period('M').to_timestamp()

    X_high = X_high[(X_high.index >= month_start) & (X_high.index <= month_end)]

    n_low = len(y_low)
    n_high = n_low * n_high_per_low

    # Trim X_high to exact size
    X_high = X_high.iloc[:n_high]

    if len(X_high) < n_high:
        print(f"⚠️ Warning: Not enough high-freq data. Need {n_high}, have {len(X_high)}")
        # Pad with last value
        pad_size = n_high - len(X_high)
        last_idx = X_high.index[-1]
        new_idx = pd.date_range(start=last_idx + pd.offsets.MonthBegin(), periods=pad_size, freq='MS')
        pad_df = pd.DataFrame(index=new_idx, columns=X_high.columns)
        for col in X_high.columns:
            pad_df[col] = X_high[col].iloc[-1]
        X_high = pd.concat([X_high, pad_df])

    # 3) Build aggregation matrix C (n_low x n_high)
    C = np.zeros((n_low, n_high))
    for i in range(n_low):
        start_col = i * n_high_per_low
        end_col = start_col + n_high_per_low
        if agg_method == 'sum':
            C[i, start_col:end_col] = 1.0
        elif agg_method == 'mean':
            C[i, start_col:end_col] = 1.0 / n_high_per_low
        else:  # 'last'
            C[i, end_col - 1] = 1.0

    # 4) Prepare X matrix (add constant)
    X = X_high.values
    if X.ndim == 1:
        X = X.reshape(-1, 1)
    X = np.column_stack([np.ones(n_high), X])  # Add intercept

    # 5) Aggregate X to match y_low
    X_low = C @ X
    y = y_low.values.flatten()

    # 6) OLS on aggregated data
    beta_ols = np.linalg.lstsq(X_low, y, rcond=None)[0]
    u_low = y - X_low @ beta_ols

    # 7) Estimate rho (AR1 coefficient) if not provided
    if rho is None:
        if len(u_low) > 1:
            rho = np.corrcoef(u_low[:-1], u_low[1:])[0, 1]
            rho = np.clip(rho, -0.99, 0.99)
        else:
            rho = 0.0

    # 8) Build covariance matrix V (AR1 structure)
    V = np.zeros((n_high, n_high))
    for i in range(n_high):
        for j in range(n_high):
            V[i, j] = rho ** abs(i - j)

    # 9) GLS estimation
    V_low = C @ V @ C.T
    try:
        V_low_inv = np.linalg.inv(V_low)
    except np.linalg.LinAlgError:
        V_low_inv = np.linalg.pinv(V_low)

    XVX = X_low.T @ V_low_inv @ X_low
    XVy = X_low.T @ V_low_inv @ y
    try:
        beta_gls = np.linalg.solve(XVX, XVy)
    except np.linalg.LinAlgError:
        beta_gls = np.linalg.lstsq(XVX, XVy, rcond=None)[0]

    # 10) Preliminary high-frequency estimate
    p_high = X @ beta_gls

    # 11) Distribute residuals
    u_low_gls = y - X_low @ beta_gls
    VCt = V @ C.T
    try:
        dist_matrix = VCt @ np.linalg.inv(V_low)
    except np.linalg.LinAlgError:
        dist_matrix = VCt @ np.linalg.pinv(V_low)

    # Final disaggregated series
    y_high = p_high + dist_matrix @ u_low_gls

    # 12) Create output Series
    result = pd.Series(y_high, index=X_high.index, name=y_low.name or 'GDP_Monthly')

    return result, beta_gls, rho


# ================== 2) Load GDP Quarterly Data ==================
print("="*60)
print("📊 BLOCK 1.5: Chow-Lin Temporal Disaggregation")
print("="*60)

print("\n📥 Loading GDP Quarterly data...")

gdp_df = pd.read_excel("thai_gdp_quarterly_full.xlsx", sheet_name=0)

# Create proper datetime index
quarter_map = {'Q1': '01', 'Q2': '04', 'Q3': '07', 'Q4': '10'}
gdp_df['Date'] = gdp_df.apply(
    lambda row: pd.Timestamp(f"{row['Year']}-{quarter_map[row['Quarter']]}-01"),
    axis=1
)
gdp_df = gdp_df.set_index('Date').sort_index()

# Use Total_CUR (Current Price GDP)
gdp_quarterly = gdp_df['Total_CUR'].copy()
gdp_quarterly.name = 'GDP_CUR'

print(f"✅ GDP Quarterly loaded: {len(gdp_quarterly)} quarters")
print(f"   Period: {gdp_quarterly.index.min().strftime('%Y-%m')} to {gdp_quarterly.index.max().strftime('%Y-%m')}")


# ================== 3) Prepare IPI as High-Frequency Indicator ==================
print("\n📥 Preparing IPI as high-frequency indicator...")

# IPI should already be in 'df' from Block 1
# If running standalone, load from Macro_data.xlsx
if 'df' not in globals() or 'IPI' not in df.columns:
    macro_temp = pd.read_excel("Macro_data.xlsx", sheet_name=0)
    macro_temp['Date'] = pd.to_datetime(macro_temp['Date'], dayfirst=True, errors='coerce')
    macro_temp = macro_temp.set_index('Date').sort_index()
    ipi_monthly = macro_temp['IPI_Actual'].dropna().copy()
else:
    ipi_monthly = df['IPI'].dropna().copy()

ipi_monthly.name = 'IPI'

# Convert to level if it's in growth rate form
if ipi_monthly.abs().mean() < 1:  # It's in decimal/percentage
    ipi_level = (1 + ipi_monthly).cumprod() * 100
else:
    ipi_level = ipi_monthly

ipi_level.name = 'IPI_Level'

print(f"✅ IPI Monthly loaded: {len(ipi_level)} months")
print(f"   Period: {ipi_level.index.min().strftime('%Y-%m')} to {ipi_level.index.max().strftime('%Y-%m')}")


# ================== 4) Apply Chow-Lin Disaggregation ==================
print("\n🔄 Applying Chow-Lin disaggregation (GDP Quarterly → Monthly)...")

X_indicator = pd.DataFrame({'IPI': ipi_level})

gdp_monthly, beta_chowlin, rho_chowlin = chow_lin_disaggregate(
    y_low=gdp_quarterly,
    X_high=X_indicator,
    agg_method='sum',  # GDP is a flow variable
    rho=None  # Auto-estimate
)

# Calculate GDP Growth Rate (Monthly, Log %)
gdp_growth_monthly = np.log(gdp_monthly / gdp_monthly.shift(1))
gdp_growth_monthly.name = 'GDP_Growth_Monthly'

print(f"\n✅ Chow-Lin disaggregation completed!")
print(f"   - Estimated AR(1) rho: {rho_chowlin:.4f}")
print(f"   - Monthly GDP observations: {len(gdp_monthly)}")


# ================== 5) Validation ==================
print("\n🔍 Validation: Aggregating monthly back to quarterly...")

gdp_monthly_to_q = gdp_monthly.resample('QS').sum()
validation = pd.DataFrame({
    'Original_Q': gdp_quarterly,
    'ChowLin_Agg_Q': gdp_monthly_to_q
}).dropna()
validation['Error_%'] = abs(validation['ChowLin_Agg_Q'] - validation['Original_Q']) / validation['Original_Q'] * 100

print(f"✅ Mean Aggregation Error: {validation['Error_%'].mean():.6f}%")
print("   (Should be ~0% if Chow-Lin is working correctly)")


# ================== 6) Add to Main DataFrame ==================
print("\n📌 Adding GDP_Monthly and GDP_Growth to main dataframe...")

# Reindex to match the main df index
if 'df' in globals():
    # Add GDP columns to df
    df['GDP_Monthly'] = gdp_monthly.reindex(df.index)
    df['GDP_Growth'] = gdp_growth_monthly.reindex(df.index)

    # Forward fill any gaps (for dates where we have stock data but no GDP yet)
    df['GDP_Monthly'] = df['GDP_Monthly'].ffill()
    df['GDP_Growth'] = df['GDP_Growth'].ffill()

    print(f"✅ Added 'GDP_Monthly' and 'GDP_Growth' to df")
    print(f"   df shape: {df.shape}")
    print(f"\n📊 GDP columns preview:")
    print(df[['GDP_Monthly', 'GDP_Growth']].dropna().tail(10))
else:
    print("⚠️ Main dataframe 'df' not found. Creating standalone GDP dataframe...")
    gdp_output = pd.DataFrame({
        'GDP_Monthly': gdp_monthly,
        'GDP_Growth': gdp_growth_monthly
    })
    print(gdp_output.tail(10))


# ================== 7) Summary ==================
print("\n" + "="*60)
print("✅ BLOCK 1.5 COMPLETED: Chow-Lin Temporal Disaggregation")
print("="*60)
print(f"""
Summary:
- Input:  GDP Quarterly (Total_CUR) - {len(gdp_quarterly)} quarters
- Output: GDP Monthly - {len(gdp_monthly)} months
- Indicator used: IPI (Industrial Production Index)
- Method: Chow-Lin (1971) with AR(1) residual structure
- Estimated rho: {rho_chowlin:.4f}

New columns added to df:
- 'GDP_Monthly': Monthly GDP level (Current Price, Mil THB)
- 'GDP_Growth': Monthly GDP growth rate (Log %)
""")

📊 BLOCK 1.5: Chow-Lin Temporal Disaggregation

📥 Loading GDP Quarterly data...
✅ GDP Quarterly loaded: 51 quarters
   Period: 2013-01 to 2025-07

📥 Preparing IPI as high-frequency indicator...
✅ IPI Monthly loaded: 132 months
   Period: 2015-02 to 2026-01

🔄 Applying Chow-Lin disaggregation (GDP Quarterly → Monthly)...
⚠️ Warning: Not enough high-freq data. Need 129, have 128

✅ Chow-Lin disaggregation completed!
   - Estimated AR(1) rho: 0.8322
   - Monthly GDP observations: 129

🔍 Validation: Aggregating monthly back to quarterly...
✅ Mean Aggregation Error: 1.868700%
   (Should be ~0% if Chow-Lin is working correctly)

📌 Adding GDP_Monthly and GDP_Growth to main dataframe...
✅ Added 'GDP_Monthly' and 'GDP_Growth' to df
   df shape: (132, 37)

📊 GDP columns preview:
             GDP_Monthly  GDP_Growth
Date                                
2025-04-01  1.566892e+06   -0.014493
2025-05-01  1.536566e+06   -0.019544
2025-06-01  1.521829e+06   -0.009637
2025-07-01  1.522122e+06    0.000192

**2 Data preprocessing**

In [ ]:
# =====================================================
# ⚙️ BLOCK 2: Prepare Combined Data (No PCA, No LASSO)
# =====================================================

import numpy as np
import pandas as pd

# ======== 1) Filter & split ========
df_filtered = df.iloc[14:].copy()  # ตัดช่วงแรกที่ NA เยอะ

# เลือกหุ้น (ตัวอย่าง 8 ตัวแรก)
stock_columns = [c for c in df_filtered.columns if c.startswith("Close_")]
selected_stock_data = df_filtered[stock_columns]

# เลือกตัวแปร macro ที่มีอยู่จริง
macro_columns = [
    "BroadMoney_M1M2_Growth",
    "Inflation_Surprise",
    "IPI_Surprise",
    "THOR_1M",
    "THOR_6M",
    "3M_1M_THOR_Spread",
    "6M_1M_THOR_Spread",
    "5Y_1Y_Bond_Spread",
    "10Y_5Y_Bond_Spread",
    "THB_per_USD",
    # "Brent_Oil_USD_per_bbl",
    "SP500_Index",
    "SET_Index",
    "THB_per_CNY",
    "GDP_Growth"
]
macro_columns = [c for c in macro_columns if c in df_filtered.columns]
selected_macro_data = df_filtered[macro_columns]

# ======== 2) Combine + clean ========
combined_df = pd.concat([selected_stock_data, selected_macro_data], axis=1)
combined_df = combined_df.ffill().replace([np.inf, -np.inf], np.nan).dropna(how="any")

print("✅ Combined data shape:", combined_df.shape)
print("✅ Stocks:", len(stock_columns), "| Macros:", len(macro_columns))

# ======== 3) Log-transform หุ้นและสร้าง Δlog ========
combined_dfs = combined_df.copy()
for c in stock_columns:
    combined_dfs[f"Logclose_{c.replace('Close_','')}"] = np.log(combined_df[c])

combined_dfs_diff = combined_dfs.copy()
for c in stock_columns:
    stock_name = c.replace("Close_","")
    combined_dfs_diff[f"D_Logclose_{stock_name}"] = combined_dfs[f"Logclose_{stock_name}"].diff()

# ======== 4) Drop แถวแรกของ differenced ========
combined_dfs_diff = combined_dfs_diff.dropna()

print("✅ Differenced data ready:", combined_dfs_diff.shape)

# ======== 5) ตัวอย่างผลลัพธ์ ========
print("\n=== Combined_dfs (Levels) ===")
display(combined_dfs.head())

print("\n=== Combined_dfs_diff (Differenced) ===")
display(combined_dfs_diff.head())


✅ Combined data shape: (118, 25)
✅ Stocks: 11 | Macros: 14
✅ Differenced data ready: (117, 47)

=== Combined_dfs (Levels) ===


,Close_BBL,Close_KBANK,Close_KKP,Close_KTB,Close_TCAP,Close_TISCO,Close_TTB,Close_KTC,Close_AEONTS,Close_JMT,...,Logclose_KBANK,Logclose_KKP,Logclose_KTB,Logclose_TCAP,Logclose_TISCO,Logclose_TTB,Logclose_KTC,Logclose_AEONTS,Logclose_JMT,Logclose_SAWAD
Date,,,,,,,,,,,,,,,,,,,,,
2016-04-01,111.808441,117.630341,21.347910,11.048728,17.233999,20.880671,1.363181,7.560795,68.491669,3.997082,...,4.767547,3.060954,2.402315,2.846884,3.038824,0.309821,2.022976,4.226712,1.385564,3.230630
2016-05-01,114.526146,125.922653,22.376720,10.796185,17.386007,21.757637,1.397547,7.273935,66.148521,3.582022,...,4.835668,3.108021,2.379193,2.855666,3.079965,0.334718,1.984297,4.191903,1.275927,3.236598
2016-06-01,111.023811,123.027908,23.455124,10.761127,17.897360,25.046576,1.323992,8.134515,70.771049,4.203393,...,4.812411,3.155089,2.375940,2.884653,3.220737,0.280651,2.096116,4.259450,1.435892,3.124680
2016-07-01,119.429390,142.929504,28.307909,11.487339,20.837645,27.323536,1.385288,10.040080,73.528366,5.117174,...,4.962352,3.343141,2.441245,3.036761,3.307748,0.325908,2.306585,4.297671,1.632602,3.137752
2016-08-01,118.728935,142.929504,27.903509,12.543647,20.454126,27.323536,1.385288,11.392418,72.609253,5.336482,...,4.962352,3.328752,2.529214,3.018185,3.307748,0.325908,2.432948,4.285092,1.674567,3.163394



=== Combined_dfs_diff (Differenced) ===


,Close_BBL,Close_KBANK,Close_KKP,Close_KTB,Close_TCAP,Close_TISCO,Close_TTB,Close_KTC,Close_AEONTS,Close_JMT,...,D_Logclose_KBANK,D_Logclose_KKP,D_Logclose_KTB,D_Logclose_TCAP,D_Logclose_TISCO,D_Logclose_TTB,D_Logclose_KTC,D_Logclose_AEONTS,D_Logclose_JMT,D_Logclose_SAWAD
Date,,,,,,,,,,,,,,,,,,,,,
2016-05-01,114.526146,125.922653,22.376720,10.796185,17.386007,21.757637,1.397547,7.273935,66.148521,3.582022,...,0.068121,0.047067,-0.023122,0.008782,0.041141,0.024898,-0.038679,-0.034810,-0.109637,0.005968
2016-06-01,111.023811,123.027908,23.455124,10.761127,17.897360,25.046576,1.323992,8.134515,70.771049,4.203393,...,-0.023257,0.047068,-0.003253,0.028988,0.140772,-0.054067,0.111819,0.067547,0.159965,-0.111918
2016-07-01,119.429390,142.929504,28.307909,11.487339,20.837645,27.323536,1.385288,10.040080,73.528366,5.117174,...,0.149940,0.188052,0.065305,0.152108,0.087011,0.045257,0.210469,0.038221,0.196710,0.013072
2016-08-01,118.728935,142.929504,27.903509,12.543647,20.454126,27.323536,1.385288,11.392418,72.609253,5.336482,...,0.000000,-0.014389,0.087969,-0.018577,0.000000,0.000000,0.126363,-0.012579,0.041964,0.025642
2016-09-01,113.825684,135.692520,28.577501,11.619377,20.709803,26.564548,1.299473,12.089076,73.344536,5.117174,...,-0.051960,0.023867,-0.076540,0.012423,-0.028171,-0.063949,0.059354,0.010076,-0.041964,-0.092782


In [ ]:
# ======== Filter sector log-prices + macro variables ========

# Macro columns ที่มีอยู่จริงใน Excel + yfinance
macro_columns = [
    "BroadMoney_M1M2_Growth",
    "Inflation_Surprise",
    "IPI_Surprise",
    "THOR_1M",
    "THOR_6M",
    "3M_1M_THOR_Spread",
    "6M_1M_THOR_Spread",
    "5Y_1Y_Bond_Spread",
    "10Y_5Y_Bond_Spread",
    "THB_per_USD",
    # "Brent_Oil_USD_per_bbl",
    "SP500_Index",
    "SET_Index",
    "THB_per_CNY",
    "GDP_Growth"
]



# ======== ตรวจสอบว่า df ถูกสร้างแล้วจาก Block ก่อนหน้า ========
if 'df' not in globals():
    raise RuntimeError("❌ Variable 'df' not found. Please run the data-preparation block first.")

df_filtered_macro = df

# เก็บผลลัพธ์แยก sector
combined_dfs = {}

for sector, tickers_list in SECTORS.items():
    sector_frames = []

    for ticker in tickers_list:
        colname = f"Close_{ticker}"
        if colname not in df.columns:
            print(f"⚠️ {colname} not found in df. Skipping...")
            continue

        # ✅ ใช้ Logclose แทน log-return
        tmp = df[[colname]].copy()
        tmp[f"Logclose_{ticker}"] = np.log(tmp[colname])
        tmp = tmp[[f"Logclose_{ticker}"]]

        # align index ให้ตรงกับ macro
        tmp = tmp.loc[df_filtered_macro.index]
        sector_frames.append(tmp)

    if len(sector_frames) == 0:
        continue

    # รวม Logclose ของหุ้นทุกตัวใน sector
    sector_data = pd.concat(sector_frames, axis=1)

    # รวมกับ macro
    combined_df = pd.concat([sector_data, df_filtered_macro], axis=1).ffill()

    # ตัดข้อมูลปี 2025 (ป้องกัน incomplete data)
    combined_df = combined_df[combined_df.index.year != 2025]

    combined_dfs[sector] = combined_df

# ======== แสดงตัวอย่างผลลัพธ์ ========
for sector, combined_df in combined_dfs.items():
    print(f"\n📊 Combined data for sector: {sector}")
    cols_to_show = [c for c in combined_df.columns if c.startswith("Logclose_")] + macro_columns
    cols_to_show_present = [col for col in cols_to_show if col in combined_df.columns]
    display(combined_df[cols_to_show_present].head())



📊 Combined data for sector: Banking


,Logclose_BBL,Logclose_KBANK,Logclose_KKP,Logclose_KTB,Logclose_TCAP,Logclose_TISCO,Logclose_TTB,BroadMoney_M1M2_Growth,Inflation_Surprise,IPI_Surprise,...,THOR_6M,3M_1M_THOR_Spread,6M_1M_THOR_Spread,5Y_1Y_Bond_Spread,10Y_5Y_Bond_Spread,THB_per_USD,SP500_Index,SET_Index,THB_per_CNY,GDP_Growth
Date,,,,,,,,,,,,,,,,,,,,,
2015-02-01,4.790190,5.017582,2.939447,2.627824,2.802560,3.072488,0.569241,0.009765,-0.0066,-0.000066,...,0.020023,0.000015,0.000068,0.0035,0.0056,32.341000,2104.500000,1582.699951,0.19335,NaN
2015-03-01,4.798353,5.071406,2.933057,2.627824,2.795391,3.045532,0.555908,0.004617,-0.0019,0.000380,...,0.020019,0.000032,0.000077,0.0045,0.0058,32.540001,2067.889893,1582.140015,0.19040,-0.020455
2015-04-01,4.801060,4.984792,2.880413,2.496796,2.773572,3.034543,0.419496,-0.001550,-0.0012,-0.000670,...,0.019729,0.001167,0.001528,0.0048,0.0051,32.838001,2085.510010,1525.579956,0.18827,-0.015059
2015-05-01,4.808427,4.925754,2.888474,2.436026,2.810098,3.072725,0.417810,0.001422,-0.0024,-0.000330,...,0.019148,0.001280,0.002220,0.0042,0.0078,33.734001,2107.389893,1519.880005,0.18351,-0.011596
2015-06-01,4.786204,4.894501,2.867421,2.379193,2.780466,3.094704,0.335797,-0.005075,-0.0015,-0.000280,...,0.018408,0.001854,0.003469,0.0056,0.0082,33.729000,2063.110107,1476.869995,0.18359,-0.002867



📊 Combined data for sector: Finance_Securities


,Logclose_KTC,Logclose_AEONTS,Logclose_JMT,Logclose_SAWAD,BroadMoney_M1M2_Growth,Inflation_Surprise,IPI_Surprise,THOR_1M,THOR_6M,3M_1M_THOR_Spread,6M_1M_THOR_Spread,5Y_1Y_Bond_Spread,10Y_5Y_Bond_Spread,THB_per_USD,SP500_Index,SET_Index,THB_per_CNY,GDP_Growth
Date,,,,,,,,,,,,,,,,,,
2015-02-01,1.911173,4.324111,1.979026,3.156030,0.009765,-0.0066,-0.000066,0.019955,0.020023,0.000015,0.000068,0.0035,0.0056,32.341000,2104.500000,1582.699951,0.19335,NaN
2015-03-01,2.032421,4.281752,1.914487,3.106020,0.004617,-0.0019,0.000380,0.019942,0.020019,0.000032,0.000077,0.0045,0.0058,32.540001,2067.889893,1582.140015,0.19040,-0.020455
2015-04-01,2.123729,4.252482,1.897821,3.193960,-0.001550,-0.0012,-0.000670,0.018201,0.019729,0.001167,0.001528,0.0048,0.0051,32.838001,2085.510010,1525.579956,0.18827,-0.015059
2015-05-01,1.998252,4.206817,1.863420,3.200002,0.001422,-0.0024,-0.000330,0.016928,0.019148,0.001280,0.002220,0.0042,0.0078,33.734001,2107.389893,1519.880005,0.18351,-0.011596
2015-06-01,2.047805,4.201526,1.712987,3.151212,-0.005075,-0.0015,-0.000280,0.014939,0.018408,0.001854,0.003469,0.0056,0.0082,33.729000,2063.110107,1476.869995,0.18359,-0.002867


In [ ]:
# Select the dataframe for the first stock (e.g., 'BBL') from the combined_dfs dictionary
# This cell is intended to display example dataframes from combined_dfs,
# which are structured by sector, not individual tickers.
# Iterate through the combined_dfs dictionary using sector names as keys.

if 'combined_dfs' in globals() and combined_dfs:
    print("Displaying head of combined dataframes for each sector:")
    for sector, combined_df_example in combined_dfs.items():
        print(f"\n--- Combined dataframe for {sector} ---")
        # Display the columns of the example combined dataframe
        print(f"Columns in the combined dataframe for {sector}:")
        print(combined_df_example.columns)

        # Display the head of the example combined dataframe
        print(f"\nHead of the combined dataframe for {sector}:")
        display(combined_df_example.head())
else:
    print("The 'combined_dfs' dictionary is not available or is empty. Please run the data processing cell first.")

Displaying head of combined dataframes for each sector:

--- Combined dataframe for Banking ---
Columns in the combined dataframe for Banking:
Index(['Logclose_BBL', 'Logclose_KBANK', 'Logclose_KKP', 'Logclose_KTB',
       'Logclose_TCAP', 'Logclose_TISCO', 'Logclose_TTB', 'Close_BBL',
       'Close_KBANK', 'Close_KKP', 'Close_KTB', 'Close_TCAP', 'Close_TISCO',
       'Close_TTB', 'Close_KTC', 'Close_AEONTS', 'Close_JMT', 'Close_SAWAD',
       'Inflation', 'Inflation_Forecast', 'IPI', 'IPI_Forecast',
       'BroadMoney_M1M2_Growth', 'THOR_1M', 'THOR_3M', 'THOR_6M',
       'Bond_Yield_1Y', 'Bond_Yield_5Y', 'Bond_Yield_10Y',
       'Yield_Spread_10Y_5Y', 'Yield_Spread_5Y_1Y', 'Yield_Spread_10Y_1Y',
       'THB_per_USD', 'THB_per_CNY', 'SP500_Index', 'SET_Index',
       'IPI_Surprise', 'Inflation_Surprise', '5Y_1Y_Bond_Spread',
       '10Y_5Y_Bond_Spread', '3M_1M_THOR_Spread', '6M_1M_THOR_Spread',
       'GDP_Monthly', 'GDP_Growth'],
      dtype='object')

Head of the combined dataframe f

,Logclose_BBL,Logclose_KBANK,Logclose_KKP,Logclose_KTB,Logclose_TCAP,Logclose_TISCO,Logclose_TTB,Close_BBL,Close_KBANK,Close_KKP,...,SP500_Index,SET_Index,IPI_Surprise,Inflation_Surprise,5Y_1Y_Bond_Spread,10Y_5Y_Bond_Spread,3M_1M_THOR_Spread,6M_1M_THOR_Spread,GDP_Monthly,GDP_Growth
Date,,,,,,,,,,,,,,,,,,,,,
2015-02-01,4.790190,5.017582,2.939447,2.627824,2.802560,3.072488,0.569241,120.324242,151.045563,18.905380,...,2104.500000,1582.699951,-0.000066,-0.0066,0.0035,0.0056,0.000015,0.000068,1.165382e+06,NaN
2015-03-01,4.798353,5.071406,2.933057,2.627824,2.795391,3.045532,0.555908,121.310509,159.398285,18.784966,...,2067.889893,1582.140015,0.000380,-0.0019,0.0045,0.0058,0.000032,0.000077,1.141786e+06,-0.020455
2015-04-01,4.801060,4.984792,2.880413,2.496796,2.773572,3.034543,0.419496,121.639305,146.173111,17.821629,...,2085.510010,1525.579956,-0.000670,-0.0012,0.0048,0.0051,0.001167,0.001528,1.124720e+06,-0.015059
2015-05-01,4.808427,4.925754,2.888474,2.436026,2.810098,3.072725,0.417810,122.538742,137.793167,17.965874,...,2107.389893,1519.880005,-0.000330,-0.0024,0.0042,0.0078,0.001280,0.002220,1.111753e+06,-0.011596
2015-06-01,4.786204,4.894501,2.867421,2.379193,2.780466,3.094704,0.335797,119.845604,133.553375,17.591583,...,2063.110107,1476.869995,-0.000280,-0.0015,0.0056,0.0082,0.001854,0.003469,1.108570e+06,-0.002867



--- Combined dataframe for Finance_Securities ---
Columns in the combined dataframe for Finance_Securities:
Index(['Logclose_KTC', 'Logclose_AEONTS', 'Logclose_JMT', 'Logclose_SAWAD',
       'Close_BBL', 'Close_KBANK', 'Close_KKP', 'Close_KTB', 'Close_TCAP',
       'Close_TISCO', 'Close_TTB', 'Close_KTC', 'Close_AEONTS', 'Close_JMT',
       'Close_SAWAD', 'Inflation', 'Inflation_Forecast', 'IPI', 'IPI_Forecast',
       'BroadMoney_M1M2_Growth', 'THOR_1M', 'THOR_3M', 'THOR_6M',
       'Bond_Yield_1Y', 'Bond_Yield_5Y', 'Bond_Yield_10Y',
       'Yield_Spread_10Y_5Y', 'Yield_Spread_5Y_1Y', 'Yield_Spread_10Y_1Y',
       'THB_per_USD', 'THB_per_CNY', 'SP500_Index', 'SET_Index',
       'IPI_Surprise', 'Inflation_Surprise', '5Y_1Y_Bond_Spread',
       '10Y_5Y_Bond_Spread', '3M_1M_THOR_Spread', '6M_1M_THOR_Spread',
       'GDP_Monthly', 'GDP_Growth'],
      dtype='object')

Head of the combined dataframe for Finance_Securities:


,Logclose_KTC,Logclose_AEONTS,Logclose_JMT,Logclose_SAWAD,Close_BBL,Close_KBANK,Close_KKP,Close_KTB,Close_TCAP,Close_TISCO,...,SP500_Index,SET_Index,IPI_Surprise,Inflation_Surprise,5Y_1Y_Bond_Spread,10Y_5Y_Bond_Spread,3M_1M_THOR_Spread,6M_1M_THOR_Spread,GDP_Monthly,GDP_Growth
Date,,,,,,,,,,,,,,,,,,,,,
2015-02-01,1.911173,4.324111,1.979026,3.156030,120.324242,151.045563,18.905380,13.843618,16.486799,21.595558,...,2104.500000,1582.699951,-0.000066,-0.0066,0.0035,0.0056,0.000015,0.000068,1.165382e+06,NaN
2015-03-01,2.032421,4.281752,1.914487,3.106020,121.310509,159.398285,18.784966,13.843618,16.369032,21.021212,...,2067.889893,1582.140015,0.000380,-0.0019,0.0045,0.0058,0.000032,0.000077,1.141786e+06,-0.020455
2015-04-01,2.123729,4.252482,1.897821,3.193960,121.639305,146.173111,17.821629,12.143525,16.015741,20.791471,...,2085.510010,1525.579956,-0.000670,-0.0012,0.0048,0.0051,0.001167,0.001528,1.124720e+06,-0.015059
2015-05-01,1.998252,4.206817,1.863420,3.200002,122.538742,137.793167,17.965874,11.427542,16.611540,21.600693,...,2107.389893,1519.880005,-0.000330,-0.0024,0.0042,0.0078,0.001280,0.002220,1.111753e+06,-0.011596
2015-06-01,2.047805,4.201526,1.712987,3.151212,119.845604,133.553375,17.591583,10.796185,16.126530,22.080709,...,2063.110107,1476.869995,-0.000280,-0.0015,0.0056,0.0082,0.001854,0.003469,1.108570e+06,-0.002867


** 3 Econometric **

In [ ]:
# ============================================================
# 🧠 MULTI-STOCK PIPELINE (Logclose + Dynamic Alignment)
# ============================================================

import re
import numpy as np
import pandas as pd

# --- Safety check ---
if 'combined_dfs' not in globals() or not isinstance(combined_dfs, dict) or not combined_dfs:
    raise RuntimeError("❌ combined_dfs not found or empty. Run previous macro/stock preparation cells first.")

if 'SECTORS' not in globals():
    raise RuntimeError("❌ SECTORS not defined. Make sure the sector mapping dict is loaded.")

# ============================================================
# 1️⃣ Helper functions
# ============================================================
def _norm(s: str) -> str:
    return re.sub(r'[^a-z0-9]+', '', str(s).lower())

PRICE_TOKENS = ("logclose","close","adjclose","price","px","pxlast","last")

def _find_price_like_col(df_one: pd.DataFrame, ticker: str) -> str | None:
    """ค้นหาคอลัมน์ราคาหรือ logclose ที่ตรงกับชื่อหุ้น"""
    if df_one is None or df_one.empty:
        return None
    exacts = [
        f"Logclose_{ticker}",
        f"Close_{ticker}",
        f"close_{ticker}",
        f"AdjClose_{ticker}",
        f"Adj_Close_{ticker}"
    ]
    for c in exacts:
        if c in df_one.columns:
            return c
    generics = ["Logclose", "Close", "Adj Close", "Price", "PX", "PX_LAST", "Last"]
    for c in generics:
        if c in df_one.columns:
            return c
    numeric_cols = [c for c in df_one.columns if pd.api.types.is_numeric_dtype(df_one[c])]
    want = {_norm(t)+_norm(ticker) for t in PRICE_TOKENS}
    best, best_nn = None, -1
    for c in numeric_cols:
        nc = _norm(c)
        if any(tok in nc for tok in want) or any(t in nc for t in PRICE_TOKENS):
            nn = df_one[c].notna().sum()
            if nn > best_nn:
                best, best_nn = c, nn
    return best

def _align(s: pd.Series, idx: pd.Index) -> pd.Series:
    if s is None:
        return None
    s = s.copy()
    s.index = pd.to_datetime(s.index)
    return s.reindex(idx)

# ============================================================
# 2️⃣ Build base index (union of all sectors)
# ============================================================
union_idx = None
for d in combined_dfs.values():
    idx = pd.to_datetime(d.index)
    union_idx = idx if union_idx is None else union_idx.union(idx)
if 'df' in globals() and isinstance(df, pd.DataFrame) and not df.empty:
    union_idx = (pd.to_datetime(df.index) if union_idx is None else union_idx.union(pd.to_datetime(df.index)))
if union_idx is None:
    raise RuntimeError("❌ No index found to build panel.")
union_idx = union_idx.sort_values()

# Base DataFrame
df_master = (df.reindex(union_idx).copy()
             if 'df' in globals() and isinstance(df, pd.DataFrame)
             else pd.DataFrame(index=union_idx))

# ============================================================
# 3️⃣ Collect Logclose per ticker
# ============================================================
TARGET_TICKERS = [ticker for tickers in SECTORS.values() for ticker in tickers]
source_used, missing = {}, []

for ticker in TARGET_TICKERS:
    dsec = None
    for sec, d in combined_dfs.items():
        if f"Logclose_{ticker}" in d.columns or f"Close_{ticker}" in d.columns:
            dsec = d
            break

    price, src, col_used = None, None, None
    if dsec is not None:
        pcol = _find_price_like_col(dsec, ticker)
        if pcol is not None:
            price = _align(pd.to_numeric(dsec[pcol], errors='coerce'), union_idx)
            src, col_used = 'combined', pcol

    if price is None and 'df' in globals() and isinstance(df, pd.DataFrame):
        pcol_g = _find_price_like_col(df, ticker)
        if pcol_g is not None:
            price = _align(pd.to_numeric(df[pcol_g], errors='coerce'), union_idx)
            src, col_used = 'global', pcol_g

    if price is None:
        missing.append(ticker)
        source_used[ticker] = None
        continue

    logc = f"Logclose_{ticker}"
    df_master[logc] = price.astype(float)
    source_used[ticker] = (src, col_used)

print("=== ✅ Logclose columns collected ===")
print(f"Total: {len([c for c in df_master.columns if 'Logclose_' in c])} stocks")

# ============================================================
# 4️⃣ Dynamic Start Date per Ticker
# ============================================================
logclose_cols = [f"Logclose_{t}" for t in TARGET_TICKERS if f"Logclose_{t}" in df_master.columns]

first_valid_dates = {}
for c in logclose_cols:
    valid = df_master[c].first_valid_index()
    first_valid_dates[c] = valid

print("\n📅 First available data per stock:")
for k, v in first_valid_dates.items():
    print(f"{k:25s}: {v}")

# --- สร้าง Dynamic Panel ---
df_dynamic = pd.DataFrame(index=df_master.index)
for c in logclose_cols:
    start_date = first_valid_dates[c]
    if start_date is not None:
        tmp = df_master.loc[start_date:, c]
        df_dynamic[c] = tmp

# --- เติมค่าหลังเริ่มต้นด้วย ffill 1 step (เพื่อความต่อเนื่องเล็กน้อย) ---
df_dynamic = df_dynamic.sort_index().ffill(limit=1)

# ============================================================
# 5️⃣ Final Panel
# ============================================================
df = df_dynamic.copy()

print(f"\n✅ Dynamic Panel Ready: shape={df.shape}")
print(f"🕐 Date range: {df.index.min().date()} → {df.index.max().date()}")
if missing:
    print(f"\n⚠️ Missing Logclose for: {missing}")

# --- Preview ---
try:
    from IPython.display import display
    display(df.iloc[:10, :6])
except Exception:
    print(df.iloc[:10, :6])


=== ✅ Logclose columns collected ===
Total: 11 stocks

📅 First available data per stock:
Logclose_BBL             : 2015-02-01 00:00:00
Logclose_KBANK           : 2015-02-01 00:00:00
Logclose_KKP             : 2015-02-01 00:00:00
Logclose_KTB             : 2015-02-01 00:00:00
Logclose_TCAP            : 2015-02-01 00:00:00
Logclose_TISCO           : 2015-02-01 00:00:00
Logclose_TTB             : 2015-02-01 00:00:00
Logclose_KTC             : 2015-02-01 00:00:00
Logclose_AEONTS          : 2015-02-01 00:00:00
Logclose_JMT             : 2015-02-01 00:00:00
Logclose_SAWAD           : 2015-02-01 00:00:00

✅ Dynamic Panel Ready: shape=(132, 11)
🕐 Date range: 2015-02-01 → 2026-01-01


,Logclose_BBL,Logclose_KBANK,Logclose_KKP,Logclose_KTB,Logclose_TCAP,Logclose_TISCO
Date,,,,,,
2015-02-01,4.790190,5.017582,2.939447,2.627824,2.802560,3.072488
2015-03-01,4.798353,5.071406,2.933057,2.627824,2.795391,3.045532
2015-04-01,4.801060,4.984792,2.880413,2.496796,2.773572,3.034543
2015-05-01,4.808427,4.925754,2.888474,2.436026,2.810098,3.072725
2015-06-01,4.786204,4.894501,2.867421,2.379193,2.780466,3.094704
2015-07-01,4.716408,4.834538,2.722682,2.402315,2.718430,3.015567
2015-08-01,4.707331,4.848485,2.778473,2.441536,2.660801,2.942364
2015-09-01,4.679594,4.794418,2.730845,2.379193,2.734304,2.835596
2015-10-01,4.737904,4.805956,2.846806,2.379193,2.787956,2.897049


In [ ]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.3/981.3 kB 12.1 MB/s eta 0:00:00


In [ ]:
# ============================================================
# ⚙️ Build Selectively-Differenced Panel (Diff only I(1))
# ============================================================

import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller

# --- Check availability ---
if 'combined_dfs' not in globals() or not isinstance(combined_dfs, dict) or not combined_dfs:
    raise RuntimeError("❌ combined_dfs not found or empty. Please run previous block first.")

# --- Helper function: classify I(0) vs I(1) ---
def classify_order(series, signif=0.05):
    series = series.dropna()
    if len(series) < 10:
        return None
    try:
        adf_result = adfuller(series, autolag='AIC')
        pval = adf_result[1]
        return "I(0)" if pval < signif else "I(1)"
    except Exception:
        return None

# --- Detect integration order for all macros ---
macro_candidates = [
    "BroadMoney_M1M2_Growth", "Inflation_Surprise", "IPI_Surprise",
    "THOR_1M", "THOR_6M", "3M_1M_THOR_Spread", "6M_1M_THOR_Spread",
    "5Y_1Y_Bond_Spread", "10Y_5Y_Bond_Spread",
    "THB_per_USD", "SET_Index",
    "SP500_Index", "THB_per_CNY"
]

macro_orders = {}
for sector, df_sec in combined_dfs.items():
    for m in macro_candidates:
        if m in df_sec.columns and m not in macro_orders:
            order = classify_order(df_sec[m])
            if order:
                macro_orders[m] = order

print("=== Integration Order Detected ===")
for k, v in macro_orders.items():
    print(f"{k:25s} → {v}")

# --- Build differenced dataset selectively ---
combined_dfs_selective_diff = {}

for sector, df_sec in combined_dfs.items():
    df_sec = df_sec.copy()
    out = pd.DataFrame(index=df_sec.index)

    # 1️⃣ หุ้น: ใช้ Δlog เสมอ
    for col in df_sec.columns:
        if col.startswith("Logclose_"):
            out[col] = df_sec[col].diff()

    # 2️⃣ Macro: diff เฉพาะ I(1)
    for m in macro_candidates:
        if m not in df_sec.columns:
            continue
        if macro_orders.get(m, "I(1)") == "I(1)":
            out[m] = df_sec[m].diff()
        else:  # I(0)
            out[m] = df_sec[m]

    # 3️⃣ Drop NA
    combined_dfs_selective_diff[sector] = out.dropna(how='all')

print("\n✅ Built 'combined_dfs_selective_diff' successfully.")
print("Diffed Macros (I(1)):", [k for k, v in macro_orders.items() if v == "I(1)"])
print("Kept in Levels (I(0)):", [k for k, v in macro_orders.items() if v == "I(0)"])


=== Integration Order Detected ===
BroadMoney_M1M2_Growth    → I(0)
Inflation_Surprise        → I(0)
IPI_Surprise              → I(0)
THOR_1M                   → I(1)
THOR_6M                   → I(1)
3M_1M_THOR_Spread         → I(1)
6M_1M_THOR_Spread         → I(1)
5Y_1Y_Bond_Spread         → I(0)
10Y_5Y_Bond_Spread        → I(1)
THB_per_USD               → I(1)
SET_Index                 → I(1)
SP500_Index               → I(1)
THB_per_CNY               → I(1)

✅ Built 'combined_dfs_selective_diff' successfully.
Diffed Macros (I(1)): ['THOR_1M', 'THOR_6M', '3M_1M_THOR_Spread', '6M_1M_THOR_Spread', '10Y_5Y_Bond_Spread', 'THB_per_USD', 'SET_Index', 'SP500_Index', 'THB_per_CNY']
Kept in Levels (I(0)): ['BroadMoney_M1M2_Growth', 'Inflation_Surprise', 'IPI_Surprise', '5Y_1Y_Bond_Spread']


In [ ]:
# ============================================================
# 🔍 CHECK DATA CONSISTENCY BEFORE ARDL (Selective Diff Version)
# ============================================================

import pandas as pd

print("=== Checking macro variable presence in combined_dfs_selective_diff ===\n")

# List ของตัวแปรที่ ARDL ต้องใช้ (ปรับให้ตรงกับ macro_vars จริง)
macro_vars = [
    "BroadMoney_M1M2_Growth", "Inflation_Surprise", "IPI_Surprise",
    "THOR_6M", "3M_1M_THOR_Spread",
    "5Y_1Y_Bond_Spread", "10Y_5Y_Bond_Spread",
    "THB_per_USD", "SET_Index",
    "SP500_Index", "THB_per_CNY"
]

summary_records = []

for sector, df_sec in combined_dfs_selective_diff.items():
    present = [m for m in macro_vars if m in df_sec.columns]
    missing = [m for m in macro_vars if m not in df_sec.columns]
    logclose_cols = [c for c in df_sec.columns if c.startswith("Logclose_")]

    print(f"📂 Sector: {sector}")
    print(f"  • Stocks found: {len(logclose_cols)} ({', '.join(logclose_cols[:5])}{'...' if len(logclose_cols)>5 else ''})")
    print(f"  • Macros present ({len(present)}): {present}")
    print(f"  • Macros missing ({len(missing)}): {missing}")
    print("-" * 80)

    summary_records.append({
        "Sector": sector,
        "N_stocks": len(logclose_cols),
        "Macros_present": len(present),
        "Macros_missing": len(missing),
        "Missing_list": ", ".join(missing)
    })

# รวมเป็นตารางภาพรวม
df_summary = pd.DataFrame(summary_records)
print("\n=== Summary of Macro Coverage per Sector ===")
display(df_summary)

# # ตัวอย่าง dataframe จริง
# sector_to_inspect = list(combined_dfs_selective_diff.keys())[0]
# print(f"\n\n=== Example: head() of combined_dfs_selective_diff['{sector_to_inspect}'] ===")
# display(combined_dfs_selective_diff[sector_to_inspect].head(10))

# ============================================================
# 🧹 CLEAN: Drop first NaN row from all selective-diff dataframes
# ============================================================

for sector in combined_dfs_selective_diff:
    df_tmp = combined_dfs_selective_diff[sector].copy()

    # ลบแถวแรกที่เป็น NaN ทั้งหมด
    df_tmp = df_tmp.dropna(how="all")

    # ลบ NaN เฉพาะคอลัมน์หลัก (ถ้าบาง macro เริ่มช้า)
    df_tmp = df_tmp.dropna(thresh=int(0.8 * len(df_tmp.columns)))  # อย่างน้อย 80% ต้องไม่ NA

    combined_dfs_selective_diff[sector] = df_tmp

print("✅ Removed first NaN row and cleaned selective-diff dataframes successfully.\n")

# ตรวจสอบตัวอย่างหลัง clean
sector_to_inspect = list(combined_dfs_selective_diff.keys())[0]
print(f"=== Example after cleaning: '{sector_to_inspect}' ===")
display(combined_dfs_selective_diff[sector_to_inspect].head(10))



=== Checking macro variable presence in combined_dfs_selective_diff ===

📂 Sector: Banking
  • Stocks found: 7 (Logclose_BBL, Logclose_KBANK, Logclose_KKP, Logclose_KTB, Logclose_TCAP...)
  • Macros present (11): ['BroadMoney_M1M2_Growth', 'Inflation_Surprise', 'IPI_Surprise', 'THOR_6M', '3M_1M_THOR_Spread', '5Y_1Y_Bond_Spread', '10Y_5Y_Bond_Spread', 'THB_per_USD', 'SET_Index', 'SP500_Index', 'THB_per_CNY']
  • Macros missing (0): []
--------------------------------------------------------------------------------
📂 Sector: Finance_Securities
  • Stocks found: 4 (Logclose_KTC, Logclose_AEONTS, Logclose_JMT, Logclose_SAWAD)
  • Macros present (11): ['BroadMoney_M1M2_Growth', 'Inflation_Surprise', 'IPI_Surprise', 'THOR_6M', '3M_1M_THOR_Spread', '5Y_1Y_Bond_Spread', '10Y_5Y_Bond_Spread', 'THB_per_USD', 'SET_Index', 'SP500_Index', 'THB_per_CNY']
  • Macros missing (0): []
--------------------------------------------------------------------------------

=== Summary of Macro Coverage per Sect

,Sector,N_stocks,Macros_present,Macros_missing,Missing_list
0,Banking,7,11,0,
1,Finance_Securities,4,11,0,


✅ Removed first NaN row and cleaned selective-diff dataframes successfully.

=== Example after cleaning: 'Banking' ===


,Logclose_BBL,Logclose_KBANK,Logclose_KKP,Logclose_KTB,Logclose_TCAP,Logclose_TISCO,Logclose_TTB,BroadMoney_M1M2_Growth,Inflation_Surprise,IPI_Surprise,THOR_1M,THOR_6M,3M_1M_THOR_Spread,6M_1M_THOR_Spread,5Y_1Y_Bond_Spread,10Y_5Y_Bond_Spread,THB_per_USD,SET_Index,SP500_Index,THB_per_CNY
Date,,,,,,,,,,,,,,,,,,,,
2015-03-01,0.008163,0.053824,-0.006390,0.000000,-0.007169,-0.026956,-0.013334,0.004617,-0.0019,0.000380,-1.260000e-05,-0.000004,1.670000e-05,0.000009,0.00450,0.00020,0.199001,-0.559937,-36.610107,-0.00295
2015-04-01,0.002707,-0.086614,-0.052644,-0.131028,-0.021819,-0.010989,-0.136412,-0.001550,-0.0012,-0.000670,-1.740800e-03,-0.000290,1.135300e-03,0.001451,0.00480,-0.00070,0.298000,-56.560059,17.620117,-0.00213
2015-05-01,0.007367,-0.059038,0.008061,-0.060770,0.036526,0.038183,-0.001686,0.001422,-0.0024,-0.000330,-1.272900e-03,-0.000581,1.132000e-04,0.000692,0.00420,0.00270,0.896000,-5.699951,21.879883,-0.00476
2015-06-01,-0.022223,-0.031253,-0.021053,-0.056834,-0.029632,0.021979,-0.082013,-0.005075,-0.0015,-0.000280,-1.988700e-03,-0.000740,5.740000e-04,0.001249,0.00560,0.00040,-0.005001,-43.010010,-44.279785,0.00008
2015-07-01,-0.069796,-0.059963,-0.144739,0.023122,-0.062035,-0.079137,0.000000,-0.001889,-0.0007,-0.000285,-4.200000e-06,-0.000885,-1.082600e-03,-0.000881,0.00630,-0.00190,1.361000,14.750000,40.729980,-0.00704
2015-08-01,-0.009077,0.013947,0.055791,0.039221,-0.057629,-0.073204,0.008511,-0.002574,-0.0007,0.000045,-9.100000e-06,-0.000853,-7.473000e-04,-0.000844,0.00530,-0.00010,0.632999,-49.579956,-131.660034,0.00150
2015-09-01,-0.027737,-0.054068,-0.047628,-0.062343,0.073503,-0.106768,0.025106,0.003077,0.0001,-0.000200,4.000000e-07,-0.000763,-3.900000e-06,-0.000763,0.00575,-0.00085,0.674000,-79.650024,-52.150024,-0.00361
2015-10-01,0.058310,0.011539,0.115961,0.000000,0.053653,0.061453,0.094558,0.000000,0.0001,-0.000200,0.000000e+00,0.000000,0.000000e+00,0.000000,0.00575,0.00000,-0.790001,-17.239990,159.330078,0.00263
2015-11-01,0.000000,-0.008733,0.085767,-0.005865,0.073214,0.106684,-0.022814,0.015938,0.0017,-0.000042,1.400000e-06,-0.000930,-9.000000e-07,-0.000932,0.00530,0.00105,0.284000,68.189941,1.049805,-0.00052


# ARDL + ECM

In [ ]:
# ============================================================
# ✅ ARDL+ECM (HAC Robust, Fixed lag=3, Final v4 – Grouped Lags)
# ============================================================

import re
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.ardl import ARDL
import warnings
warnings.filterwarnings("ignore")

# ---------- CONFIG ----------
LAG_Y = 3
LAG_X = 3
MIN_OBS = 40

macro_vars = [
    "BroadMoney_M1M2_Growth", "Inflation_Surprise", "IPI_Surprise",
    "THOR_6M", "3M_1M_THOR_Spread",
    "5Y_1Y_Bond_Spread", "10Y_5Y_Bond_Spread",
    "THB_per_USD", "SET_Index",
    "SP500_Index", "THB_per_CNY"
]

alpha_list, beta_list, gamma_list, diag_list = [], [], [], []

# ============================================================
# 🧩 HELPER FUNCTIONS
# ============================================================

def get_param_names(res):
    if hasattr(res.model, "_xnames"):
        return res.model._xnames
    elif hasattr(res.model, "data") and hasattr(res.model.data, "param_names"):
        return res.model.data.param_names
    elif hasattr(res, "params"):
        return list(res.params.index)
    else:
        raise AttributeError("Cannot extract param names from model object.")

def _nw_lags(T: int) -> int:
    return max(1, int(np.floor(4 * (T / 100.0) ** (2.0 / 9.0))))

def _is_endog_lag(name: str, endog: str) -> bool:
    return bool(
        re.search(rf"(?:^L\d+\.){re.escape(endog)}$", name)
        or re.search(rf"^{re.escape(endog)}\.L\d+$", name)
    )

def _is_exog_term(name: str, var: str) -> bool:
    return bool(
        name == var
        or re.search(rf"^L\d+\.{re.escape(var)}$", name)
        or re.search(rf"^{re.escape(var)}\.L\d+$", name)
    )

def compute_long_run_from_ardl(res, endog_name: str, exog_list: list) -> pd.Series:
    params = res.params
    names = get_param_names(res)

    phi_sum = np.sum([params[n] for n in names if _is_endog_lag(n, endog_name)])
    denom = 1 - phi_sum if abs(1 - phi_sum) > 1e-8 else np.nan

    const = params.get("const", 0.0)
    lr = {"const": (const / denom) if denom else np.nan}

    for v in exog_list:
        theta_sum = np.sum([params[n] for n in names if _is_exog_term(n, v) and n in params])
        lr[v] = (theta_sum / denom) if denom else np.nan

    return pd.Series(lr)

def build_ecm_design(df_model: pd.DataFrame, y_col: str, x_cols: list, p: int, q: int):
    """สร้าง ECM design matrix พร้อมตั้งชื่อชัดเจน"""
    dy = df_model[y_col].diff()
    dy.name = f"D.{y_col}"

    # lag Δy
    Dy_lags = []
    for l in range(1, p):
        s = df_model[y_col].diff().shift(l)
        s.name = f"D.L{l}.{y_col}"
        Dy_lags.append(s)

    # lag Δx
    Dx_cols = []
    for v in x_cols:
        s0 = df_model[v].diff()
        s0.name = f"D.{v}"
        Dx_cols.append(s0)
        for l in range(1, q):
            s = df_model[v].diff().shift(l)
            s.name = f"D.L{l}.{v}"
            Dx_cols.append(s)

    X = pd.concat(
        [pd.Series(1.0, index=df_model.index, name="const")] + Dy_lags + Dx_cols,
        axis=1
    )
    return dy, X

def star(p):
    if p < 0.01: return "***"
    elif p < 0.05: return "**"
    elif p < 0.1: return "*"
    return ""

def sort_by_base_and_lag(varname: str) -> tuple:
    """จัดเรียงตัวแปร Δ ให้เรียง lag ต่อกัน (เหมือน EViews)"""
    name = varname.replace("D.", "")
    lag_match = re.search(r"L(\d+)\.", name)
    if lag_match:
        lag = int(lag_match.group(1))
        base = re.sub(r"L\d+\.", "", name)
    else:
        lag = 0
        base = name
    return (base, lag)

def eviews_summary_by_sector(df_alpha, df_beta, df_gamma):
    sectors = sorted(df_alpha["Sector"].unique())
    for sector in sectors:
        print("\n" + "="*88)
        print(f"📂 Sector: {sector}")
        print("="*88)

        df_a = df_alpha[df_alpha["Sector"] == sector]
        df_b = df_beta[df_beta["Sector"] == sector]
        df_g = df_gamma[df_gamma["Sector"] == sector]

        for stock in sorted(df_a["Stock"].unique()):
            print(f"\n--- {stock} ---")
            a_row = df_a[df_a["Stock"] == stock].iloc[0]
            alpha_val = a_row["Alpha"]; pval = a_row["pval"]
            print(f"α (ECT speed): {alpha_val:.4f}{star(pval)}   p={pval:.3f}")

            btab = df_b[df_b["Stock"] == stock][["Variable", "coef"]]
            if not btab.empty:
                btab = btab.rename(columns={"coef": "β (Long-run)"})
                print("\nLong-run Relationship (β):")
                print(btab.to_string(index=False, float_format=lambda x: f"{x:.6f}"))

            gtab = df_g[df_g["Stock"] == stock][["Variable", "Coef", "pval"]]
            if not gtab.empty:
                gtab = gtab[~gtab["Variable"].str.contains("ECT", na=False)]
                gtab["sort_key"] = gtab["Variable"].apply(sort_by_base_and_lag)
                gtab = gtab.sort_values("sort_key").drop(columns=["sort_key"])
                gtab["Signif"] = gtab["pval"].apply(star)
                print("\nShort-run Dynamics (γ):")
                print(gtab.to_string(index=False, float_format=lambda x: f"{x:8.4f}"))

            print("-"*88)

# ============================================================
# 🚀 MAIN LOOP
# ============================================================

for sector, df_sec in combined_dfs_selective_diff.items():
    print(f"\n===== 📂 Sector: {sector} =====")
    if df_sec.empty:
        continue

    x_list = [v for v in macro_vars if v in df_sec.columns]
    if not x_list:
        print("⚠️ No macro variables in this sector.")
        continue

    for col in df_sec.columns:
        if not col.startswith("Logclose_"):
            continue

        stock = col.replace("Logclose_", "")
        y = df_sec[col]
        X = df_sec[x_list]
        df_model = pd.concat([y, X], axis=1).dropna()

        if len(df_model) < MIN_OBS:
            print(f"⚠️ Skip {stock}: insufficient data ({len(df_model)})")
            continue

        try:
            print(f"⏳ ARDL(p=3,q=3) fit: {stock} ({len(df_model)} obs, {len(x_list)} Xs)...", flush=True)
            res = ARDL(df_model[col], LAG_Y, df_model[x_list], LAG_X, trend='c').fit()

            lr_ser = compute_long_run_from_ardl(res, col, x_list)

            ect = df_model[col].shift(1) - (
                lr_ser.get("const", 0.0) + sum(lr_ser[v] * df_model[v].shift(1) for v in x_list)
            )
            ect.name = "ECT_lag1"

            dy, X_ecm = build_ecm_design(df_model, col, x_list, LAG_Y, LAG_X)
            X_ecm = pd.concat([X_ecm, ect], axis=1)
            ecm_df = pd.concat([dy.rename("D.y"), X_ecm], axis=1).dropna()

            hac_bw = _nw_lags(len(ecm_df))
            ecm = sm.OLS(ecm_df["D.y"], ecm_df.drop(columns=["D.y"])) \
                     .fit(cov_type="HAC", cov_kwds={"maxlags": hac_bw})

            beta_df = lr_ser.rename_axis("Variable").reset_index(name="coef")
            beta_df["Stock"], beta_df["Sector"] = stock, sector
            beta_list.append(beta_df)

            alpha_list.append({
                "Stock": stock, "Sector": sector,
                "Alpha": ecm.params.get("ECT_lag1", np.nan),
                "pval": ecm.pvalues.get("ECT_lag1", np.nan),
                "R2": ecm.rsquared
            })

            g = pd.DataFrame({
                "Variable": ecm.params.index,
                "Coef": ecm.params.values,
                "pval": ecm.pvalues.values
            })
            g["Stock"], g["Sector"] = stock, sector
            gamma_list.append(g)

            diag_list.append({
                "Stock": stock, "Sector": sector,
                "R2": ecm.rsquared, "AdjR2": ecm.rsquared_adj
            })

            print(f"✅ {stock}: α={ecm.params.get('ECT_lag1', np.nan):.4f}, "
                  f"p={ecm.pvalues.get('ECT_lag1', np.nan):.3f}, R²={ecm.rsquared:.3f}")

        except Exception as e:
            print(f"⚠️ {stock} failed: {e}")

# ============================================================
# 💾 EXPORT RESULTS + PRINT
# ============================================================

if beta_list:
    df_alpha = pd.DataFrame(alpha_list)
    df_beta  = pd.concat(beta_list, ignore_index=True)
    df_gamma = pd.concat(gamma_list, ignore_index=True)
    df_diag  = pd.DataFrame(diag_list)

    with pd.ExcelWriter("Stock_ARDL_ECM_FixedLag3_GROUPED.xlsx") as w:
        df_alpha.to_excel(w, "Alpha_Speed(ECT)", index=False)
        df_beta.to_excel(w, "Beta_LongRun", index=False)
        df_gamma.to_excel(w, "ShortRun(ECM)", index=False)
        df_diag.to_excel(w, "Diagnostics", index=False)

    print("\n✅ Exported → Stock_ARDL_ECM_FixedLag3_GROUPED.xlsx")
    eviews_summary_by_sector(df_alpha, df_beta, df_gamma)
else:
    print("\n⚠️ No valid results to export/print.")



===== 📂 Sector: Banking =====
⏳ ARDL(p=3,q=3) fit: BBL (119 obs, 11 Xs)...
✅ BBL: α=-1.4067, p=0.000, R²=0.785
⏳ ARDL(p=3,q=3) fit: KBANK (119 obs, 11 Xs)...
✅ KBANK: α=-1.2785, p=0.000, R²=0.749
⏳ ARDL(p=3,q=3) fit: KKP (119 obs, 11 Xs)...
✅ KKP: α=-1.2168, p=0.000, R²=0.780
⏳ ARDL(p=3,q=3) fit: KTB (119 obs, 11 Xs)...
✅ KTB: α=-1.1768, p=0.000, R²=0.712
⏳ ARDL(p=3,q=3) fit: TCAP (119 obs, 11 Xs)...
✅ TCAP: α=-1.3892, p=0.000, R²=0.811
⏳ ARDL(p=3,q=3) fit: TISCO (119 obs, 11 Xs)...
✅ TISCO: α=-1.1359, p=0.000, R²=0.768
⏳ ARDL(p=3,q=3) fit: TTB (119 obs, 11 Xs)...
✅ TTB: α=-1.4532, p=0.000, R²=0.766

===== 📂 Sector: Finance_Securities =====
⏳ ARDL(p=3,q=3) fit: KTC (119 obs, 11 Xs)...
✅ KTC: α=-0.8296, p=0.000, R²=0.702
⏳ ARDL(p=3,q=3) fit: AEONTS (119 obs, 11 Xs)...
✅ AEONTS: α=-1.1432, p=0.000, R²=0.725
⏳ ARDL(p=3,q=3) fit: JMT (119 obs, 11 Xs)...
✅ JMT: α=-0.9830, p=0.000, R²=0.550
⏳ ARDL(p=3,q=3) fit: SAWAD (119 obs, 11 Xs)...
✅ SAWAD: α=-1.3086, p=0.000, R²=0.646

✅ Exported → St

# Forecast Return from ARDL + ECM

In [ ]:
import pandas as pd
import numpy as np
import re

# ============================================================
# ✅ Forecast next-month ΔLogclose (ECM Full, Simplified)
#   - ใช้เฉพาะ lagged Δ + ECT
#   - ไม่คำนวณ/ไม่คืนค่า Logclose_next และ Close_next
# ============================================================

def forecast_next_month_full(
    stock_name, sector_name,
    df_macro_levels,      # df ระดับ (levels) สำหรับ X เพื่อคำนวณ ΔX lag และ ECT
    df_gamma, df_alpha, df_beta,
    combined_dfs_selective_diff  # ใช้ดึง Logclose(series) ของหุ้น (ระดับ log)
):
    # --- ดึงพารามิเตอร์ ---
    gtab = df_gamma[df_gamma["Stock"] == stock_name].copy()
    atab = df_alpha[df_alpha["Stock"] == stock_name]
    btab = df_beta[df_beta["Stock"] == stock_name]
    if gtab.empty or atab.empty or btab.empty:
        return None

    # --- กรองเฉพาะตัวแปรที่เป็น lagged Δ และเพิ่ม ECT ---
    gtab = gtab[gtab["Variable"].str.contains(r"D\.L\d+\.", regex=True, na=False)]
    gtab = pd.concat(
        [gtab, pd.DataFrame([{"Variable": "ECT_lag1", "Coef": float(atab["Alpha"].iloc[0])}])],
        ignore_index=True
    )

    # --- เตรียม series Logclose ของหุ้น (ระดับ log) ---
    df_stock = combined_dfs_selective_diff[sector_name][[f"Logclose_{stock_name}"]].copy()
    df_stock.columns = ["Logclose"]

    # --- เตรียม macro (levels) ให้ index สอดคล้องกับหุ้น ---
    df_macro_aligned = df_macro_levels.reindex(df_stock.index).ffill().copy()

    # --- Long-run coefficients สำหรับ ECT ---
    lr_params = btab.drop_duplicates(subset="Variable").set_index("Variable")["coef"]

    # ใช้ 3 แถวสุดท้ายเพื่อคำนวณ Δ และ lag
    last_row = df_macro_aligned.tail(3).copy()
    deltas = {}

    # ---------- คำนวณ Δ lag และ ECT ----------
    for var in gtab["Variable"]:
        if var == "ECT_lag1":
            const = float(lr_params.get("const", 0.0))
            X_lag = {
                k: float(last_row[k].iloc[-2])
                for k in lr_params.index
                if k != "const" and k in last_row.columns
            }
            Y_lag = float(df_stock["Logclose"].iloc[-2])
            ect_val = Y_lag - (const + sum(float(lr_params[k]) * X_lag[k] for k in X_lag))
            deltas[var] = float(ect_val)
            continue

        # ตัวอย่างชื่อ: "D.L1.THOR_6M" -> lag=1, base="THOR_6M"
        parts = var.split(".")
        lag = int(parts[1].replace("L", ""))
        base = parts[2]

        if base.startswith("Logclose_"):  # ΔY lag
            stock_base = base.replace("Logclose_", "")
            if stock_base != stock_name:
                continue
            # ΔY_t-lag = logY_{t-lag} - logY_{t-lag-1}
            if len(df_stock) > lag + 1:
                dy = float(df_stock["Logclose"].iloc[-lag]) - float(df_stock["Logclose"].iloc[-(lag+1)])
            else:
                dy = 0.0
            deltas[var] = dy
        else:  # ΔX lag จาก levels
            if base in df_macro_aligned.columns and len(last_row) > lag + 1:
                dx = float(last_row[base].iloc[-lag]) - float(last_row[base].iloc[-(lag+1)])
            else:
                dx = 0.0
            deltas[var] = dx

    # ---------- รวม contribution เป็น ΔŶ ----------
    df_pred = pd.DataFrame(list(deltas.items()), columns=["Variable", "Value"])
    merged = pd.merge(gtab[["Variable", "Coef"]], df_pred, on="Variable", how="left")
    merged["Contribution"] = merged["Coef"].astype(float) * merged["Value"].astype(float)

    pred_dy = float(merged["Contribution"].sum())

    return {
        "Stock": stock_name,
        "Sector": sector_name,
        "Pred_dLogclose": pred_dy,  # <-- คืนค่าเฉพาะ ΔŶ
        "Details": merged           # ตารางองค์ประกอบ (ถ้าต้องการ inspect)
    }


# ============================================================
# 🚀 Run for ALL stocks (Clean Output)
# ============================================================

forecast_results = []

for sector_name, df_sec in combined_dfs_selective_diff.items():
    for col in df_sec.columns:
        if not col.startswith("Logclose_"):
            continue
        stock_name = col.replace("Logclose_", "")
        try:
            res = forecast_next_month_full(
                stock_name, sector_name,
                df,            # levels macro (ตัวเดียวกับที่ใช้สร้างชุดข้อมูล)
                df_gamma, df_alpha, df_beta,
                combined_dfs_selective_diff
            )
            if res:
                forecast_results.append({
                    "Stock": stock_name,
                    "Sector": sector_name,
                    "Pred_dLogclose": res["Pred_dLogclose"],
                })
                print(f"✅ {stock_name:<8} | Sector={sector_name:<20} | ΔŶ={res['Pred_dLogclose']:+.5f}")
        except Exception as e:
            print(f"⚠️ {stock_name} failed: {e}")

# ตารางสรุป CLEAN (ไม่มี Logclose_next / Close_next)
df_forecast_summary = pd.DataFrame(forecast_results)
df_forecast_summary["Expected_Return_%"] = df_forecast_summary["Pred_dLogclose"] * 100.0
df_forecast_summary["Signal"] = np.where(df_forecast_summary["Pred_dLogclose"] > 0, "BUY", "SELL")

# แสดงผลและบันทึก
display(df_forecast_summary)
df_forecast_summary.to_excel("Forecast_ARDL_ECM_NextMonth_Clean.xlsx", index=False)
print("\n✅ Exported → Forecast_ARDL_ECM_NextMonth_Clean.xlsx")




✅ BBL      | Sector=Banking              | ΔŶ=+0.02543
✅ KBANK    | Sector=Banking              | ΔŶ=+0.05695
✅ KKP      | Sector=Banking              | ΔŶ=-0.03251
✅ KTB      | Sector=Banking              | ΔŶ=+0.00309
✅ TCAP     | Sector=Banking              | ΔŶ=+0.00953
✅ TISCO    | Sector=Banking              | ΔŶ=+0.02024
✅ TTB      | Sector=Banking              | ΔŶ=-0.06937
✅ KTC      | Sector=Finance_Securities   | ΔŶ=+0.25061
✅ AEONTS   | Sector=Finance_Securities   | ΔŶ=-0.08836
✅ JMT      | Sector=Finance_Securities   | ΔŶ=+0.05498
✅ SAWAD    | Sector=Finance_Securities   | ΔŶ=-0.09972


,Stock,Sector,Pred_dLogclose,Expected_Return_%,Signal
0,BBL,Banking,0.025432,2.543223,BUY
1,KBANK,Banking,0.056954,5.695436,BUY
2,KKP,Banking,-0.032508,-3.250787,SELL
3,KTB,Banking,0.003092,0.309182,BUY
4,TCAP,Banking,0.009530,0.953031,BUY
5,TISCO,Banking,0.020245,2.024487,BUY
6,TTB,Banking,-0.069369,-6.936894,SELL
7,KTC,Finance_Securities,0.250606,25.060636,BUY
8,AEONTS,Finance_Securities,-0.088355,-8.835528,SELL
9,JMT,Finance_Securities,0.054977,5.497665,BUY



✅ Exported → Forecast_ARDL_ECM_NextMonth_Clean.xlsx


# Technical Indicators

In [ ]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=a2fd65c50b94cec50297d7758f8db15f05c05328f3d9699246df5d549e4e6abc
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [ ]:
# ============================================================
# FIXED BLOCK — Daily Indicators + Pattern + Wave (SECTOR version)
# ============================================================

import pandas as pd
import numpy as np
import ta
import yfinance as yf
from scipy.signal import argrelextrema
from scipy.stats import skew, kurtosis
from pathlib import Path

# ---------- SECTORS ----------
SECTORS = {
    "Banking": ["BBL","KBANK","KKP","KTB","TCAP","TISCO","TTB"],
    "Finance_Securities": ["KTC","AEONTS","JMT","SAWAD"],
    # "Energy_Utilities": ["BANPU","BCP","EGCO","GPSC","GULF","OR","PTT","PTTEP","RATCH","TOP","BGRIM","BCPG","EA","GUNKUL","IRPC","SPRC","WHAUP","CKP"],
    # "Food_Beverage": ["CBG","CPF","OSP","TU","ITC","BTG","ICHI","M","TFG","COCOCO","SAPPE","SNNP"],
    # "Commerce": ["BJC","COM7","CPALL","CRC","HMPRO","GLOBAL","DOHOME","MEGA","MOSHI"],
    # "Property_Development": ["AWC","CPN","LH","WHA","AMATA","AP","MBK","QH","SIRI","SPALI","ROJNA"],
    # "Health_Care_Services": ["BDMS","BH","BCH","CHG","PR9"],
    # "Electronic_Components": ["CCET","DELTA","HANA","KCE"],
    # "Construction_Materials": ["SCC","TASCO","TOA"],
    # "ICT": ["ADVANC","TRUE","JAS","JMART","JTS","SKY"],
    # "Transportation_Logistics": ["AOT","BEM","BTS","AAV","BA","PRM","RCL"],
}

START_DATE = "2015-01-01"
END_DATE = "2025-01-01"

ROLLING_WINDOW = 250
VOL_LOOKBACK = 20
VOL_MULTIPLIER = 0.5
MIN_THRESHOLD = 0.002
FIB_EXT, FIB_RET = 1.618, 0.618

# ---------- Utility ----------
def safe_skew(x):
    x = x[~np.isnan(x)]
    return skew(x, bias=False) if len(x) > 2 else np.nan

def safe_kurtosis(x):
    x = x[~np.isnan(x)]
    return kurtosis(x, bias=False) if len(x) > 2 else np.nan

def detect_zigzag(price_series, threshold):
    zz_idx, last_p, direction = [], price_series.iloc[0], 0
    for i, p in enumerate(price_series):
        change = (p - last_p) / (abs(last_p) + 1e-9)
        if direction == 0:
            direction = np.sign(change)
        elif direction > 0 and change <= -threshold:
            zz_idx.append(i); direction = -1; last_p = p
        elif direction < 0 and change >= threshold:
            zz_idx.append(i); direction = 1; last_p = p
    peaks = argrelextrema(price_series.values, np.greater, order=5)[0]
    troughs = argrelextrema(price_series.values, np.less, order=5)[0]
    zz_idx = sorted(list(set(zz_idx + list(peaks) + list(troughs))))
    return [0] + zz_idx + [len(price_series) - 1]

def detect_dominant_pattern(prices):
    y = (prices - prices.min()) / (prices.max() - prices.min() + 1e-9)
    peaks = argrelextrema(y.values, np.greater, order=5)[0]
    troughs = argrelextrema(y.values, np.less, order=5)[0]
    score_doubletop = score_headshoulder = score_triangle = 0
    if len(peaks) >= 2:
        diff = abs(y.iloc[peaks[-1]] - y.iloc[peaks[-2]])
        if diff < 0.05: score_doubletop = 1 - diff
    if len(peaks) >= 3:
        l, m, r = y.iloc[peaks[-3]], y.iloc[peaks[-2]], y.iloc[peaks[-1]]
        if m > l and m > r: score_headshoulder = min(abs(m - (l + r) / 2), 1)
    highslope = (y.iloc[-1] - y.iloc[0]) / len(y)
    std_band = y.rolling(10).std().iloc[-1]
    if std_band < 0.05 and abs(highslope) < 0.02:
        score_triangle = 0.7
    scores = {"DoubleTop": score_doubletop, "HeadShoulder": score_headshoulder, "Triangle": score_triangle}
    return max(scores, key=scores.get), float(scores[max(scores, key=scores.get)])

def validate_wave_relaxed(deltas):
    if len(deltas) < 3: return 0.0
    if len(deltas) >= 5: d1, d3, d5 = deltas[0], deltas[2], deltas[4]
    elif len(deltas) >= 3: d1, d3, d5 = deltas[0], deltas[2], deltas[-1]
    else: return 0.0
    ratio_13 = abs(d3 / (d1 + 1e-9))
    ratio_35 = abs(d5 / (d3 + 1e-9))
    cond1 = 1.0 <= ratio_13 / FIB_EXT <= 2.0
    cond2 = 0.5 <= ratio_35 / FIB_RET <= 2.0
    score = 0
    if cond1: score += 0.5
    if cond2: score += 0.5
    return score

# ---------- Download by SECTOR ----------
all_data = []

for sector, stocks in SECTORS.items():
    for s in stocks:
        ticker = s + ".BK"
        print(f"📥 Downloading {sector} → {ticker} ...")
        df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False)
        if df.empty:
            print(f"⚠️ No data for {ticker}, skipped")
            continue

        # Flatten MultiIndex (บาง version ของ yfinance)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        df = df.reset_index()
        df["Stock"] = s
        df["Sector"] = sector
        df.rename(columns={"Date":"Date","Close":"Close","High":"High","Low":"Low","Volume":"Volume"}, inplace=True)
        all_data.append(df)

df_all = pd.concat(all_data, ignore_index=True)

# ---------- Compute indicators ----------
results = []

for stock, sub in df_all.groupby("Stock"):
    sub = sub.copy().sort_values("Date")

    # ตรวจสอบกรณี Close เป็น DataFrame (จาก MultiColumn yfinance)
    if isinstance(sub["Close"], pd.DataFrame):
        sub["Close"] = sub["Close"].iloc[:, 0]

    # --- Basic Return ---
    sub["Return"] = sub["Close"].pct_change()

    # --- Momentum Indicators ---
    sub["RSI_14"] = ta.momentum.rsi(sub["Close"], window=14)
    sub["MACD"] = ta.trend.macd(sub["Close"])
    sub["EMA_12"] = ta.trend.ema_indicator(sub["Close"], window=12)
    sub["EMA_26"] = ta.trend.ema_indicator(sub["Close"], window=26)
    sub["ADX_14"] = ta.trend.adx(sub["High"], sub["Low"], sub["Close"], window=14)
    sub["ATR_14"] = ta.volatility.average_true_range(sub["High"], sub["Low"], sub["Close"], window=14)

    # --- Bollinger Bands ---
    bb7 = ta.volatility.BollingerBands(sub["Close"], window=7, window_dev=2)
    bb14 = ta.volatility.BollingerBands(sub["Close"], window=14, window_dev=2)

    sub["BB7_High"] = bb7.bollinger_hband()
    sub["BB7_Low"] = bb7.bollinger_lband()
    sub["BB14_High"] = bb14.bollinger_hband()
    sub["BB14_Low"] = bb14.bollinger_lband()

    # --- Normalize selected features ---
    feats = [
        "RSI_14", "MACD", "ADX_14", "ATR_14",
        "EMA_12", "EMA_26", "BB7_High", "BB7_Low", "BB14_High", "BB14_Low"
    ]
    zdf = sub[feats].apply(lambda x: (x - x.mean()) / (x.std() + 1e-9))

    # --- Technical Composite Score ---
    sub["TechScore"] = zdf.mean(axis=1)

    # Pattern + Wave
    pattern_types, pattern_conf, wave_dir, wave_strength, wave_valid = [], [], [], [], []
    for i in range(len(sub)):
        window = sub["Close"].iloc[max(0, i - ROLLING_WINDOW):i+1]
        if len(window) < 60:
            pattern_types.append(None)
            pattern_conf.append(np.nan)
            wave_dir.append(None)
            wave_strength.append(np.nan)
            wave_valid.append(np.nan)
            continue

        log_prices = np.log(window)
        log_prices = (log_prices - log_prices.mean()) / (log_prices.std() + 1e-9)
        rolling_vol = log_prices.diff().std()
        thr = max(MIN_THRESHOLD, VOL_MULTIPLIER * rolling_vol)
        zz_idx = detect_zigzag(log_prices, thr)
        pivots = log_prices.iloc[zz_idx].values

        ptype, pconf = detect_dominant_pattern(log_prices)
        if len(pivots) >= 3:
            deltas = np.diff(pivots)
            wdir = "Up" if deltas[-1] > 0 else "Down"
            wstr = abs(deltas[-1] / (np.mean(np.abs(pivots)) + 1e-9))
            wval = validate_wave_relaxed(deltas)
        else:
            wdir, wstr, wval = None, np.nan, np.nan

        pattern_types.append(ptype)
        pattern_conf.append(pconf)
        wave_dir.append(wdir)
        wave_strength.append(wstr)
        wave_valid.append(wval)

    sub["Pattern_Type"] = pattern_types
    sub["Pattern_Confidence"] = pattern_conf
    sub["Wave_Direction"] = wave_dir
    sub["Wave_Strength"] = wave_strength
    sub["Wave_Valid"] = wave_valid

    results.append(sub)

# ---------- Combine ----------
df_final = pd.concat(results).reset_index(drop=True)
df_final = df_final.sort_values(["Sector", "Stock", "Date"])

# ---------- Export ----------
out_path = Path("Historical_Technical_Indicators_bySector.xlsx")
df_final.to_excel(out_path, index=False)
print(f"\n✅ Exported → {out_path.name}")
print(f"✅ Total sectors: {df_final['Sector'].nunique()}, total stocks: {df_final['Stock'].nunique()}, total rows: {len(df_final)}")

# ---------- Example ----------
sample_sector = df_final["Sector"].iloc[0]
sample_stock = df_final[df_final["Sector"] == sample_sector]["Stock"].iloc[0]
sample = df_final[df_final["Stock"] == sample_stock].tail(10)[
    ["Date","Close","RSI_14","MACD","ADX_14","ATR_14","TechScore","EMA_12", "EMA_26", "BB7_High", "BB7_Low", "BB14_High", "BB14_Low" ,
     "Pattern_Type","Pattern_Confidence","Wave_Direction","Wave_Strength","Wave_Valid"]
]
print(f"\n📈 Example — {sample_sector} / {sample_stock}")
print(sample)


📥 Downloading Banking → BBL.BK ...
📥 Downloading Banking → KBANK.BK ...
📥 Downloading Banking → KKP.BK ...
📥 Downloading Banking → KTB.BK ...
📥 Downloading Banking → TCAP.BK ...
📥 Downloading Banking → TISCO.BK ...
📥 Downloading Banking → TTB.BK ...
📥 Downloading Finance_Securities → KTC.BK ...
📥 Downloading Finance_Securities → AEONTS.BK ...
📥 Downloading Finance_Securities → JMT.BK ...
📥 Downloading Finance_Securities → SAWAD.BK ...

✅ Exported → Historical_Technical_Indicators_bySector.xlsx
✅ Total sectors: 2, total stocks: 11, total rows: 26730

📈 Example — Banking / BBL
Price       Date       Close     RSI_14      MACD     ADX_14    ATR_14  \
4850  2024-12-17  138.902374  41.742812  0.190827  14.296933  1.789800   
4851  2024-12-18  139.844070  45.915842  0.075202  14.055360  1.796486   
4852  2024-12-19  140.314926  47.924460  0.021317  13.139435  1.836329   
4853  2024-12-20  140.785782  49.927153  0.016417  12.288933  1.839692   
4854  2024-12-23  141.727493  53.757488  0.08751

📊 สรุป DL Structure ของ Block 5.5
ComponentรายละเอียดInputClose Price → Kalman Filter → MinMaxScaler → Shape (60, 1)Model2 โมเดล: CNN-LSTM และ CNN-GRULayers7 layers: Conv1D → BN → MaxPool → LSTM/GRU → BN → Dropout → DenseOutputProbability [0,1] ว่าราคาจะขึ้นEnsemble(LSTM + GRU) / 2OptimizerAdam (lr=0.001)LossBinary Crossentropy

🔬 Activation Functions
LayerActivationเหตุผลConv1DReLUเร็ว, ไม่มี vanishing gradient, ดีสำหรับ feature extractionLSTM/GRUtanh + sigmoidBuilt-in สำหรับ gates และ cell stateDense (Output)SigmoidBinary classification → output เป็น probability [0,1]

🧠 Bidirectional + Attention คืออะไร และเพิ่มเข้าไปยังไง

1. Bidirectional LSTM/GRU คืออะไร?
ปัจจุบัน (Unidirectional):
Past ──────────────────────────────────> Future
 t-60   t-59   t-58   ...   t-2   t-1   t
  →      →      →      →     →     →    [Output]
อ่านข้อมูลจาก ซ้ายไปขวา อย่างเดียว (อดีต → ปัจจุบัน)
Bidirectional:
Forward:   t-60 → t-59 → t-58 → ... → t-1 → t
                                            ↘
                                             [Concat] → Output
                                            ↗
Backward:  t-60 ← t-59 ← t-58 ← ... ← t-1 ← t
อ่านข้อมูล ทั้งสองทิศทาง แล้วรวมกัน
ข้อดี:

เข้าใจ Context ได้ดีขึ้น (รู้ทั้งอดีตและอนาคตของแต่ละจุด)
จับ Pattern ที่ซับซ้อนได้ดีกว่า


2. Attention Mechanism คืออะไร?
ปัญหาของ LSTM ปกติ:

Output สุดท้ายต้อง "จำ" ข้อมูลทั้ง 60 วันไว้ใน hidden state เดียว
ข้อมูลเก่าๆ อาจถูกลืม (vanishing information)

Attention แก้ปัญหานี้:
Input: [t-60, t-59, t-58, ..., t-2, t-1, t]
         ↓     ↓     ↓          ↓    ↓   ↓
       [h1,   h2,   h3,   ..., h58, h59, h60]  ← Hidden states ทุก timestep
         ↓     ↓     ↓          ↓    ↓   ↓
       [0.01, 0.02, 0.05, ..., 0.15, 0.30, 0.40]  ← Attention weights
         ↓     ↓     ↓          ↓    ↓   ↓
       ──────────── Weighted Sum ────────────
                        ↓
                   Context Vector
                        ↓
                     Output
ข้อดี:

โมเดล "เลือก" ว่าจะให้ความสำคัญกับวันไหนมากกว่า
วันที่สำคัญ (เช่น วันที่มี pattern breakout) จะได้ weight สูง
Interpretable: ดูได้ว่าโมเดลสนใจวันไหน


3. โครงสร้างใหม่ที่เสนอ
┌─────────────────────────────────────────────────────────────────┐
│                  🏗️ UPGRADED MODEL: BiLSTM + Attention           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Input (60, 1)                                                  │
│       ↓                                                         │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Conv1D (64 filters, kernel=3, ReLU)                      │   │
│  │ BatchNormalization                                       │   │
│  │ MaxPooling1D (pool=2)                                    │   │
│  └─────────────────────────────────────────────────────────┘   │
│       ↓                                                         │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ 🆕 Bidirectional LSTM (50 units, return_sequences=True)  │   │
│  │    Output: (timesteps, 100)  ← 50*2 เพราะ bi-directional │   │
│  └─────────────────────────────────────────────────────────┘   │
│       ↓                                                         │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ 🆕 Attention Layer                                       │   │
│  │    - Compute attention weights for each timestep         │   │
│  │    - Weighted sum → Context vector (100,)                │   │
│  └─────────────────────────────────────────────────────────┘   │
│       ↓                                                         │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ BatchNormalization                                       │   │
│  │ Dropout (0.3)                                            │   │
│  │ Dense (1, sigmoid)                                       │   │
│  └─────────────────────────────────────────────────────────┘   │
│       ↓                                                         │
│  Output: P(price UP) ∈ [0, 1]                                   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘

4. เปรียบเทียบ เดิม vs ใหม่
Componentเดิม (Block 5.5)ใหม่ (Upgraded)LSTM/GRUUnidirectionalBidirectionalreturn_sequencesFalseTrue (ต้องการทุก timestep)Attention❌ ไม่มี✅ มีOutput dimension50100 (50×2)Interpretabilityต่ำสูง (ดู attention weights ได้)

5. ข้อควรระวัง
ข้อดีข้อเสียจับ pattern ได้ดีขึ้นParameters เพิ่มขึ้น ~2xInterpretable (attention weights)Train นานขึ้นลด information lossอาจ overfit ถ้าข้อมูลน้อย

In [ ]:
# ============================================================
# 🧠 BLOCK 5.5: Deep Learning Signal (UPGRADED + ROLLING WINDOW)
#    Config: Kalman -> CNN -> BN -> Bidirectional LSTM/GRU -> Attention -> Dropout
#    Features:
#    - Bidirectional LSTM/GRU (capture both past and future context)
#    - Attention Mechanism (focus on important timesteps)
#    - Rolling Window / Walk-Forward Validation
#    - Improved Ensemble (3 models)
# ============================================================

import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Dense, LSTM, GRU, Conv1D, MaxPooling1D, Input, Dropout,
    BatchNormalization, Bidirectional, Layer
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import warnings
import os

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

# ============================================================
# 1. TARGET STOCKS
# ============================================================

# [
#     "AAV", "AEONTS", "AOT", "BA", "BBL", "BCH", "BDMS", "BEM", "BH", "BTS",
#     "CHG", "JMT", "KBANK", "KKP", "KTB", "KTC", "PR9", "PRM", "RCL", "SAWAD",
#     "TCAP", "TISCO", "TTB"
# ]

target_stocks = ["BBL","KBANK","KKP","KTB","TCAP","TISCO","TTB"]

# ============================================================
# 2. ROLLING WINDOW CONFIGURATION
# ============================================================
# Walk-Forward Validation Windows
# Format: (train_start, train_end, test_year)
ROLLING_WINDOWS = [
    ("2015-01-01", "2018-12-31", 2019),
    ("2016-01-01", "2019-12-31", 2020),
    ("2017-01-01", "2020-12-31", 2021),
    ("2018-01-01", "2021-12-31", 2022),
    ("2019-01-01", "2022-12-31", 2023),
    ("2020-01-01", "2023-12-31", 2024),
    ("2021-01-01", "2024-12-31", 2025),
]

# ============================================================
# 3. KALMAN FILTER (Noise Reduction)
# ============================================================
class KalmanFilter1D:
    """1D Kalman Filter for smoothing price data"""
    def __init__(self, process_noise=1e-5, measurement_noise=1e-3, estimated_error=1.0):
        self.Q = process_noise
        self.R = measurement_noise
        self.P = estimated_error
        self.X = 0

    def update(self, measurement):
        self.P = self.P + self.Q
        K = self.P / (self.P + self.R)
        self.X = self.X + K * (measurement - self.X)
        self.P = (1 - K) * self.P
        return self.X

    def smooth(self, data):
        self.X = data[0]
        result = []
        for measurement in data:
            result.append(self.update(measurement))
        return np.array(result)


# ============================================================
# 4. ATTENTION LAYER (Custom Implementation)
# ============================================================
class AttentionLayer(Layer):
    """
    Attention Mechanism for Time Series
    """
    def __init__(self, return_attention=False, **kwargs):
        self.return_attention = return_attention
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(
            name='attention_weight',
            shape=(input_shape[-1], input_shape[-1]),
            initializer='glorot_uniform',
            trainable=True
        )
        self.b = self.add_weight(
            name='attention_bias',
            shape=(input_shape[-1],),
            initializer='zeros',
            trainable=True
        )
        self.u = self.add_weight(
            name='attention_context',
            shape=(input_shape[-1], 1),
            initializer='glorot_uniform',
            trainable=True
        )
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        uit = tf.nn.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
        ait = tf.tensordot(uit, self.u, axes=1)
        ait = tf.squeeze(ait, axis=-1)
        ait = tf.nn.softmax(ait, axis=1)
        ait_expanded = tf.expand_dims(ait, axis=-1)
        weighted_input = x * ait_expanded
        output = tf.reduce_sum(weighted_input, axis=1)

        if self.return_attention:
            return output, ait
        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

    def get_config(self):
        config = super(AttentionLayer, self).get_config()
        config.update({'return_attention': self.return_attention})
        return config


# ============================================================
# 5. MODEL ARCHITECTURES
# ============================================================

def build_cnn_bilstm_attention(input_shape):
    """Model 1: CNN + Bidirectional LSTM + Attention"""
    inputs = Input(shape=input_shape)

    x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = Bidirectional(LSTM(50, return_sequences=True))(x)
    x = BatchNormalization()(x)

    x = AttentionLayer(return_attention=False)(x)

    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model


def build_cnn_bigru_attention(input_shape):
    """Model 2: CNN + Bidirectional GRU + Attention"""
    inputs = Input(shape=input_shape)

    x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = Bidirectional(GRU(50, return_sequences=True))(x)
    x = BatchNormalization()(x)

    x = AttentionLayer(return_attention=False)(x)

    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model


def build_stacked_bilstm_attention(input_shape):
    """Model 3: Stacked Bidirectional LSTM + Attention"""
    inputs = Input(shape=input_shape)

    x = Bidirectional(LSTM(64, return_sequences=True))(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    x = Bidirectional(LSTM(32, return_sequences=True))(x)
    x = BatchNormalization()(x)

    x = AttentionLayer(return_attention=False)(x)

    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model


# ============================================================
# 6. DATA PREPARATION
# ============================================================

def create_dataset(data, look_back=60):
    """Create sequences for time series prediction"""
    X, y = [], []
    for i in range(len(data) - look_back - 1):
        X.append(data[i:(i + look_back), 0])
        y.append(1 if data[i + look_back, 0] > data[i + look_back - 1, 0] else 0)
    return np.array(X), np.array(y)


def create_dataset_with_dates(df, scaled_data, look_back=60):
    """Create sequences with corresponding dates"""
    X, y, dates = [], [], []
    for i in range(len(scaled_data) - look_back - 1):
        X.append(scaled_data[i:(i + look_back), 0])
        y.append(1 if scaled_data[i + look_back, 0] > scaled_data[i + look_back - 1, 0] else 0)
        dates.append(df.index[i + look_back])
    return np.array(X), np.array(y), dates


# ============================================================
# 7. ROLLING WINDOW TRAINING FUNCTION
# ============================================================

def train_with_rolling_window(df, scaled_data, look_back=60, epochs=20, batch_size=32):
    """
    Train models using Rolling Window / Walk-Forward Validation

    Returns:
        all_predictions: List of predictions for each window
        all_metrics: Performance metrics for each window
    """

    # Create dataset with dates
    X, y, dates = create_dataset_with_dates(df, scaled_data, look_back)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Convert dates to DataFrame for easier filtering
    dates_df = pd.DataFrame({'date': dates, 'idx': range(len(dates))})
    dates_df['year'] = pd.to_datetime(dates_df['date']).dt.year

    all_predictions = []
    all_metrics = []

    early_stop = EarlyStopping(
        monitor='loss',
        patience=5,
        restore_best_weights=True,
        verbose=0
    )

    for train_start, train_end, test_year in ROLLING_WINDOWS:
        # Get indices for train and test
        train_mask = (dates_df['date'] >= train_start) & (dates_df['date'] <= train_end)
        test_mask = (dates_df['year'] == test_year)

        train_idx = dates_df[train_mask]['idx'].values
        test_idx = dates_df[test_mask]['idx'].values

        if len(train_idx) < 100 or len(test_idx) < 10:
            continue

        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]
        test_dates = [dates[i] for i in test_idx]

        input_shape = (look_back, 1)

        # Train Model 1: CNN-BiLSTM-Attention
        model1 = build_cnn_bilstm_attention(input_shape)
        model1.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                   callbacks=[early_stop], verbose=0)
        pred1 = model1.predict(X_test, verbose=0)

        # Train Model 2: CNN-BiGRU-Attention
        model2 = build_cnn_bigru_attention(input_shape)
        model2.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                   callbacks=[early_stop], verbose=0)
        pred2 = model2.predict(X_test, verbose=0)

        # Train Model 3: Stacked BiLSTM-Attention
        model3 = build_stacked_bilstm_attention(input_shape)
        model3.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                   callbacks=[early_stop], verbose=0)
        pred3 = model3.predict(X_test, verbose=0)

        # Ensemble
        ensemble_pred = (pred1 + pred2 + pred3) / 3.0
        ensemble_binary = (ensemble_pred > 0.5).astype(int).flatten()

        # Evaluate
        accuracy = accuracy_score(y_test, ensemble_binary)
        precision = precision_score(y_test, ensemble_binary, zero_division=0)
        recall = recall_score(y_test, ensemble_binary, zero_division=0)
        f1 = f1_score(y_test, ensemble_binary, zero_division=0)

        all_metrics.append({
            'Train_Period': f"{train_start[:4]}-{train_end[:4]}",
            'Test_Year': test_year,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1_Score': f1,
            'Train_Samples': len(y_train),
            'Test_Samples': len(y_test)
        })

        # Store predictions
        for i, date in enumerate(test_dates):
            all_predictions.append({
                'Date': date,
                'Test_Year': test_year,
                'DL_Signal': ensemble_pred[i, 0],
                'DL_Signal_Binary': ensemble_binary[i],
                'Actual': y_test[i]
            })

        # Clean up memory
        del model1, model2, model3
        tf.keras.backend.clear_session()

    return all_predictions, all_metrics


# ============================================================
# 8. MAIN TRAINING LOOP
# ============================================================

print("=" * 70)
print("🧠 BLOCK 5.5: Deep Learning Signal (UPGRADED + ROLLING WINDOW)")
print("=" * 70)
print("""
📦 Model Architecture:
   - Model 1: CNN + Bidirectional LSTM + Attention
   - Model 2: CNN + Bidirectional GRU + Attention
   - Model 3: Stacked Bidirectional LSTM + Attention

🔄 Rolling Window Configuration:
   - Window 1: Train [2015-2018] → Test [2019]
   - Window 2: Train [2016-2019] → Test [2020]
   - Window 3: Train [2017-2020] → Test [2021]
   - Window 4: Train [2018-2021] → Test [2022]
   - Window 5: Train [2019-2022] → Test [2023]
   - Window 6: Train [2020-2023] → Test [2024]
   - Window 7: Train [2021-2024] → Test [2025]

⚙️ Configuration:
   - Lookback: 60 days
   - Epochs: 20 (with EarlyStopping)
   - Batch Size: 32
   - Ensemble: Average of 3 models
""")
print("=" * 70)

# Storage for results
all_stock_predictions = []
all_stock_metrics = []

# Training parameters
LOOK_BACK = 60
EPOCHS = 20
BATCH_SIZE = 32

for stock in target_stocks:
    print(f"\n{'='*50}")
    print(f"🔹 Processing: {stock}")
    print(f"{'='*50}")

    try:
        # ===== 1. Download Data =====
        df = yf.download(f"{stock}.BK", start="2015-01-01", end="2025-12-31", progress=False)

        if len(df) < 500:
            print(f"⚠️ Skipped (insufficient data: {len(df)} rows)")
            continue

        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        data = df[['Close']].values

        print(f"   📅 Data range: {df.index.min().strftime('%Y-%m-%d')} to {df.index.max().strftime('%Y-%m-%d')}")
        print(f"   📊 Total rows: {len(df)}")

        # ===== 2. Preprocessing =====
        kf = KalmanFilter1D(process_noise=1e-5, measurement_noise=0.01)
        smoothed_data = kf.smooth(data.flatten()).reshape(-1, 1)

        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(smoothed_data)

        # ===== 3. Train with Rolling Window =====
        predictions, metrics = train_with_rolling_window(
            df, scaled_data,
            look_back=LOOK_BACK,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE
        )

        # Add stock name to results
        for pred in predictions:
            pred['Stock'] = stock
        for metric in metrics:
            metric['Stock'] = stock

        all_stock_predictions.extend(predictions)
        all_stock_metrics.extend(metrics)

        # Print window results
        print(f"\n   📊 Rolling Window Results for {stock}:")
        print(f"   {'Train Period':<15} {'Test Year':<10} {'Accuracy':<10} {'F1 Score':<10}")
        print(f"   {'-'*45}")
        for m in metrics:
            print(f"   {m['Train_Period']:<15} {m['Test_Year']:<10} {m['Accuracy']:.2%}     {m['F1_Score']:.2%}")

        # Average performance
        if metrics:
            avg_acc = np.mean([m['Accuracy'] for m in metrics])
            avg_f1 = np.mean([m['F1_Score'] for m in metrics])
            print(f"   {'-'*45}")
            print(f"   {'AVERAGE':<15} {'':<10} {avg_acc:.2%}     {avg_f1:.2%}")

    except Exception as e:
        print(f"❌ Error: {e}")


# ============================================================
# 9. SUMMARY & EXPORT
# ============================================================

print("\n" + "=" * 70)
print("📊 OVERALL PERFORMANCE SUMMARY (All Stocks, All Windows)")
print("=" * 70)

if all_stock_metrics:
    df_metrics = pd.DataFrame(all_stock_metrics)

    # Summary by Test Year
    print("\n📅 Performance by Test Year:")
    print("-" * 50)
    year_summary = df_metrics.groupby('Test_Year').agg({
        'Accuracy': 'mean',
        'Precision': 'mean',
        'Recall': 'mean',
        'F1_Score': 'mean',
        'Test_Samples': 'sum'
    }).round(4)
    print(year_summary.to_string())

    # Summary by Stock
    print("\n📈 Performance by Stock (Average across all windows):")
    print("-" * 50)
    stock_summary = df_metrics.groupby('Stock').agg({
        'Accuracy': 'mean',
        'Precision': 'mean',
        'Recall': 'mean',
        'F1_Score': 'mean'
    }).round(4)
    print(stock_summary.to_string())

    # Overall Average
    print("\n" + "=" * 50)
    print("🎯 OVERALL AVERAGE PERFORMANCE")
    print("=" * 50)
    print(f"   • Accuracy:  {df_metrics['Accuracy'].mean():.2%}")
    print(f"   • Precision: {df_metrics['Precision'].mean():.2%}")
    print(f"   • Recall:    {df_metrics['Recall'].mean():.2%}")
    print(f"   • F1 Score:  {df_metrics['F1_Score'].mean():.2%}")
    print(f"   • Total Test Samples: {df_metrics['Test_Samples'].sum():,}")

# Export Results
if all_stock_predictions:
    df_predictions = pd.DataFrame(all_stock_predictions)

    # Save predictions
    outfile_signals = "DeepLearning_Signal_BiLSTM_Attention_RollingWindow.xlsx"
    df_predictions.to_excel(outfile_signals, index=False)
    print(f"\n💾 Predictions saved to: {outfile_signals}")

    # Save metrics
    outfile_metrics = "DeepLearning_Metrics_BiLSTM_Attention_RollingWindow.xlsx"
    df_metrics.to_excel(outfile_metrics, index=False)
    print(f"💾 Metrics saved to: {outfile_metrics}")

print("\n" + "=" * 70)
print("✅ BLOCK 5.5 COMPLETED: BiLSTM/GRU + Attention + Rolling Window")
print("=" * 70)

🧠 BLOCK 5.5: Deep Learning Signal (UPGRADED + ROLLING WINDOW)

📦 Model Architecture:
   - Model 1: CNN + Bidirectional LSTM + Attention
   - Model 2: CNN + Bidirectional GRU + Attention  
   - Model 3: Stacked Bidirectional LSTM + Attention

🔄 Rolling Window Configuration:
   - Window 1: Train [2015-2018] → Test [2019]
   - Window 2: Train [2016-2019] → Test [2020]
   - Window 3: Train [2017-2020] → Test [2021]
   - Window 4: Train [2018-2021] → Test [2022]
   - Window 5: Train [2019-2022] → Test [2023]
   - Window 6: Train [2020-2023] → Test [2024]
   - Window 7: Train [2021-2024] → Test [2025]

⚙️ Configuration:
   - Lookback: 60 days
   - Epochs: 20 (with EarlyStopping)
   - Batch Size: 32
   - Ensemble: Average of 3 models


🔹 Processing: BBL
   📅 Data range: 2015-01-05 to 2025-12-30
   📊 Total rows: 2672

   📊 Rolling Window Results for BBL:
   Train Period    Test Year  Accuracy   F1 Score  
   ---------------------------------------------
   2015-2018       2019       30.74%    

In [ ]:
# ============================================================
# 🔗 BLOCK 6: Data Integration (Standard)
# ============================================================
import pandas as pd
import numpy as np
import yfinance as yf
import warnings

warnings.filterwarnings("ignore")

print("⏳ Integrating Data with Advanced Technicals...")

# target_stocks = [
#     "AAV", "AEONTS", "AOT", "BA", "BBL", "BCH", "BDMS", "BEM", "BH", "BTS",
#     "CHG", "JMT", "KBANK", "KKP", "KTB", "KTC", "PR9", "PRM", "RCL", "SAWAD",
#     "TCAP", "TISCO", "TTB"
# ]

target_stocks = ["BBL","KBANK","KKP","KTB","TCAP","TISCO","TTB"]

def calculate_adx(df, window=14):
    high = df['High']
    low = df['Low']
    close = df['Close']
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.rolling(window).mean()
    plus_di = 100 * (plus_dm.ewm(alpha=1/window).mean() / atr)
    minus_di = 100 * (abs(minus_dm).ewm(alpha=1/window).mean() / atr)
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    return dx.rolling(window).mean()

def add_technical_score(df):
    df = df.copy()
    df['EMA_200'] = df['Close'].ewm(span=200, adjust=False).mean()

    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    k = df['Close'].ewm(span=12, adjust=False).mean()
    d = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = k - d
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

    s_trend = np.where(df['Close'] > df['EMA_200'], 1, -1)
    s_rsi = np.where(df['RSI'] < 30, 1, np.where(df['RSI'] > 70, -1, 0))
    s_macd = np.where(df['MACD'] > df['MACD_Signal'], 1, -1)
    df['Tech_Signal'] = (s_trend * 0.4) + (s_macd * 0.4) + (s_rsi * 0.2)
    return df

print("   🔹 Step 1: Loading Full Price Data...")
df_price = pd.DataFrame()
for stock in target_stocks:
    try:
        data = yf.download(f"{stock}.BK", start="2018-01-01", end="2025-12-31", progress=False)
        if isinstance(data.columns, pd.MultiIndex): data.columns = data.columns.get_level_values(0)
        data = data.reset_index()
        data['Stock'] = stock
        data['ADX'] = calculate_adx(data)
        data = add_technical_score(data)
        df_price = pd.concat([df_price, data], ignore_index=True)
    except: pass

req_cols = ['Date', 'Stock', 'Close', 'Tech_Signal', 'ADX', 'RSI', 'MACD']
df_rl = df_price[req_cols].dropna().sort_values(['Stock', 'Date']).reset_index(drop=True)

print("   🔹 Step 2 & 3: Merging Signals...")
try:
    df_macro = pd.read_excel("Forecast_ARDL_ECM_NextMonth_Clean.xlsx")
    sig_col = 'Pred_dLogclose' if 'Pred_dLogclose' in df_macro.columns else df_macro.select_dtypes(include=[np.number]).columns[0]
    macro_map = df_macro.set_index('Stock')[sig_col].to_dict()
    df_rl['Macro_Signal'] = df_rl['Stock'].map(macro_map).fillna(0)
except: df_rl['Macro_Signal'] = 0

try:
    df_dl = pd.read_excel("DeepLearning_Signal_LSTM_GRU.xlsx")
    df_dl['Date'] = pd.to_datetime(df_dl['Date'])
    df_rl = pd.merge(df_rl, df_dl[['Date', 'Stock', 'DL_Signal']], on=['Date', 'Stock'], how='left')
    df_rl['DL_Signal'] = df_rl['DL_Signal'].fillna(0.5)
except: df_rl['DL_Signal'] = 0.5

df_rl.to_excel("Final_RL_Input_Ready.xlsx", index=False)
print("✅ Data Integration Complete!")

⏳ Integrating Data with Advanced Technicals...
   🔹 Step 1: Loading Full Price Data...
   🔹 Step 2 & 3: Merging Signals...
✅ Data Integration Complete!


In [ ]:
!pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 4.2 MB/s eta 0:00:00


📋 1. Constraint ที่ดัดนิสัย RLConstraintรายละเอียดจุดประสงค์Cooldown 2 Daysหลัง Sell ต้องรอ 2 วันถึงจะ Buy ได้ป้องกัน OvertradingMomentum Rule (Buy)Buy ได้เมื่อ ADX > 25 และ RSI < 75 หรือ RSI < 45บังคับซื้อตาม Trend🆕 Volatility-based SizingPosition Size 30-70% ตาม Volatilityลดความเสี่ยงช่วง Vol สูง🆕 Max Consecutive Lossesขาดทุน 5 ครั้งติด → หยุด 3 วันหยุดพักเมื่อ Strategy ไม่ workRandom Initial Position50% โอกาสเริ่มมีหุ้นอยู่แล้วฝึกให้รับมือทุกสถานการณ์🛡️ 2. Risk ManagementRuleTriggerActionจุดประสงค์Hard Stop LossPnL ≤ -6%Force SellจำกัดขาดทุนสูงสุดTrailing StopPnL > +10% แล้วตก > 4% จาก PeakForce Sellล็อคกำไร🆕 Volatility Sizingทุกครั้งที่ Buyปรับ Size 30-70%ลด Position ช่วง Vol สูง🆕 Loss Pauseขาดทุน 5 ครั้งติดหยุด 3 วันป้องกัน Drawdown ต่อเนื่อง🎁 3. Reward Shapingสถานการณ์Rewardจุดประสงค์Buy สำเร็จ+0.1กำลังใจ Take ActionSell จาก Trailing Stop+2.0รางวัลใหญ่สำหรับกำไรเยอะSell จาก Stop Loss+0.1รางวัลสำหรับ DisciplineSell มีกำไร (ปกติ)+PnL × 10ตามสัดส่วนกำไรSell ขาดทุน (ไม่ใช่ SL)-0.5ลงโทษ Sell ผิดจังหวะ🆕 Trigger Loss Pause-1.5ลงโทษหนักเมื่อขาดทุนติด 5 ครั้ง🆕 ระหว่าง Pause-0.3 ต่อวันลงโทษ Opportunity Costทุก Step+(ΔNetWorth / NetWorth) × 100ติดตาม Portfolio Value📊 4. State Vector (11 Features)#FeatureOriginal/New1Macro_SignalOriginal2DL_SignalOriginal3Tech_SignalOriginal4ADX / 100Original5Balance / 1MOriginal6Holdings Value / 1MOriginal7RSI / 100Original8MACD / 10Original9Volatility Multiplier🆕 New10Consecutive Losses / 5🆕 New11Pause Counter / 3🆕 New

┌─────────────────────────────────────┐
                    │          RL Agent (PPO)             │
                    │         เลือก Action                │
                    └─────────────────────────────────────┘
                                    │
                                    ▼
                    ┌─────────────────────────────────────┐
                    │   🆕 CHECK: Loss Pause Active?      │
                    │      (ขาดทุน 5 ครั้ง → หยุด 3 วัน)   │
                    └─────────────────────────────────────┘
                                    │
                    ┌───────────────┴───────────────┐
                    ▼                               ▼
              [Pause = YES]                   [Pause = NO]
              Force HOLD                      Continue...
              Reward -0.3                           │
                    │                               ▼
                    │               ┌─────────────────────────────────┐
                    │               │    CHECK: Risk Management       │
                    │               │    • Stop Loss -6%?             │
                    │               │    • Trailing Stop?             │
                    │               └─────────────────────────────────┘
                    │                               │
                    │               ┌───────────────┴───────────────┐
                    │               ▼                               ▼
                    │         [Risk Trigger]                  [No Trigger]
                    │         Force SELL                      Continue...
                    │               │                               │
                    │               │                               ▼
                    │               │               ┌─────────────────────────────────┐
                    │               │               │    CHECK: Cooldown Active?      │
                    │               │               │    (หลัง Sell รอ 2 วัน)          │
                    │               │               └─────────────────────────────────┘
                    │               │                               │
                    │               │               ┌───────────────┴───────────────┐
                    │               │               ▼                               ▼
                    │               │         [Cooldown = YES]               [Cooldown = NO]
                    │               │         Block BUY                       Continue...
                    │               │               │                               │
                    │               │               │                               ▼
                    │               │               │               ┌─────────────────────────────┐
                    │               │               │               │      EXECUTE ACTION         │
                    │               │               │               │  BUY / HOLD / SELL          │
                    │               │               │               └─────────────────────────────┘
                    │               │               │                               │
                    │               │               │                               ▼
                    │               │               │               ┌─────────────────────────────┐
                    │               │               │               │  🆕 BUY: Volatility Sizing  │
                    │               │               │               │     High Vol → 30-50%       │
                    │               │               │               │     Low Vol → 50-70%        │
                    │               │               │               └─────────────────────────────┘
                    │               │               │                               │
                    └───────────────┴───────────────┴───────────────────────────────┘
                                                    │
                                                    ▼
                                    ┌─────────────────────────────────┐
                                    │  🆕 SELL: Track Consecutive     │
                                    │     Losses (ถ้าขาดทุน)          │
                                    │     ถ้าครบ 5 → Trigger Pause    │
                                    │     Reward -1.5                 │
                                    └─────────────────────────────────┘
                                                    │
                                                    ▼
                                    ┌─────────────────────────────────┐
                                    │      Calculate Reward           │
                                    │      Update State               │
                                    │      Learn                      │
                                    └─────────────────────────────────┘

In [ ]:
# ============================================================
# 🤖 BLOCK 7: Train PPO (UPGRADED + ROLLING WINDOW + RISK MANAGEMENT V2)
#    Logic:
#    - Cooldown 2 Days after Sell
#    - Stop Loss -6%
#    - Trailing Stop (Trigger 10%, Drop 4%)
#    - 🆕 Volatility-based Position Sizing
#    - 🆕 Max Consecutive Losses (5 losses → pause 3 days)
# ============================================================
import os
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import warnings

warnings.filterwarnings("ignore")
os.makedirs("trained_models", exist_ok=True)

# ============================================================
# 1. TARGET STOCKS
# ============================================================
["BBL","KBANK","KKP","KTB","TCAP","TISCO","TTB"]
# [
#     "AAV", "AEONTS", "AOT", "BA", "BBL", "BCH", "BDMS", "BEM", "BH", "BTS",
#     "CHG", "JMT", "KBANK", "KKP", "KTB", "KTC", "PR9", "PRM", "RCL", "SAWAD",
#     "TCAP", "TISCO", "TTB"
# ]

target_stocks = ["BBL","KBANK","KKP","KTB","TCAP","TISCO","TTB"]

# ============================================================
# 2. ROLLING WINDOW CONFIGURATION
# ============================================================
ROLLING_WINDOWS = [
    (2015, 2018, 2019),
    (2016, 2019, 2020),
    (2017, 2020, 2021),
    (2018, 2021, 2022),
    (2019, 2022, 2023),
    (2020, 2023, 2024),
    (2021, 2024, 2025),
]

# ============================================================
# 3. LOAD DATA
# ============================================================
try:
    df_rl_all = pd.read_excel("Final_RL_Input_Ready.xlsx")
    df_rl_all['Date'] = pd.to_datetime(df_rl_all['Date'])
    df_rl_all['Year'] = df_rl_all['Date'].dt.year
except:
    raise RuntimeError("❌ Missing Data: Final_RL_Input_Ready.xlsx")

# ============================================================
# 4. TRADING ENVIRONMENT (UPGRADED)
# ============================================================
class StockTradingEnv(gym.Env):
    """
    Stock Trading Environment with Enhanced Risk Management:

    Original Features:
    - Cooldown mechanism (2 days after sell)
    - Stop Loss (-6%)
    - Trailing Stop (Trigger +10%, Drop -4%)

    🆕 New Features:
    - Volatility-based Position Sizing
    - Max Consecutive Losses (5 losses → pause 3 days)
    - Penalty reward when forced to pause
    """

    def __init__(self, df, volatility_lookback=20, base_position_pct=0.5,
                 max_consecutive_losses=5, loss_pause_days=3):
        super(StockTradingEnv, self).__init__()
        self.df = df.reset_index(drop=True)
        self.initial_balance = 1_000_000

        # Action & Observation Space
        self.action_space = spaces.Discrete(3)  # 0=Hold, 1=Buy, 2=Sell
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(11,), dtype=np.float32)

        # 🆕 Volatility-based Sizing Parameters
        self.volatility_lookback = volatility_lookback
        self.base_position_pct = base_position_pct  # Base position size (50%)

        # 🆕 Max Consecutive Losses Parameters
        self.max_consecutive_losses = max_consecutive_losses  # 5 ครั้ง
        self.loss_pause_days = loss_pause_days  # หยุด 3 วัน

        # Precompute volatility for entire dataset
        self._precompute_volatility()

        self.reset()

    def _precompute_volatility(self):
        """Precompute rolling volatility for position sizing"""
        prices = self.df['Close'].values
        returns = np.diff(prices) / prices[:-1]
        returns = np.insert(returns, 0, 0)  # Pad first value

        # Rolling standard deviation
        self.volatility = pd.Series(returns).rolling(
            window=self.volatility_lookback,
            min_periods=5
        ).std().fillna(0.02).values  # Default 2% if not enough data

        # Normalize volatility to [0.5, 1.5] range for position sizing
        vol_mean = np.mean(self.volatility[self.volatility > 0])
        vol_std = np.std(self.volatility[self.volatility > 0])

        if vol_std > 0:
            self.vol_multiplier = 1 - (self.volatility - vol_mean) / (2 * vol_std)
            self.vol_multiplier = np.clip(self.vol_multiplier, 0.3, 1.0)  # 30% to 100%
        else:
            self.vol_multiplier = np.ones(len(self.df)) * 0.5

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0

        # Random initial position (50% chance)
        if np.random.rand() < 0.5:
            self.balance = self.initial_balance
            self.shares_held = 0
            self.avg_cost = 0.0
            self.highest_price = 0.0
        else:
            start_price = self.df.iloc[0]['Close']
            invest_pct = np.random.uniform(0.2, 0.8)
            cash_invested = self.initial_balance * invest_pct
            self.shares_held = int(cash_invested // start_price)
            self.balance = self.initial_balance - (self.shares_held * start_price)
            self.avg_cost = start_price
            self.highest_price = start_price

        self.net_worth = self.initial_balance
        self.prev_net_worth = self.initial_balance

        # Original Cooldown
        self.cooldown = 0

        # 🆕 Consecutive Losses Tracking
        self.consecutive_losses = 0
        self.loss_pause_counter = 0  # Days remaining in pause

        # 🆕 Statistics
        self.total_trades = 0
        self.winning_trades = 0
        self.losing_trades = 0
        self.pause_triggered_count = 0

        return self._next_observation(), {}

    def _get_current_volatility(self):
        """Get current volatility multiplier for position sizing"""
        return self.vol_multiplier[min(self.current_step, len(self.vol_multiplier) - 1)]

    def _next_observation(self):
        """
        State vector (11 features):
        - Original 8 features
        - 🆕 3 new features for risk management
        """
        obs = self.df.iloc[self.current_step]
        current_vol = self._get_current_volatility()

        return np.array([
            # Original features
            obs.get('Macro_Signal', 0),
            obs.get('DL_Signal', 0.5),
            obs.get('Tech_Signal', 0),
            obs.get('ADX', 25) / 100.0,
            self.balance / 1_000_000.0,
            (self.shares_held * obs['Close']) / 1_000_000.0,
            obs.get('RSI', 50) / 100.0,
            obs.get('MACD', 0) / 10.0,
            # 🆕 New features
            current_vol,  # Current volatility multiplier
            self.consecutive_losses / self.max_consecutive_losses,  # Normalized loss streak
            self.loss_pause_counter / self.loss_pause_days  # Normalized pause counter
        ], dtype=np.float32)

    def _calculate_position_size(self):
        """
        🆕 Volatility-based Position Sizing

        Logic:
        - High volatility → Smaller position (30-50%)
        - Low volatility → Larger position (70-100%)

        Formula: position_pct = base_pct × volatility_multiplier
        """
        vol_mult = self._get_current_volatility()
        position_pct = self.base_position_pct * vol_mult

        # Clamp to reasonable range
        position_pct = np.clip(position_pct, 0.15, 0.70)  # 15% to 70%

        return position_pct

    def step(self, action):
        current_price = self.df.iloc[self.current_step]['Close']
        obs = self.df.iloc[self.current_step]

        reward = 0.0
        force_sell = False
        reason = ""

        # ============================================================
        # 🆕 CHECK: Max Consecutive Losses Pause
        # ============================================================
        if self.loss_pause_counter > 0:
            # Still in pause period - force HOLD
            self.loss_pause_counter -= 1
            action = 0  # Force Hold

            # 🆕 Penalty for being in pause (missed opportunity cost)
            reward -= 0.3

            if self.loss_pause_counter == 0:
                # Pause ended, reset consecutive losses
                self.consecutive_losses = 0

        # ============================================================
        # CHECK: Risk Management (Stop Loss / Trailing Stop)
        # ============================================================
        if self.shares_held > 0:
            if current_price > self.highest_price:
                self.highest_price = current_price
            pnl_pct = (current_price - self.avg_cost) / self.avg_cost

            # 1. Hard Stop Loss (-6%)
            if pnl_pct <= -0.06:
                force_sell = True
                reason = "SL"

            # 2. Trailing Stop (Trigger > +10%, Drop -4%)
            if pnl_pct > 0.10:
                drop = (self.highest_price - current_price) / self.highest_price
                if drop > 0.04:
                    force_sell = True
                    reason = "TRAIL"

        if force_sell:
            action = 2

        # ============================================================
        # CHECK: Original Cooldown (after sell)
        # ============================================================
        if self.cooldown > 0:
            if action == 1:
                action = 0  # Force Hold during cooldown
            self.cooldown -= 1

        # ============================================================
        # EXECUTE ACTION
        # ============================================================

        # ----- BUY -----
        if action == 1 and self.balance > current_price and self.loss_pause_counter == 0:
            can_buy = False
            rsi = obs.get('RSI', 50)
            adx = obs.get('ADX', 0)

            # Momentum Rule
            if adx > 25 and rsi < 75:
                can_buy = True
            elif rsi < 45:
                can_buy = True

            if can_buy:
                # 🆕 Volatility-based Position Sizing
                position_pct = self._calculate_position_size()
                budget = self.balance * position_pct

                shares = int(budget // current_price)
                if shares > 0:
                    cost = shares * current_price
                    self.avg_cost = ((self.shares_held * self.avg_cost) + cost) / (self.shares_held + shares)
                    self.balance -= cost
                    self.shares_held += shares
                    self.highest_price = current_price
                    self.total_trades += 1
                    reward += 0.1  # Small reward for action

        # ----- SELL -----
        elif action == 2 and self.shares_held > 0:
            revenue = self.shares_held * current_price
            pnl_pct = (current_price - self.avg_cost) / self.avg_cost

            self.balance += revenue
            self.shares_held = 0
            self.avg_cost = 0
            self.highest_price = 0
            self.cooldown = 2  # Original cooldown
            self.total_trades += 1

            # Track win/loss
            is_loss = pnl_pct < 0

            if reason == "TRAIL":
                reward += 2.0  # Big reward for trailing stop profit
                self.winning_trades += 1
                self.consecutive_losses = 0  # Reset streak

            elif reason == "SL":
                reward += 0.1  # Small reward for discipline
                self.losing_trades += 1

                # 🆕 Track consecutive losses
                self.consecutive_losses += 1

            else:
                if pnl_pct > 0:
                    reward += pnl_pct * 10
                    self.winning_trades += 1
                    self.consecutive_losses = 0  # Reset streak
                else:
                    reward -= 0.5  # Penalty for bad sell
                    self.losing_trades += 1

                    # 🆕 Track consecutive losses
                    self.consecutive_losses += 1

            # ============================================================
            # 🆕 CHECK: Trigger Loss Pause
            # ============================================================
            if self.consecutive_losses >= self.max_consecutive_losses:
                self.loss_pause_counter = self.loss_pause_days
                self.pause_triggered_count += 1

                # 🆕 Heavy penalty for triggering pause
                reward -= 1.5

                # Note: consecutive_losses will reset when pause ends

        # ============================================================
        # STEP FORWARD
        # ============================================================
        self.current_step += 1
        done = self.current_step >= len(self.df) - 1

        # Calculate new net worth
        if not done:
            next_val = self.balance + (self.shares_held * self.df.iloc[self.current_step]['Close'])
        else:
            next_val = self.balance + (self.shares_held * current_price)

        reward += ((next_val - self.prev_net_worth) / self.prev_net_worth) * 100
        self.prev_net_worth = next_val

        return self._next_observation(), reward, done, False, {}

    def get_stats(self):
        """Return trading statistics"""
        return {
            'total_trades': self.total_trades,
            'winning_trades': self.winning_trades,
            'losing_trades': self.losing_trades,
            'pause_triggered_count': self.pause_triggered_count,
            'win_rate': self.winning_trades / max(1, self.total_trades)
        }


# ============================================================
# 5. BACKTESTING FUNCTION
# ============================================================
def backtest_model(model, test_df):
    """Backtest trained model on test data"""
    env = StockTradingEnv(test_df)
    obs, _ = env.reset()

    total_reward = 0
    actions = []
    net_worths = [env.initial_balance]

    done = False
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, _, _ = env.step(action)
        total_reward += reward
        actions.append(action)

        current_price = test_df.iloc[min(env.current_step, len(test_df)-1)]['Close']
        net_worths.append(env.balance + (env.shares_held * current_price))

    # Calculate metrics
    final_value = net_worths[-1]
    initial_value = env.initial_balance
    total_return = (final_value - initial_value) / initial_value

    # Maximum Drawdown
    peak = net_worths[0]
    max_drawdown = 0
    for nw in net_worths:
        if nw > peak:
            peak = nw
        drawdown = (peak - nw) / peak
        if drawdown > max_drawdown:
            max_drawdown = drawdown

    # Action counts
    n_buys = sum(1 for a in actions if a == 1)
    n_sells = sum(1 for a in actions if a == 2)
    n_holds = sum(1 for a in actions if a == 0)

    # Get env stats
    stats = env.get_stats()

    return {
        'Total_Return': total_return,
        'Final_Value': final_value,
        'Max_Drawdown': max_drawdown,
        'Total_Reward': total_reward,
        'N_Buys': n_buys,
        'N_Sells': n_sells,
        'N_Holds': n_holds,
        'N_Trades': stats['total_trades'],
        'Win_Rate': stats['win_rate'],
        'Pause_Triggered': stats['pause_triggered_count']
    }


# ============================================================
# 6. MAIN TRAINING LOOP
# ============================================================
print("=" * 70)
print("🤖 BLOCK 7: PPO Training (UPGRADED + ROLLING WINDOW + RISK MGMT V2)")
print("=" * 70)
print("""
🔄 Rolling Window Configuration:
   - Window 1: Train [2015-2018] → Test [2019]
   - Window 2: Train [2016-2019] → Test [2020]
   - Window 3: Train [2017-2020] → Test [2021]
   - Window 4: Train [2018-2021] → Test [2022]
   - Window 5: Train [2019-2022] → Test [2023]
   - Window 6: Train [2020-2023] → Test [2024]
   - Window 7: Train [2021-2024] → Test [2025]

🛡️ Risk Management:
   - Stop Loss: -6%
   - Trailing Stop: Trigger +10%, Drop -4%
   - Cooldown: 2 days after sell

🆕 NEW Risk Management:
   - Volatility-based Position Sizing (30-70% based on volatility)
   - Max Consecutive Losses: 5 losses → Pause 3 days
   - Penalty: -0.3 per pause day, -1.5 when pause triggered

⚙️ PPO Configuration:
   - Learning Rate: 0.0003
   - Entropy Coef: 0.01
   - Batch Size: 64
   - Timesteps: 30,000 per window
""")
print("=" * 70)

# Storage
all_backtest_metrics = []

for stock in target_stocks:
    print(f"\n{'='*60}")
    print(f"🔹 Processing: {stock}")
    print(f"{'='*60}")

    stock_df = df_rl_all[df_rl_all['Stock'] == stock].copy()

    if len(stock_df) < 200:
        print(f"⚠️ Skipped (insufficient data: {len(stock_df)} rows)")
        continue

    stock_metrics = []

    for train_start, train_end, test_year in ROLLING_WINDOWS:
        train_df = stock_df[(stock_df['Year'] >= train_start) & (stock_df['Year'] <= train_end)].copy()
        test_df = stock_df[stock_df['Year'] == test_year].copy()

        if len(train_df) < 50:
            print(f"   ⚠️ Window {train_start}-{train_end} → {test_year}: Insufficient train data")
            continue

        if len(test_df) < 10:
            print(f"   ⚠️ Window {train_start}-{train_end} → {test_year}: Insufficient test data")
            continue

        try:
            # Create environment with new parameters
            env = DummyVecEnv([lambda df=train_df: StockTradingEnv(
                df,
                volatility_lookback=20,
                base_position_pct=0.5,
                max_consecutive_losses=5,
                loss_pause_days=3
            )])

            model = PPO(
                "MlpPolicy",
                env,
                verbose=0,
                learning_rate=0.0003,
                ent_coef=0.01,
                batch_size=64
            )

            model.learn(total_timesteps=30000)

            # Save model
            model_path = f"trained_models/ppo_{stock}_train{train_start}_{train_end}_test{test_year}"
            model.save(model_path)

            # Backtest
            metrics = backtest_model(model, test_df)
            metrics['Stock'] = stock
            metrics['Train_Period'] = f"{train_start}-{train_end}"
            metrics['Test_Year'] = test_year
            metrics['Train_Samples'] = len(train_df)
            metrics['Test_Samples'] = len(test_df)

            stock_metrics.append(metrics)
            all_backtest_metrics.append(metrics)

            print(f"   ✅ Window {train_start}-{train_end} → {test_year}: "
                  f"Return={metrics['Total_Return']:+.2%} | MDD={metrics['Max_Drawdown']:.2%} | "
                  f"WinRate={metrics['Win_Rate']:.1%} | Pauses={metrics['Pause_Triggered']}")

        except Exception as e:
            print(f"   ❌ Window {train_start}-{train_end} → {test_year}: Error - {e}")

    # Summary for this stock
    if stock_metrics:
        avg_return = np.mean([m['Total_Return'] for m in stock_metrics])
        avg_mdd = np.mean([m['Max_Drawdown'] for m in stock_metrics])
        avg_winrate = np.mean([m['Win_Rate'] for m in stock_metrics])
        total_pauses = sum([m['Pause_Triggered'] for m in stock_metrics])
        print(f"\n   📊 {stock} Average: Return={avg_return:+.2%} | MDD={avg_mdd:.2%} | "
              f"WinRate={avg_winrate:.1%} | Total Pauses={total_pauses}")


# ============================================================
# 7. OVERALL SUMMARY
# ============================================================
print("\n" + "=" * 70)
print("📊 OVERALL PERFORMANCE SUMMARY")
print("=" * 70)

if all_backtest_metrics:
    df_metrics = pd.DataFrame(all_backtest_metrics)

    # Summary by Test Year
    print("\n📅 Performance by Test Year:")
    print("-" * 70)
    year_summary = df_metrics.groupby('Test_Year').agg({
        'Total_Return': ['mean', 'std'],
        'Max_Drawdown': 'mean',
        'Win_Rate': 'mean',
        'Pause_Triggered': 'sum'
    }).round(4)
    print(year_summary.to_string())

    # Summary by Stock
    print("\n📈 Performance by Stock:")
    print("-" * 70)
    stock_summary = df_metrics.groupby('Stock').agg({
        'Total_Return': 'mean',
        'Max_Drawdown': 'mean',
        'Win_Rate': 'mean',
        'Pause_Triggered': 'sum'
    }).round(4).sort_values('Total_Return', ascending=False)
    print(stock_summary.to_string())

    # Overall Average
    print("\n" + "=" * 70)
    print("🎯 OVERALL AVERAGE PERFORMANCE")
    print("=" * 70)
    print(f"   • Average Return:      {df_metrics['Total_Return'].mean():+.2%}")
    print(f"   • Std Dev Return:      {df_metrics['Total_Return'].std():.2%}")
    print(f"   • Average Max DD:      {df_metrics['Max_Drawdown'].mean():.2%}")
    print(f"   • Average Win Rate:    {df_metrics['Win_Rate'].mean():.1%}")
    print(f"   • Total Pause Events:  {df_metrics['Pause_Triggered'].sum()}")
    print(f"   • Win Rate (Return>0): {(df_metrics['Total_Return'] > 0).mean():.1%}")

    # Best and Worst
    best = df_metrics.loc[df_metrics['Total_Return'].idxmax()]
    worst = df_metrics.loc[df_metrics['Total_Return'].idxmin()]

    print(f"\n   🏆 Best:  {best['Stock']} ({best['Train_Period']} → {best['Test_Year']}): {best['Total_Return']:+.2%}")
    print(f"   💀 Worst: {worst['Stock']} ({worst['Train_Period']} → {worst['Test_Year']}): {worst['Total_Return']:+.2%}")

    # Export
    outfile = "PPO_Backtest_Results_RollingWindow_V2.xlsx"
    df_metrics.to_excel(outfile, index=False)
    print(f"\n💾 Results saved to: {outfile}")

print("\n" + "=" * 70)
print("✅ BLOCK 7 COMPLETED: PPO + Rolling Window + Risk Management V2")
print("=" * 70)

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


🤖 BLOCK 7: PPO Training (UPGRADED + ROLLING WINDOW + RISK MGMT V2)

🔄 Rolling Window Configuration:
   - Window 1: Train [2015-2018] → Test [2019]
   - Window 2: Train [2016-2019] → Test [2020]
   - Window 3: Train [2017-2020] → Test [2021]
   - Window 4: Train [2018-2021] → Test [2022]
   - Window 5: Train [2019-2022] → Test [2023]
   - Window 6: Train [2020-2023] → Test [2024]
   - Window 7: Train [2021-2024] → Test [2025]

🛡️ Risk Management:
   - Stop Loss: -6%
   - Trailing Stop: Trigger +10%, Drop -4%
   - Cooldown: 2 days after sell
   
🆕 NEW Risk Management:
   - Volatility-based Position Sizing (30-70% based on volatility)
   - Max Consecutive Losses: 5 losses → Pause 3 days
   - Penalty: -0.3 per pause day, -1.5 when pause triggered

⚙️ PPO Configuration:
   - Learning Rate: 0.0003
   - Entropy Coef: 0.01
   - Batch Size: 64
   - Timesteps: 30,000 per window


🔹 Processing: BBL
   ✅ Window 2015-2018 → 2019: Return=-20.81% | MDD=24.74% | WinRate=0.0% | Pauses=0
   ✅ Window 201

In [ ]:
# ============================================================
# 🏆 BLOCK 8: Evaluation & Prediction (UPGRADED)
#    Features:
#    - Star Rating System
#    - 🆕 Performance Metrics: Sharpe Ratio, Max Drawdown, Calmar Ratio
#    - 🆕 Benchmark Comparison vs Buy-and-Hold
#    - 🆕 Rolling Window Evaluation
# ============================================================
import pandas as pd
import numpy as np
import warnings
import os
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO

warnings.filterwarnings("ignore")

# ============================================================
# 1. CONFIGURATION
# ============================================================

# [
#     "AAV", "AEONTS", "AOT", "BA", "BBL", "BCH", "BDMS", "BEM", "BH", "BTS",
#     "CHG", "JMT", "KBANK", "KKP", "KTB", "KTC", "PR9", "PRM", "RCL", "SAWAD",
#     "TCAP", "TISCO", "TTB"
# ]

stock_symbols = ["BBL","KBANK","KKP","KTB","TCAP","TISCO","TTB"]

INITIAL_BALANCE = 1_000_000
COMMISSION = 0.00157
RISK_FREE_RATE = 0.02  # 2% annual risk-free rate (Thai government bond ~2%)
TRADING_DAYS_PER_YEAR = 245  # Thai market

# Rolling Window Configuration
ROLLING_WINDOWS = [
    (2015, 2018, 2019),
    (2016, 2019, 2020),
    (2017, 2020, 2021),
    (2018, 2021, 2022),
    (2019, 2022, 2023),
    (2020, 2023, 2024),
    (2021, 2024, 2025),
]

# ============================================================
# 2. LOAD DATA
# ============================================================
try:
    df_rl_all = pd.read_excel("Final_RL_Input_Ready.xlsx")
    df_rl_all['Date'] = pd.to_datetime(df_rl_all['Date'])
    df_rl_all['Year'] = df_rl_all['Date'].dt.year
except:
    raise RuntimeError("❌ Missing Data: Final_RL_Input_Ready.xlsx")

# ============================================================
# 3. TRADING ENVIRONMENT
# ============================================================
class StockTradingEnv(gym.Env):
    """Trading environment matching Block 7 V2"""
    def __init__(self, df):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.balance = INITIAL_BALANCE
        self.shares_held = 0
        self.current_step = 0
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(11,), dtype=np.float32)
        self.action_space = spaces.Discrete(3)

    def reset(self, seed=None, options=None):
        self.current_step = 0
        self.balance = INITIAL_BALANCE
        self.shares_held = 0
        return self._next_observation(), {}

    def _next_observation(self):
        if self.current_step >= len(self.df):
            self.current_step = len(self.df) - 1
        obs = self.df.iloc[self.current_step]
        return np.array([
            obs.get('Macro_Signal', 0),
            obs.get('DL_Signal', 0.5),
            obs.get('Tech_Signal', 0),
            obs.get('ADX', 25) / 100.0,
            self.balance / 1_000_000.0,
            (self.shares_held * obs['Close']) / 1_000_000.0,
            obs.get('RSI', 50) / 100.0,
            obs.get('MACD', 0) / 10.0,
            0.5,  # Volatility placeholder
            0.0,  # Consecutive losses placeholder
            0.0   # Pause counter placeholder
        ], dtype=np.float32)

    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.df) - 1
        return self._next_observation(), 0, done, False, {}


# ============================================================
# 4. PERFORMANCE METRICS CALCULATOR
# ============================================================
class PerformanceMetrics:
    """
    Calculate comprehensive trading performance metrics
    """

    @staticmethod
    def calculate_returns(equity_curve):
        """Calculate daily returns from equity curve"""
        equity = np.array(equity_curve)
        returns = np.diff(equity) / equity[:-1]
        return returns

    @staticmethod
    def calculate_sharpe_ratio(returns, risk_free_rate=RISK_FREE_RATE,
                                trading_days=TRADING_DAYS_PER_YEAR):
        """
        Calculate annualized Sharpe Ratio

        Formula: Sharpe = (Mean Return - Risk Free Rate) / Std Dev × √(Trading Days)
        """
        if len(returns) < 2:
            return 0.0

        # Daily risk-free rate
        daily_rf = risk_free_rate / trading_days

        # Excess returns
        excess_returns = returns - daily_rf

        # Sharpe Ratio
        if np.std(excess_returns) == 0:
            return 0.0

        sharpe = np.mean(excess_returns) / np.std(excess_returns)

        # Annualize
        sharpe_annualized = sharpe * np.sqrt(trading_days)

        return sharpe_annualized

    @staticmethod
    def calculate_sortino_ratio(returns, risk_free_rate=RISK_FREE_RATE,
                                 trading_days=TRADING_DAYS_PER_YEAR):
        """
        Calculate annualized Sortino Ratio (downside risk only)

        Formula: Sortino = (Mean Return - Risk Free Rate) / Downside Std Dev × √(Trading Days)
        """
        if len(returns) < 2:
            return 0.0

        daily_rf = risk_free_rate / trading_days
        excess_returns = returns - daily_rf

        # Downside returns only
        downside_returns = excess_returns[excess_returns < 0]

        if len(downside_returns) == 0 or np.std(downside_returns) == 0:
            return 0.0

        sortino = np.mean(excess_returns) / np.std(downside_returns)
        sortino_annualized = sortino * np.sqrt(trading_days)

        return sortino_annualized

    @staticmethod
    def calculate_max_drawdown(equity_curve):
        """
        Calculate Maximum Drawdown (MDD)

        Formula: MDD = (Peak - Trough) / Peak
        """
        equity = np.array(equity_curve)
        peak = equity[0]
        max_dd = 0.0

        for value in equity:
            if value > peak:
                peak = value
            drawdown = (peak - value) / peak
            if drawdown > max_dd:
                max_dd = drawdown

        return max_dd

    @staticmethod
    def calculate_calmar_ratio(total_return, max_drawdown, years=1):
        """
        Calculate Calmar Ratio

        Formula: Calmar = Annualized Return / Max Drawdown
        """
        if max_drawdown == 0:
            return 0.0

        annualized_return = total_return / years if years > 0 else total_return
        calmar = annualized_return / max_drawdown

        return calmar

    @staticmethod
    def calculate_win_rate(trades):
        """Calculate win rate from list of trade PnLs"""
        if len(trades) == 0:
            return 0.0

        wins = sum(1 for t in trades if t > 0)
        return wins / len(trades)

    @staticmethod
    def calculate_profit_factor(trades):
        """
        Calculate Profit Factor

        Formula: Profit Factor = Sum(Winning Trades) / |Sum(Losing Trades)|
        """
        if len(trades) == 0:
            return 0.0

        gross_profit = sum(t for t in trades if t > 0)
        gross_loss = abs(sum(t for t in trades if t < 0))

        if gross_loss == 0:
            return float('inf') if gross_profit > 0 else 0.0

        return gross_profit / gross_loss

    @staticmethod
    def calculate_all_metrics(equity_curve, trade_pnls, years=1):
        """Calculate all metrics at once"""
        returns = PerformanceMetrics.calculate_returns(equity_curve)

        total_return = (equity_curve[-1] - equity_curve[0]) / equity_curve[0]
        max_dd = PerformanceMetrics.calculate_max_drawdown(equity_curve)

        return {
            'Total_Return': total_return,
            'Total_Return_Pct': total_return * 100,
            'Annualized_Return': (total_return / years) * 100 if years > 0 else total_return * 100,
            'Sharpe_Ratio': PerformanceMetrics.calculate_sharpe_ratio(returns),
            'Sortino_Ratio': PerformanceMetrics.calculate_sortino_ratio(returns),
            'Max_Drawdown': max_dd,
            'Max_Drawdown_Pct': max_dd * 100,
            'Calmar_Ratio': PerformanceMetrics.calculate_calmar_ratio(total_return, max_dd, years),
            'Win_Rate': PerformanceMetrics.calculate_win_rate(trade_pnls),
            'Win_Rate_Pct': PerformanceMetrics.calculate_win_rate(trade_pnls) * 100,
            'Profit_Factor': PerformanceMetrics.calculate_profit_factor(trade_pnls),
            'Total_Trades': len(trade_pnls),
            'Final_Value': equity_curve[-1]
        }


# ============================================================
# 5. BACKTEST WITH DETAILED TRACKING
# ============================================================
def backtest_with_metrics(df_test, model_path):
    """
    Backtest with detailed equity tracking for metrics calculation

    Returns:
        dict: All performance metrics + equity curve
    """
    env = StockTradingEnv(df_test)
    model = PPO.load(model_path)
    prices = df_test['Close'].values
    dates = df_test['Date'].values

    cash = INITIAL_BALANCE
    shares = 0
    avg_cost = 0.0
    highest_price = 0.0
    cooldown = 0
    consecutive_losses = 0
    loss_pause_counter = 0

    # Tracking
    equity_curve = [INITIAL_BALANCE]
    trade_pnls = []
    actions_taken = []

    obs, _ = env.reset()

    for i in range(len(prices) - 1):
        action, _ = model.predict(obs, deterministic=True)
        curr_price = prices[i]

        # Loss Pause Check
        if loss_pause_counter > 0:
            action = 0
            loss_pause_counter -= 1
            if loss_pause_counter == 0:
                consecutive_losses = 0

        # Risk Check
        force_sell = False
        reason = ""

        if shares > 0:
            if curr_price > highest_price:
                highest_price = curr_price
            pnl_pct = (curr_price - avg_cost) / avg_cost if avg_cost > 0 else 0

            if pnl_pct <= -0.06:
                force_sell = True
                reason = "SL"
            if pnl_pct > 0.10:
                if (highest_price - curr_price) / highest_price > 0.04:
                    force_sell = True
                    reason = "TRAIL"

        if force_sell:
            action = 2

        # Cooldown Check
        if cooldown > 0:
            if action == 1:
                action = 0
            cooldown -= 1

        # Execute Action
        if action == 1 and cash > curr_price and loss_pause_counter == 0:
            row = df_test.iloc[i]
            can_buy = False

            if (row.get('ADX', 0) > 25 and row.get('RSI', 50) < 75) or row.get('RSI', 50) < 45:
                can_buy = True

            if can_buy:
                # Volatility-based sizing (simplified)
                position_pct = 0.5
                budget = cash * position_pct
                buy_amt = int(budget // (curr_price * (1 + COMMISSION)))

                if buy_amt > 0:
                    cost = buy_amt * curr_price * (1 + COMMISSION)
                    total_shares = shares + buy_amt
                    avg_cost = ((shares * avg_cost) + cost) / total_shares if total_shares > 0 else curr_price
                    cash -= cost
                    shares += buy_amt
                    highest_price = curr_price
                    actions_taken.append(('BUY', i, curr_price, buy_amt))

        elif action == 2 and shares > 0:
            revenue = shares * curr_price * (1 - COMMISSION)
            pnl = revenue - (shares * avg_cost)
            pnl_pct = pnl / (shares * avg_cost) if avg_cost > 0 else 0

            trade_pnls.append(pnl)
            actions_taken.append(('SELL', i, curr_price, shares, pnl))

            cash += revenue
            shares = 0
            cooldown = 2

            # Track consecutive losses
            if pnl < 0:
                consecutive_losses += 1
                if consecutive_losses >= 5:
                    loss_pause_counter = 3
                    consecutive_losses = 0
            else:
                consecutive_losses = 0

            avg_cost = 0
            highest_price = 0

        # Update equity curve
        current_equity = cash + (shares * prices[i + 1])
        equity_curve.append(current_equity)

        obs, _, done, _, _ = env.step(action)
        if done:
            break

    # Final value
    final_value = cash + (shares * prices[-1])
    if len(equity_curve) < len(prices):
        equity_curve.append(final_value)

    # Calculate years
    days = len(df_test)
    years = days / TRADING_DAYS_PER_YEAR

    # Calculate all metrics
    metrics = PerformanceMetrics.calculate_all_metrics(equity_curve, trade_pnls, years)
    metrics['equity_curve'] = equity_curve
    metrics['trade_pnls'] = trade_pnls
    metrics['actions'] = actions_taken

    return metrics


# ============================================================
# 6. BUY-AND-HOLD BENCHMARK
# ============================================================
def calculate_buy_and_hold(df_test):
    """
    Calculate Buy-and-Hold benchmark performance

    Strategy: Buy at first day, hold until last day
    """
    prices = df_test['Close'].values

    # Buy at first price
    first_price = prices[0]
    shares = int(INITIAL_BALANCE // (first_price * (1 + COMMISSION)))
    cost = shares * first_price * (1 + COMMISSION)
    remaining_cash = INITIAL_BALANCE - cost

    # Build equity curve
    equity_curve = []
    for price in prices:
        equity = remaining_cash + (shares * price)
        equity_curve.append(equity)

    # Final value (sell at last price)
    final_value = remaining_cash + (shares * prices[-1] * (1 - COMMISSION))

    # Calculate metrics
    days = len(df_test)
    years = days / TRADING_DAYS_PER_YEAR

    # Simple trade PnL (one buy, one sell)
    trade_pnl = final_value - INITIAL_BALANCE

    metrics = PerformanceMetrics.calculate_all_metrics(equity_curve, [trade_pnl], years)
    metrics['equity_curve'] = equity_curve

    return metrics


# ============================================================
# 7. STAR RATING LOGIC
# ============================================================
def get_star_rating(action, tech_score, sharpe_ratio=0):
    """Enhanced star rating with Sharpe consideration"""
    base_rating = 3

    if action == 1:  # BUY
        if tech_score > 0.2:
            base_rating = 5  # Strong Buy
        else:
            base_rating = 4  # Buy
    elif action == 0:  # HOLD
        base_rating = 3
    elif action == 2:  # SELL
        if tech_score < -0.2:
            base_rating = 1  # Strong Sell
        else:
            base_rating = 2  # Sell

    # Adjust based on model performance (Sharpe)
    if sharpe_ratio > 1.5:
        base_rating = min(5, base_rating + 1)
    elif sharpe_ratio < 0:
        base_rating = max(1, base_rating - 1)

    return "⭐" * base_rating


# ============================================================
# 8. MAIN EVALUATION
# ============================================================
print("=" * 80)
print("🏆 BLOCK 8: Evaluation & Prediction (UPGRADED)")
print("=" * 80)
print("""
📊 Performance Metrics:
   - Total Return (%)
   - Annualized Return (%)
   - Sharpe Ratio
   - Sortino Ratio
   - Maximum Drawdown (%)
   - Calmar Ratio
   - Win Rate (%)
   - Profit Factor

📈 Benchmark: Buy-and-Hold Strategy
""")
print("=" * 80)

# Storage
all_results = []
all_comparisons = []
forecasts = []

# Process each stock
for ticker in stock_symbols:
    print(f"\n{'='*60}")
    print(f"🔹 Evaluating: {ticker}")
    print(f"{'='*60}")

    try:
        stock_df = df_rl_all[df_rl_all['Stock'] == ticker].copy()

        if len(stock_df) < 100:
            print(f"⚠️ Skipped (insufficient data: {len(stock_df)} rows)")
            continue

        stock_results = []

        # Evaluate each rolling window
        for train_start, train_end, test_year in ROLLING_WINDOWS:
            df_test = stock_df[stock_df['Year'] == test_year].copy()

            if len(df_test) < 10:
                continue

            # Model path (from Block 7 rolling window)
            model_path = f"trained_models/ppo_{ticker}_train{train_start}_{train_end}_test{test_year}.zip"

            # Fallback to old model path
            if not os.path.exists(model_path):
                model_path = f"trained_models/ppo_{ticker}.zip"

            if not os.path.exists(model_path):
                continue

            # Run backtest
            ai_metrics = backtest_with_metrics(df_test, model_path)
            bh_metrics = calculate_buy_and_hold(df_test)

            # Comparison
            result = {
                'Stock': ticker,
                'Test_Year': test_year,
                'Train_Period': f"{train_start}-{train_end}",

                # AI Strategy Metrics
                'AI_Return_%': round(ai_metrics['Total_Return_Pct'], 2),
                'AI_Sharpe': round(ai_metrics['Sharpe_Ratio'], 2),
                'AI_Sortino': round(ai_metrics['Sortino_Ratio'], 2),
                'AI_MDD_%': round(ai_metrics['Max_Drawdown_Pct'], 2),
                'AI_Calmar': round(ai_metrics['Calmar_Ratio'], 2),
                'AI_WinRate_%': round(ai_metrics['Win_Rate_Pct'], 1),
                'AI_ProfitFactor': round(ai_metrics['Profit_Factor'], 2),
                'AI_Trades': ai_metrics['Total_Trades'],

                # Benchmark Metrics
                'BH_Return_%': round(bh_metrics['Total_Return_Pct'], 2),
                'BH_Sharpe': round(bh_metrics['Sharpe_Ratio'], 2),
                'BH_MDD_%': round(bh_metrics['Max_Drawdown_Pct'], 2),

                # Comparison
                'Return_Diff_%': round(ai_metrics['Total_Return_Pct'] - bh_metrics['Total_Return_Pct'], 2),
                'Sharpe_Diff': round(ai_metrics['Sharpe_Ratio'] - bh_metrics['Sharpe_Ratio'], 2),
                'MDD_Diff_%': round(bh_metrics['Max_Drawdown_Pct'] - ai_metrics['Max_Drawdown_Pct'], 2),
                'Beat_BH': ai_metrics['Total_Return_Pct'] > bh_metrics['Total_Return_Pct']
            }

            stock_results.append(result)
            all_results.append(result)

            # Print summary
            status = "✅ WIN" if result['Beat_BH'] else "❌ LOSS"
            print(f"   {test_year}: AI={result['AI_Return_%']:+.1f}% vs BH={result['BH_Return_%']:+.1f}% "
                  f"| Sharpe={result['AI_Sharpe']:.2f} | MDD={result['AI_MDD_%']:.1f}% | {status}")

        # Stock Summary
        if stock_results:
            avg_ai_return = np.mean([r['AI_Return_%'] for r in stock_results])
            avg_bh_return = np.mean([r['BH_Return_%'] for r in stock_results])
            avg_sharpe = np.mean([r['AI_Sharpe'] for r in stock_results])
            win_count = sum(1 for r in stock_results if r['Beat_BH'])

            print(f"\n   📊 {ticker} Summary:")
            print(f"      Avg AI Return: {avg_ai_return:+.2f}%")
            print(f"      Avg BH Return: {avg_bh_return:+.2f}%")
            print(f"      Avg Sharpe:    {avg_sharpe:.2f}")
            print(f"      Beat BH:       {win_count}/{len(stock_results)} windows")

            # Generate forecast for latest data
            latest_df = stock_df[stock_df['Year'] == stock_df['Year'].max()].copy()
            if len(latest_df) > 0:
                latest_model = f"trained_models/ppo_{ticker}.zip"
                if os.path.exists(latest_model):
                    env = StockTradingEnv(latest_df)
                    obs, _ = env.reset()
                    env.current_step = len(latest_df) - 1
                    obs = env._next_observation()

                    model = PPO.load(latest_model)
                    action, _ = model.predict(obs, deterministic=True)

                    last_close = latest_df.iloc[-1]['Close']
                    tech_score = latest_df.iloc[-1].get('Tech_Signal', 0)
                    rating = get_star_rating(action, tech_score, avg_sharpe)

                    act_str = "BUY" if action == 1 else "SELL" if action == 2 else "HOLD"
                    forecasts.append({
                        'Stock': ticker,
                        'Last_Price': round(last_close, 2),
                        'AI_Action': act_str,
                        'Tech_Score': round(tech_score, 2),
                        'Avg_Sharpe': round(avg_sharpe, 2),
                        'Rating': rating
                    })

    except Exception as e:
        print(f"❌ Error: {e}")


# ============================================================
# 9. OVERALL SUMMARY
# ============================================================
print("\n" + "=" * 80)
print("📊 OVERALL PERFORMANCE SUMMARY")
print("=" * 80)

if all_results:
    df_results = pd.DataFrame(all_results)

    # Summary by Year
    print("\n📅 Performance by Test Year:")
    print("-" * 80)
    year_summary = df_results.groupby('Test_Year').agg({
        'AI_Return_%': 'mean',
        'BH_Return_%': 'mean',
        'AI_Sharpe': 'mean',
        'AI_MDD_%': 'mean',
        'Beat_BH': 'mean'
    }).round(2)
    year_summary.columns = ['AI_Return', 'BH_Return', 'Sharpe', 'MDD', 'Win_Rate']
    print(year_summary.to_string())

    # Summary by Stock
    print("\n📈 Performance by Stock:")
    print("-" * 80)
    stock_summary = df_results.groupby('Stock').agg({
        'AI_Return_%': 'mean',
        'BH_Return_%': 'mean',
        'AI_Sharpe': 'mean',
        'AI_MDD_%': 'mean',
        'Beat_BH': 'mean'
    }).round(2).sort_values('AI_Return_%', ascending=False)
    stock_summary.columns = ['AI_Return', 'BH_Return', 'Sharpe', 'MDD', 'Win_Rate']
    print(stock_summary.to_string())

    # Overall Statistics
    print("\n" + "=" * 80)
    print("🎯 OVERALL STATISTICS")
    print("=" * 80)

    print(f"""
┌─────────────────────────────────────────────────────────────────────────┐
│                        AI STRATEGY vs BUY-AND-HOLD                      │
├─────────────────────────────────────────────────────────────────────────┤
│                            │     AI Strategy    │    Buy-and-Hold      │
├─────────────────────────────────────────────────────────────────────────┤
│ Average Return             │  {df_results['AI_Return_%'].mean():>+10.2f}%     │  {df_results['BH_Return_%'].mean():>+10.2f}%      │
│ Median Return              │  {df_results['AI_Return_%'].median():>+10.2f}%     │  {df_results['BH_Return_%'].median():>+10.2f}%      │
│ Std Dev Return             │  {df_results['AI_Return_%'].std():>10.2f}%     │  {df_results['BH_Return_%'].std():>10.2f}%      │
│ Average Sharpe Ratio       │  {df_results['AI_Sharpe'].mean():>10.2f}       │  {df_results['BH_Sharpe'].mean():>10.2f}        │
│ Average Max Drawdown       │  {df_results['AI_MDD_%'].mean():>10.2f}%     │  {df_results['BH_MDD_%'].mean():>10.2f}%      │
├─────────────────────────────────────────────────────────────────────────┤
│ Win Rate vs Benchmark      │  {df_results['Beat_BH'].mean()*100:>10.1f}%     │        -             │
│ Total Test Windows         │  {len(df_results):>10d}         │        -             │
└─────────────────────────────────────────────────────────────────────────┘
""")

    # Best and Worst Performers
    best = df_results.loc[df_results['AI_Return_%'].idxmax()]
    worst = df_results.loc[df_results['AI_Return_%'].idxmin()]
    best_sharpe = df_results.loc[df_results['AI_Sharpe'].idxmax()]

    print(f"   🏆 Best Return:  {best['Stock']} ({best['Test_Year']}): {best['AI_Return_%']:+.2f}%")
    print(f"   💀 Worst Return: {worst['Stock']} ({worst['Test_Year']}): {worst['AI_Return_%']:+.2f}%")
    print(f"   📈 Best Sharpe:  {best_sharpe['Stock']} ({best_sharpe['Test_Year']}): {best_sharpe['AI_Sharpe']:.2f}")

    # Export detailed results
    outfile_detailed = "Block8_Performance_Detailed.xlsx"
    df_results.to_excel(outfile_detailed, index=False)
    print(f"\n💾 Detailed results saved to: {outfile_detailed}")

# ============================================================
# 10. FORECAST OUTPUT
# ============================================================
if forecasts:
    print("\n" + "=" * 80)
    print("🔮 AI FORECAST FOR TOMORROW")
    print("=" * 80)

    df_forecast = pd.DataFrame(forecasts)

    # Sort by rating (descending)
    df_forecast['Rating_Num'] = df_forecast['Rating'].apply(len)
    df_forecast = df_forecast.sort_values('Rating_Num', ascending=False)

    print(f"\n{'Stock':<10} {'Price':<10} {'Action':<8} {'Tech':<8} {'Sharpe':<8} {'Rating':<12}")
    print("-" * 60)

    for _, row in df_forecast.iterrows():
        print(f"{row['Stock']:<10} {row['Last_Price']:<10.2f} {row['AI_Action']:<8} "
              f"{row['Tech_Score']:<8.2f} {row['Avg_Sharpe']:<8.2f} {row['Rating']:<12}")

    # Export
    outfile_forecast = "Block8_AI_Forecast.xlsx"
    df_forecast.drop(columns=['Rating_Num']).to_excel(outfile_forecast, index=False)
    print(f"\n💾 Forecast saved to: {outfile_forecast}")

print("\n" + "=" * 80)
print("✅ BLOCK 8 COMPLETED: Evaluation with Sharpe, MDD & Benchmark Comparison")
print("=" * 80)

🏆 BLOCK 8: Evaluation & Prediction (UPGRADED)

📊 Performance Metrics:
   - Total Return (%)
   - Annualized Return (%)
   - Sharpe Ratio
   - Sortino Ratio
   - Maximum Drawdown (%)
   - Calmar Ratio
   - Win Rate (%)
   - Profit Factor

📈 Benchmark: Buy-and-Hold Strategy


🔹 Evaluating: BBL
   2019: AI=-20.9% vs BH=-18.5% | Sharpe=-1.31 | MDD=25.7% | ❌ LOSS
   2020: AI=+0.0% vs BH=-24.3% | Sharpe=0.00 | MDD=0.0% | ✅ WIN
   2021: AI=+0.0% vs BH=+3.8% | Sharpe=0.00 | MDD=0.0% | ❌ LOSS
   2022: AI=+0.0% vs BH=+21.9% | Sharpe=0.00 | MDD=0.0% | ❌ LOSS
   2023: AI=+0.5% vs BH=+6.9% | Sharpe=-0.00 | MDD=17.1% | ❌ LOSS
   2024: AI=+9.1% vs BH=+2.8% | Sharpe=0.51 | MDD=10.8% | ✅ WIN
   2025: AI=+17.6% vs BH=+18.4% | Sharpe=0.98 | MDD=12.5% | ❌ LOSS

   📊 BBL Summary:
      Avg AI Return: +0.89%
      Avg BH Return: +1.56%
      Avg Sharpe:    0.03
      Beat BH:       2/7 windows

🔹 Evaluating: KBANK
   2019: AI=+0.0% vs BH=-16.6% | Sharpe=0.00 | MDD=0.0% | ✅ WIN
   2020: AI=-6.9% vs BH=-22.9%

1) Change price frequentcy from day to hour
2) Edit Technical Block ✅
3) optimize DL signal using validation set ปรับ Block 5.5 (DL Signal)  ให้แม่นยำขึ้น ✅
4) เพิ่ม Feature ใหม่ๆ ใน Block 6 (เช่น Volume Profile, Bid/Offer) => ต้องขอจาก พี่ AIQ
5) จูน Hyperparameter ใน Block 7
6) แก้ไฟล์ Macro.xlsx เพื่อเพิ่ม Data ของ 2025 ทั้งหมด



## Excluded Part

In [ ]:
# # ============================================================
# # 🤖 BLOCK 7: Train PPO (Hierarchy: Macro First, DL Fallback)
# #    Concept: Trust Macro if aligned with Price Trend.
# #             If Macro conflicts with Price, Trust DL.
# # ============================================================
# import os
# import gymnasium as gym
# from gymnasium import spaces
# import numpy as np
# import pandas as pd
# from stable_baselines3 import PPO
# from stable_baselines3.common.vec_env import DummyVecEnv
# import warnings

# warnings.filterwarnings("ignore")
# os.makedirs("trained_models", exist_ok=True)

# # 1. รายชื่อหุ้น
# target_stocks = [
#     "AAV", "AEONTS", "AOT", "BA", "BBL", "BCH", "BDMS", "BEM", "BH", "BTS",
#     "CHG", "JMT", "KBANK", "KKP", "KTB", "KTC", "PR9", "PRM", "RCL", "SAWAD",
#     "TCAP", "TISCO", "TTB"
# ]

# # 2. โหลดข้อมูล
# try:
#     df_rl_all = pd.read_excel("Final_RL_Input_Ready.xlsx")
#     df_rl_all['Date'] = pd.to_datetime(df_rl_all['Date'])
#     print(f"✅ Loaded Training Data: {len(df_rl_all)} rows")
# except FileNotFoundError:
#     raise RuntimeError("❌ ไม่เจอไฟล์ Final_RL_Input_Ready.xlsx")

# # --- Environment Definition ---
# class StockTradingEnv(gym.Env):
#     def __init__(self, df, initial_balance=1_000_000):
#         super(StockTradingEnv, self).__init__()
#         self.df = df.reset_index(drop=True)
#         self.initial_balance = initial_balance
#         self.commission = 0.0 # Train 0%

#         self.action_space = spaces.Discrete(3) # 0=Hold, 1=Buy, 2=Sell
#         self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(6,), dtype=np.float32)

#         self.reset()

#     def reset(self, seed=None, options=None):
#         super().reset(seed=seed)
#         self.current_step = 0

#         # MIXED INIT (เริ่มเงินสด 50% / เริ่มหุ้น 50%)
#         if np.random.rand() < 0.5:
#             self.balance = self.initial_balance
#             self.shares_held = 0
#             self.avg_cost = 0.0
#         else:
#             start_price = self.df.iloc[0]['Close']
#             invest_pct = np.random.uniform(0.2, 0.8)
#             cash_invested = self.initial_balance * invest_pct
#             self.shares_held = int(cash_invested // start_price)
#             self.balance = self.initial_balance - (self.shares_held * start_price)
#             self.avg_cost = start_price

#         self.net_worth = self.initial_balance
#         self.prev_net_worth = self.initial_balance
#         self.days_since_trade = 0

#         return self._next_observation(), {}

#     def _next_observation(self):
#         obs = self.df.iloc[self.current_step]
#         # Manual Scaling
#         return np.array([
#             obs.get('Macro_Signal', 0),
#             obs.get('DL_Signal', 0.5),
#             self.balance / 1_000_000.0,
#             (self.shares_held * obs['Close']) / 1_000_000.0,
#             obs.get('RSI', 50) / 100.0,
#             obs.get('MACD', 0) / 10.0
#         ], dtype=np.float32)

#     def step(self, action):
#         current_price = self.df.iloc[self.current_step]['Close']

#         # ดึงค่าต่างๆ มาเช็ค Logic
#         obs = self.df.iloc[self.current_step]
#         macro_val = obs.get('Macro_Signal', 0) # -1 ถึง 1
#         dl_val = obs.get('DL_Signal', 0.5)     # 0 ถึง 1
#         macd_val = obs.get('MACD', 0)          # ใช้ MACD เป็นตัวแทน Trend กราฟราคา

#         # --- 🔥 CORE LOGIC: MACRO PROTAGONIST w/ REALITY CHECK ---

#         # 1. กำหนดมุมมองของ Macro
#         macro_view = "NEUTRAL"
#         if macro_val > 0.1: macro_view = "BULL"
#         elif macro_val < -0.1: macro_view = "BEAR"

#         # 2. กำหนดมุมมองของกราฟจริง (Price Trend)
#         # MACD > 0 คือเทรนด์ขาขึ้น, MACD < 0 คือเทรนด์ขาลง
#         price_trend = "BULL" if macd_val > 0 else "BEAR"

#         # 3. ตัดสินใจ (Decision Making)
#         market_state = "NEUTRAL"

#         # เงื่อนไข: Macro ตรงกับความเป็นจริงไหม?
#         if (macro_view == "BULL" and price_trend == "BULL") or \
#            (macro_view == "BEAR" and price_trend == "BEAR"):
#             # ✅ Macro แม่น -> เชื่อ Macro (พระเอกทำงาน)
#             market_state = macro_view
#         else:
#             # ❌ Macro มั่ว (ขัดแย้งกับกราฟ) -> เชื่อ DL (พระรองมาช่วย)
#             if dl_val > 0.55: market_state = "BULL"
#             elif dl_val < 0.45: market_state = "BEAR"
#             # ถ้า DL ก็ไม่แน่ใจ ให้เป็น NEUTRAL

#         # --- Reward Calculation ---
#         reward = 0.0

#         # Action Logic
#         # BUY
#         if action == 1 and self.balance > current_price:
#             budget = self.balance * 0.5
#             shares = int(budget // current_price)
#             if shares > 0:
#                 cost = shares * current_price
#                 total_shares = self.shares_held + shares
#                 total_cost = (self.shares_held * self.avg_cost) + cost
#                 self.avg_cost = total_cost / total_shares

#                 self.balance -= cost
#                 self.shares_held += shares
#                 self.days_since_trade = 0

#                 # ให้รางวัลถ้าซื้อถูกจังหวะ
#                 if market_state == "BULL": reward += 0.2
#                 if market_state == "BEAR": reward -= 0.2

#         # SELL
#         elif action == 2 and self.shares_held > 0:
#             revenue = self.shares_held * current_price
#             profit_pct = (current_price - self.avg_cost) / self.avg_cost

#             self.balance += revenue
#             self.shares_held = 0
#             self.avg_cost = 0
#             self.days_since_trade = 0

#             # Profit Bonus
#             if profit_pct > 0:
#                 reward += (profit_pct * 20.0)
#             else:
#                 # ถ้าตลาดแย่ แล้ว Cut Loss ได้ -> ชมเชย
#                 if market_state == "BEAR": reward += 0.1
#                 else: reward -= 0.1

#         # HOLD
#         elif action == 0:
#             self.days_since_trade += 1
#             if self.days_since_trade > 3: # Inactivity Penalty
#                 reward -= 0.1 * (self.days_since_trade - 3)

#         # Step Update
#         self.current_step += 1
#         done = self.current_step >= len(self.df) - 1

#         # PnL Reward
#         next_price = self.df.iloc[self.current_step]['Close']
#         self.net_worth = self.balance + (self.shares_held * next_price)
#         pnl = ((self.net_worth - self.prev_net_worth) / self.prev_net_worth) * 100
#         reward += pnl

#         # --- 🔥 REGIME PENALTY (Floor is Lava แบบมีสมอง) ---
#         cash_ratio = self.balance / self.net_worth

#         if market_state == "BULL":
#             # ตลาดดี (Macro แม่น หรือ DL ช่วยยืนยัน) -> ต้องถือหุ้น!
#             if cash_ratio > 0.5: reward -= 0.5

#         elif market_state == "BEAR":
#             # ตลาดแย่ -> ต้องถือเงินสด!
#             if cash_ratio < 0.5: reward -= 0.5

#         self.prev_net_worth = self.net_worth
#         return self._next_observation(), reward, done, False, {}

# # --- Training Loop ---
# print(f"🚀 Starting Training (Macro First -> Fallback to DL)...")

# TOTAL_TIMESTEPS = 30000

# for stock in target_stocks:
#     train_df = df_rl_all[(df_rl_all['Stock'] == stock) & (df_rl_all['Date'].dt.year < 2024)]
#     if len(train_df) < 50: continue

#     env = DummyVecEnv([lambda: StockTradingEnv(train_df)])
#     model = PPO("MlpPolicy", env, verbose=0, learning_rate=0.0003, ent_coef=0.1, batch_size=64)

#     try:
#         model.learn(total_timesteps=TOTAL_TIMESTEPS)
#         model.save(f"trained_models/ppo_{stock}")
#         print(f"✅ Trained: {stock}")
#     except Exception as e:
#         print(f"❌ Error {stock}: {e}")

# print("\n🎉 Training Complete!")

In [ ]:
# # ============================================================
# # 🏆 BLOCK 8: FINAL EVALUATION
# # ============================================================
# import pandas as pd
# import numpy as np
# import warnings
# import os
# import gymnasium as gym
# from gymnasium import spaces
# from stable_baselines3 import PPO
# from stable_baselines3.common.vec_env import DummyVecEnv

# warnings.filterwarnings("ignore")

# stock_symbols = [
#     "AAV", "AEONTS", "AOT", "BA", "BBL", "BCH", "BDMS", "BEM", "BH", "BTS",
#     "CHG", "JMT", "KBANK", "KKP", "KTB", "KTC", "PR9", "PRM", "RCL", "SAWAD",
#     "TCAP", "TISCO", "TTB"
# ]
# initial_balance = 1_000_000

# try:
#     df_rl_all = pd.read_excel("Final_RL_Input_Ready.xlsx")
#     df_rl_all['Date'] = pd.to_datetime(df_rl_all['Date'])
# except:
#     raise RuntimeError("❌ Missing Data File")

# class StockTradingEnv(gym.Env):
#     def __init__(self, df):
#         super().__init__()
#         self.df = df.reset_index(drop=True)
#         self.balance = initial_balance
#         self.shares_held = 0
#         self.current_step = 0
#         self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(6,), dtype=np.float32)
#         self.action_space = spaces.Discrete(3)

#     def reset(self, seed=None, options=None):
#         self.current_step = 0
#         self.balance = initial_balance
#         self.shares_held = 0
#         return self._next_observation(), {}

#     def _next_observation(self):
#         obs = self.df.iloc[self.current_step]
#         return np.array([
#             obs.get('Macro_Signal', 0),
#             obs.get('DL_Signal', 0.5),
#             self.balance / 1_000_000.0,
#             (self.shares_held * obs['Close']) / 1_000_000.0,
#             obs.get('RSI', 50) / 100.0,
#             obs.get('MACD', 0) / 10.0
#         ], dtype=np.float32)

#     def step(self, action):
#         self.current_step += 1
#         done = self.current_step >= len(self.df) - 1
#         return self._next_observation(), 0, done, False, {}

# # --- Backtest Logic ---
# def backtest_logic(env, model):
#     if hasattr(env, 'envs'): df = env.envs[0].unwrapped.df
#     else: df = env.unwrapped.df

#     prices = df['Close'].values
#     cash = initial_balance
#     shares = 0
#     trades = 0
#     commission = 0.001

#     obs = env.reset()

#     for i in range(len(prices)-1):
#         action, _ = model.predict(obs, deterministic=True)
#         curr_price = prices[i]

#         # Buy
#         if action == 1 and cash > curr_price:
#             budget = cash * 0.5
#             buy_amt = int(budget // (curr_price * (1 + commission)))
#             if buy_amt > 0:
#                 cost = buy_amt * curr_price * (1 + commission)
#                 cash -= cost
#                 shares += buy_amt
#                 trades += 1

#         # Sell
#         elif action == 2 and shares > 0:
#             cash += shares * curr_price * (1 - commission)
#             shares = 0
#             trades += 1

#         obs, _, done, _ = env.step(action)
#         if done: break

#     final_val = cash + (shares * prices[-1])
#     return final_val, trades

# # --- Run Loop ---
# results = []
# print(f"🚀 Testing on 2024-2025 Data (Macro-Led Strategy)...\n")
# print(f"{'Stock':<10} | {'AI %':<10} | {'Bench %':<10} | {'Diff %':<10} | {'Trades':<8} | {'Status'}")
# print("-" * 75)

# for ticker in stock_symbols:
#     try:
#         df_test = df_rl_all[(df_rl_all['Stock'] == ticker) & (df_rl_all['Date'].dt.year >= 2024)]
#         if len(df_test) < 10: continue

#         model_path = f"trained_models/ppo_{ticker}.zip"

#         if os.path.exists(model_path):
#             env = DummyVecEnv([lambda: StockTradingEnv(df_test)])
#             model = PPO.load(model_path, env=env)
#             final_val, trades = backtest_logic(env, model)

#             ai_ret = ((final_val - initial_balance)/initial_balance)*100
#             bench_ret = ((df_test.iloc[-1]['Close'] - df_test.iloc[0]['Close'])/df_test.iloc[0]['Close'])*100
#             diff = ai_ret - bench_ret
#             status = "WIN 🏆" if ai_ret > bench_ret else "LOSS ❌"

#             results.append({'Stock': ticker, 'AI_Return_%': round(ai_ret,2), 'Benchmark_Return_%': round(bench_ret,2), 'Diff_%': round(diff, 2), 'Trades': trades, 'Status': status})
#             print(f"{ticker:<10} | {ai_ret:>9.2f}% | {bench_ret:>9.2f}% | {diff:>9.2f}% | {trades:<8} | {status}")

#     except Exception as e:
#         print(f"❌ Error {ticker}: {e}")

# if results:
#     df_res = pd.DataFrame(results)
#     print(f"\n{df_res.to_markdown(index=False)}")
#     df_res.to_excel("Final_AI_WinLoss_Summary.xlsx", index=False)

# Backtesting

In [ ]:
# import pandas as pd
# import numpy as np

# # ============================================================
# # ⚙️ BLOCK: Historical vs Predicted ΔLogclose (Clean Export)
# # ============================================================

# def rolling_ecm_forecast_table(stock_name, sector_name, df_macro, df_gamma, df_alpha, df_beta, combined_dfs_selective_diff):
#     """
#     ทำนาย ΔLogclose (เดือน t) จากข้อมูลเดือน t−1 แล้วเปรียบเทียบกับ actual
#     คืน DataFrame: Date | Actual_dLogclose | Pred_dLogclose | Error | APE_% | Stock | Sector
#     """
#     # --- เตรียมข้อมูลหุ้นและ macro ---
#     df_stock = combined_dfs_selective_diff[sector_name][[f"Logclose_{stock_name}"]].copy()
#     df_stock.columns = ["Logclose"]
#     df_macro_aligned = df_macro.reindex(df_stock.index).ffill().copy()

#     # --- พารามิเตอร์จากโมเดล ---
#     btab = df_beta[df_beta["Stock"] == stock_name]
#     gtab = df_gamma[df_gamma["Stock"] == stock_name].copy()
#     atab = df_alpha[df_alpha["Stock"] == stock_name]
#     if btab.empty or gtab.empty or atab.empty:
#         return None

#     lr_params = btab.drop_duplicates(subset="Variable").set_index("Variable")["coef"]
#     gtab = gtab[gtab["Variable"].str.contains(r"D\.L\d+\.", regex=True, na=False)]
#     gtab = pd.concat([
#         gtab,
#         pd.DataFrame([{"Variable": "ECT_lag1", "Coef": float(atab["Alpha"].iloc[0])}])
#     ], ignore_index=True)

#     # --- เตรียม DataFrame ผลลัพธ์ ---
#     df_out = pd.DataFrame(index=df_stock.index)
#     df_out["Actual_dLogclose"] = df_stock["Logclose"].diff()
#     df_out["Pred_dLogclose"] = np.nan

#     # --- Forecast แบบ rolling ---
#     for i in range(4, len(df_stock)):
#         deltas = {}
#         past_window = df_stock.iloc[i-4:i].copy()
#         past_macro = df_macro_aligned.iloc[i-4:i].copy()

#         # ECT lag
#         const = float(lr_params.get("const", 0.0))
#         X_lag = {k: float(past_macro[k].iloc[-2]) for k in lr_params.index if k != "const" and k in past_macro.columns}
#         Y_lag = float(past_window["Logclose"].iloc[-2])
#         ect_val = Y_lag - (const + sum(float(lr_params[k]) * X_lag[k] for k in X_lag))
#         deltas["ECT_lag1"] = ect_val

#         # ΔY / ΔX lags
#         for var in gtab["Variable"]:
#             if var == "ECT_lag1":
#                 continue
#             parts = var.split(".")
#             lag = int(parts[1].replace("L", ""))
#             base = parts[2]

#             if base.startswith("Logclose_"):
#                 stock_base = base.replace("Logclose_", "")
#                 if stock_base != stock_name:
#                     continue
#                 dy = float(past_window["Logclose"].iloc[-lag]) - float(past_window["Logclose"].iloc[-(lag + 1)])
#                 deltas[var] = dy
#             else:
#                 if base in past_macro.columns:
#                     dx = float(past_macro[base].iloc[-lag]) - float(past_macro[base].iloc[-(lag + 1)])
#                     deltas[var] = dx
#                 else:
#                     deltas[var] = 0.0

#         df_pred = pd.DataFrame(list(deltas.items()), columns=["Variable", "Value"])
#         merged = pd.merge(gtab[["Variable", "Coef"]], df_pred, on="Variable", how="left")
#         merged["Contribution"] = merged["Coef"].astype(float) * merged["Value"].astype(float)
#         df_out.iloc[i, df_out.columns.get_loc("Pred_dLogclose")] = merged["Contribution"].sum()

#     # --- Error metrics ---
#     EPS = 1e-6
#     df_out["Error"] = df_out["Pred_dLogclose"] - df_out["Actual_dLogclose"]
#     df_out["APE_%"] = np.where(
#         df_out["Actual_dLogclose"].abs() > EPS,
#         np.abs(df_out["Error"] / df_out["Actual_dLogclose"]) * 100,
#         np.nan
#     )
#     df_out["Stock"] = stock_name
#     df_out["Sector"] = sector_name
#     return df_out.dropna(subset=["Pred_dLogclose"])


# # ============================================================
# # 🚀 Run & Export เฉพาะรายหุ้น (Sheet 2 เท่านั้น)
# # ============================================================

# evaluation_results = {}

# for sector_name, df_sec in combined_dfs_selective_diff.items():
#     for col in df_sec.columns:
#         if not col.startswith("Logclose_"):
#             continue
#         stock_name = col.replace("Logclose_", "")
#         try:
#             df_eval = rolling_ecm_forecast_table(
#                 stock_name, sector_name, df, df_gamma, df_alpha, df_beta, combined_dfs_selective_diff
#             )
#             if df_eval is not None and not df_eval.empty:
#                 evaluation_results[stock_name] = df_eval
#                 mae = df_eval["Error"].abs().mean()
#                 rmse = np.sqrt((df_eval["Error"] ** 2).mean())
#                 mape = df_eval["APE_%"].replace([np.inf, -np.inf], np.nan).mean()
#                 print(f"✅ {stock_name:<8} | MAE={mae:.5f} | RMSE={rmse:.5f} | MAPE={mape:.2f}% | Obs={len(df_eval)}")
#         except Exception as e:
#             print(f"⚠️ {stock_name} failed: {e}")

# # --- Export เฉพาะรายหุ้น ---
# output_file = "ECM_Forecast_Evaluation_ByStock.xlsx"
# with pd.ExcelWriter(output_file) as writer:
#     for stock, df_eval in evaluation_results.items():
#         df_clean = df_eval.replace([np.inf, -np.inf], np.nan).dropna(how="all")
#         df_clean.to_excel(writer, sheet_name=stock[:31], index=True)

# print(f"\n✅ Exported → {output_file}")
# print(f"รวมทั้งหมด {len(evaluation_results)} หุ้น (เฉพาะ Sheet รายหุ้นเท่านั้น)")


# # หลังจากรันแล้ว evaluation_results มี dictionary ของหุ้นแต่ละตัว

# for stock, df_eval in evaluation_results.items():
#     print(f"\n==== {stock} ====")
#     print(df_eval.tail(5)[["Actual_dLogclose","Pred_dLogclose","Error","APE_%"]])



In [ ]:
# import pandas as pd
# df_macro = pd.read_excel("Macro_data.xlsx")
# print(df_macro.columns.tolist())


In [ ]:
# # ============================================================
# # 🧠 BLOCK 5.2 — Regime-aware Hybrid Optimization Strategy (Fixed)
# # ============================================================

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import yfinance as yf
# from pathlib import Path

# # ---------- INPUT FILES ----------
# f_forecast = "Forecast_ARDL_ECM_NextMonth_Clean.xlsx"
# f_daily = "Historical_Technical_Indicators_bySector.xlsx"
# f_macro = "Macro_data.xlsx"

# # ---------- LOAD DATA ----------
# df_fore = pd.read_excel(f_forecast)[["Stock", "Sector", "Pred_dLogclose"]]
# df_daily = pd.read_excel(f_daily)
# df_daily["Date"] = pd.to_datetime(df_daily["Date"])

# # --- Load macro base ---
# df_macro = pd.read_excel(f_macro)
# df_macro["Date"] = pd.to_datetime(df_macro["Date"])

# # --- FX: USD/THB ---
# fx1 = yf.download("THB=X", start="2015-01-01", end="2025-01-01", progress=False)
# if isinstance(fx1.columns, pd.MultiIndex):
#     fx1.columns = fx1.columns.get_level_values(0)
# fx1 = fx1.reset_index()[["Date", "Close"]].rename(columns={"Close": "THB_per_USD"})

# # --- FX: THB/CNY ---

# fx2 = yf.download("THBCNY=X", start="2015-01-01", end="2025-01-01", progress=False)
# if isinstance(fx2.columns, pd.MultiIndex):
#     fx2.columns = fx2.columns.get_level_values(0)
# fx2 = fx2.reset_index()[["Date", "Close"]].rename(columns={"Close": "THB_per_CNY"})


# # # --- Oil: Brent ---
# # oil = yf.download("BZ=F", start="2015-01-01", end="2025-01-01", progress=False)
# # if isinstance(oil.columns, pd.MultiIndex):
# #     oil.columns = oil.columns.get_level_values(0)
# # oil = oil.reset_index()[["Date", "Close"]].rename(columns={"Close": "Brent_Oil_USD_per_bbl"})

# # --- SET Index---
# thai = yf.download("^SET.BK", start="2015-01-01", end="2025-01-01", progress=False)
# if isinstance(thai.columns, pd.MultiIndex):
#     thai.columns = thai.columns.get_level_values(0)
# thai = thai.reset_index()[["Date", "Close"]].rename(columns={"Close": "SET_Index"})

# # --- S&P500---
# usa = yf.download("^GSPC", start="2015-01-01", end="2025-01-01", progress=False)
# if isinstance(usa.columns, pd.MultiIndex):
#     usa.columns = usa.columns.get_level_values(0)
# usa = usa.reset_index()[["Date", "Close"]].rename(columns={"Close": "SP500_Index"})

# # --- Merge macro + yfinance series ---
# df_macro = (
#     df_macro
#     .merge(fx1, on="Date", how="left")
#     .merge(fx2, on="Date", how="left")
#     .merge(thai, on="Date", how="left")
#     .merge(usa, on="Date", how="left")

# )

# # --- Rename for compatibility ---
# df_macro.rename(columns={
#     "Bond_spread_10Y_5Y": "10Y_5Y_Bond_Spread",
#     "Bond_spread_5Y_1Y": "5Y_1Y_Bond_Spread",
#     "Bond_spread_10Y_1Y": "10Y_1Y_Bond_Spread"
# }, inplace=True)

# df_macro = df_macro.set_index("Date").sort_index()

# # ============================================================
# # STEP 1: Detect Macro Regime
# # ============================================================
# def detect_regime(df_macro):
#     df = df_macro.copy()

#     # Handle missing vars gracefully
#     for col in ["10Y_5Y_Bond_Spread","5Y_1Y_Bond_Spread","THOR_6M",
#                "THB_per_USD" ]:    # "Brent_Oil_USD_per_bbl"
#         if col not in df.columns:
#             df[col] = df[col].interpolate(limit_direction="both")

#     z_10_5 = (df["10Y_5Y_Bond_Spread"] - df["10Y_5Y_Bond_Spread"].mean()) / df["10Y_5Y_Bond_Spread"].std()
#     z_5_1 = (df["5Y_1Y_Bond_Spread"] - df["5Y_1Y_Bond_Spread"].mean()) / df["5Y_1Y_Bond_Spread"].std()
#     z_thor = (df["THOR_6M"] - df["THOR_6M"].mean()) / df["THOR_6M"].std()
#     # z_oil = (df["Brent_Oil_USD_per_bbl"] - df["Brent_Oil_USD_per_bbl"].mean()) / df["Brent_Oil_USD_per_bbl"].std()
#     z_fx = (df["THB_per_USD"] - df["THB_per_USD"].mean()) / df["THB_per_USD"].std()
#     z_thai = (df["SET_Index"] - df["SET_Index"].mean()) / df["SET_Index"].std()
#     z_usa = (df["SP500_Index"] - df["SP500_Index"].mean()) / df["SP500_Index"].std()

#     # Composite index
#     composite = 0.4*(z_10_5 + z_5_1)/2 - 0.3*z_thor - 0.15*z_fx + 0.15*z_thai + 0.15*z_usa
#     df["Regime_Score"] = composite
#     df["Regime"] = np.select(
#         [composite > 0.7, composite < -0.7],
#         ["Expansion", "Contraction"],
#         default="Neutral"
#     )
#     return df[["Regime","Regime_Score"]]

# df_regime = detect_regime(df_macro)

# # ============================================================
# # STEP 2: Merge Forecast + Daily + Regime
# # ============================================================
# expanded = []
# for _, row in df_fore.iterrows():
#     stock, sector, pred = row["Stock"], row["Sector"], row["Pred_dLogclose"]
#     df_s = df_daily[df_daily["Stock"] == stock].copy()
#     if df_s.empty:
#         continue
#     df_s["Pred_dLogclose"] = pred
#     df_s["Sector"] = sector
#     df_s = df_s.merge(df_regime, left_on="Date", right_index=True, how="left")
#     expanded.append(df_s)

# df_all = pd.concat(expanded, ignore_index=True).sort_values(["Sector","Stock","Date"])

# # ============================================================
# # STEP 3: Regime-dependent Weights
# # ============================================================
# def regime_weights(r):
#     if r == "Expansion": return (0.6, 0.25, 0.15)
#     elif r == "Contraction": return (0.3, 0.5, 0.2)
#     else: return (0.4, 0.4, 0.2)

# # ============================================================
# # STEP 4: Hybrid Score Computation
# # ============================================================
# df_all["Return"] = df_all.groupby("Stock")["Close"].pct_change()
# df_all["LogReturn"] = np.log1p(df_all["Return"])

# hybrid_results, equity_curves = [], {}

# for stock, sub in df_all.groupby("Stock"):
#     sub = sub.copy().sort_values("Date")

#     techconfirm = (
#         0.3 * sub["TechScore"].fillna(0)
#         + 0.2 * sub["MACD"].fillna(0)
#         + 0.1 * ((60 - abs(sub["RSI_14"].fillna(50)-50)) / 50)
#     )
#     wave_align = np.where(sub["Wave_Direction"] == ("Up" if sub["Pred_dLogclose"].iloc[0] > 0 else "Down"), 1, -1)
#     patternwave = (
#         0.2 * sub["Pattern_Confidence"].fillna(0)
#         + 0.2 * sub["Wave_Strength"].fillna(0) * wave_align
#     )

#     hybrid_score = []
#     for i, regime in enumerate(sub["Regime"]):
#         w_macro, w_tech, w_pattern = regime_weights(regime)
#         bias = np.sign(sub["Pred_dLogclose"].iloc[0])
#         score = (w_macro*bias) + (w_tech*techconfirm.iloc[i]) + (w_pattern*patternwave.iloc[i])
#         hybrid_score.append(score)

#     sub["Score"] = hybrid_score
#     sub["FinalSignal"] = np.where(abs(sub["Score"]) < 0.05, 0, np.sign(sub["Score"]))
#     sub["Strategy_Return"] = sub["FinalSignal"].shift(1) * sub["Return"]
#     sub["Cumulative_Return"] = (1 + sub["Strategy_Return"]).cumprod() - 1

#     # --- Performance Summary ---
#     hit_ratio = (np.sign(sub["Return"]) == sub["FinalSignal"].shift(1)).mean() * 100
#     cum_ret = sub["Cumulative_Return"].iloc[-1] * 100
#     avg_ret = sub["Strategy_Return"].mean() * 100
#     std_ret = sub["Strategy_Return"].std() * 100
#     sharpe = avg_ret / std_ret if std_ret > 0 else np.nan

#     hybrid_results.append({
#         "Stock": stock, "Sector": sub["Sector"].iloc[0],
#         "Hit_Ratio_%": hit_ratio, "Cumulative_Return_%": cum_ret,
#         "Avg_Daily_Return_%": avg_ret, "Volatility_%": std_ret,
#         "Sharpe_Ratio": sharpe
#     })
#     equity_curves[stock] = sub[["Date","Strategy_Return","Cumulative_Return","Regime"]]

# # ============================================================
# # STEP 5: Portfolio Summary
# # ============================================================
# all_dates = pd.concat([v["Date"] for v in equity_curves.values()]).drop_duplicates().sort_values()
# port = pd.DataFrame(index=all_dates)

# for k, v in equity_curves.items():
#     df_v = v.drop_duplicates(subset="Date").set_index("Date")  # 🔧 ป้องกัน duplicate
#     port[k] = df_v["Strategy_Return"].reindex(all_dates)

# # คำนวณผลตอบแทนพอร์ตเฉลี่ยรายวัน
# port["Portfolio_Return"] = port.mean(axis=1, skipna=True)
# port["Cumulative_Portfolio"] = (1 + port["Portfolio_Return"].fillna(0)).cumprod() - 1
# port_cum = port["Cumulative_Portfolio"].iloc[-1]*100
# port_avg = port["Portfolio_Return"].mean()*100
# port_std = port["Portfolio_Return"].std()*100
# port_sharpe = port_avg/port_std if port_std>0 else np.nan

# portfolio_summary = pd.DataFrame([{
#     "Portfolio_CumReturn_%": port_cum,
#     "Portfolio_AvgDailyReturn_%": port_avg,
#     "Portfolio_Volatility_%": port_std,
#     "Portfolio_Sharpe": port_sharpe
# }])

# df_strategy_summary = pd.DataFrame(hybrid_results).sort_values("Cumulative_Return_%", ascending=False)

# # ============================================================
# # STEP 6: Export + Visualization
# # ============================================================
# out_path = Path("Hybrid_Regime_Aware_Strategy.xlsx")
# with pd.ExcelWriter(out_path) as writer:
#     df_strategy_summary.to_excel(writer, sheet_name="Stock_Summary", index=False)
#     portfolio_summary.to_excel(writer, sheet_name="Portfolio_Summary", index=False)
#     port.to_excel(writer, sheet_name="Portfolio_EquityCurve", index=True)

# print(f"✅ Exported → {out_path.name}")

# plt.figure(figsize=(10,6))
# plt.plot(port["Cumulative_Portfolio"], label="Regime-aware Portfolio", lw=2)
# plt.title("Regime-Aware Hybrid Macro + Technical Strategy — Cumulative Return")
# plt.ylabel("Cumulative Return")
# plt.grid(True); plt.legend(); plt.show()

# # # ============================================================
# # # 📤 STEP 1.5 — Export Regime Scores for ML Fusion (used in Block 5.2B)
# # # ============================================================
# # df_regime_export = df_regime.reset_index()
# # df_regime_export.to_excel("Macro_Regime_Score.xlsx", index=False)
# # print("✅ Exported → Macro_Regime_Score.xlsx")

In [ ]:
# # ============================================================
# # 💰 BLOCK 5.3 — Dynamic Portfolio Simulation + Dashboard (Holdings Summary Version)
# # ============================================================

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import yfinance as yf

# # --- Simulation Parameters ---
# initial_capital = 10_000_000
# trade_frac = 0.05  # 3% per signal

# # ============================================================
# # 🧩 FIX — Ensure FinalSignal exists
# # ============================================================
# df_sim = df_all.copy().sort_values(["Date", "Stock"]).drop_duplicates(subset=["Date", "Stock"], keep="last")

# # ============================================================
# # 🧩 FIX — Ensure FinalSignal exists (Robust)
# # ============================================================
# df_sim = df_all.copy().sort_values(["Date", "Stock"]).drop_duplicates(subset=["Date", "Stock"], keep="last")

# if "FinalSignal" not in df_sim.columns:
#     print("⚠️  Column 'FinalSignal' not found → auto-generating ...")

#     if "Score" in df_sim.columns:
#         # ใช้ Score ถ้ามี
#         df_sim["FinalSignal"] = np.where(abs(df_sim["Score"]) < 0.05, 0, np.sign(df_sim["Score"]))

#     elif "Pred_dLogclose" in df_sim.columns:
#         # ใช้ Pred_dLogclose จาก ECM forecast
#         df_sim["FinalSignal"] = np.sign(df_sim["Pred_dLogclose"])

#     elif "Pred_Return" in df_sim.columns:
#         # fallback กรณีมีชื่ออื่น
#         df_sim["FinalSignal"] = np.sign(df_sim["Pred_Return"])

#     else:
#         # สุดท้าย ถ้าไม่มีอะไรเลย ให้ถือว่าไม่มี signal (ถือเงินสด)
#         print("⚠️  Neither Score nor Pred_dLogclose found — assuming all signals = 0 (no trades).")
#         df_sim["FinalSignal"] = 0


# # ============================================================
# # 🧮 Create pivot tables safely (handle duplicates)
# # ============================================================
# price_map = df_sim.pivot_table(index="Date", columns="Stock", values="Close", aggfunc="last")
# signal_map = df_sim.pivot_table(index="Date", columns="Stock", values="FinalSignal", aggfunc="last")

# # ============================================================
# # 🚀 Portfolio Simulation (No short-selling, no borrowing)
# # ============================================================
# cash = initial_capital
# holdings = {s: 0.0 for s in df_sim["Stock"].unique()}
# portfolio_history = []
# dates = sorted(df_sim["Date"].unique())

# for date in dates:
#     prices = price_map.loc[date]
#     signals = signal_map.loc[date]

#     # Portfolio valuation
#     holdings_value = sum(prices[s]*holdings[s] for s in holdings if not np.isnan(prices[s]))
#     total_value = cash + holdings_value

#     # --- Execute trades ---
#     for stock in holdings.keys():
#         price = prices[stock]
#         if np.isnan(price):
#             continue
#         sig = signals[stock]

#         if sig == 1:  # BUY (max 3% of remaining cash)
#             buy_amt = trade_frac * cash
#             if buy_amt > 0:
#                 qty = buy_amt / price
#                 holdings[stock] += qty
#                 cash -= qty * price

#         elif sig == -1:  # SELL (max 3% of current holdings)
#             sell_qty = trade_frac * holdings[stock]
#             if sell_qty > 0:
#                 holdings[stock] -= sell_qty
#                 cash += sell_qty * price

#     # --- Update after trades ---
#     holdings_value = sum(prices[s]*holdings[s] for s in holdings if not np.isnan(prices[s]))
#     total_value = cash + holdings_value

#     portfolio_history.append({
#         "Date": date,
#         "Cash": cash,
#         "Holdings_Value": holdings_value,
#         "Total_Portfolio": total_value
#     })

# # ============================================================
# # 📊 Portfolio Performance + Benchmark SET
# # ============================================================
# df_portfolio = pd.DataFrame(portfolio_history)
# df_portfolio["Daily_Return"] = df_portfolio["Total_Portfolio"].pct_change()
# df_portfolio["Cumulative_Return"] = (1 + df_portfolio["Daily_Return"].fillna(0)).cumprod() - 1

# # ============================================================
# # 🧭 Benchmark SET Index (Functional Fix)
# # ============================================================
# try:
#     set_idx = yf.download("^SET.BK", start=df_portfolio["Date"].min(), end=df_portfolio["Date"].max(), progress=False)
#     if not set_idx.empty:
#         if isinstance(set_idx.columns, pd.MultiIndex):
#             set_idx.columns = set_idx.columns.get_level_values(0)
#         set_idx = set_idx["Close"].resample("D").ffill().squeeze()
#         if isinstance(set_idx, pd.DataFrame):
#             set_idx = set_idx.iloc[:, 0]
#         set_ret = set_idx.pct_change().fillna(0)
#         set_cum = (1 + set_ret).cumprod() - 1
#         set_df = pd.DataFrame({"Date": set_idx.index, "SET_Close": set_idx.values, "SET_Return": set_ret.values, "SET_CumReturn": set_cum.values})
#         set_df["Date"] = pd.to_datetime(set_df["Date"])
#         df_portfolio = df_portfolio.reset_index(drop=True)
#         df_portfolio["Date"] = pd.to_datetime(df_portfolio["Date"])
#         df_portfolio = pd.merge_asof(
#             df_portfolio.sort_values("Date"),
#             set_df.sort_values("Date"),
#             on="Date",
#             direction="backward"
#         ).set_index("Date")
#         print("✅ SET Index benchmark successfully merged.")
#     else:
#         print("⚠️ Warning: SET Index data not available.")
# except Exception as e:
#     print(f"⚠️ Error merging SET Index: {e}")

# # ============================================================
# # 📈 Visualization Dashboard (Portfolio vs SET)
# # ============================================================
# plt.figure(figsize=(10,6))
# plt.plot(df_portfolio.index, df_portfolio["Cumulative_Return"], label="Strategy Portfolio", lw=2)
# plt.plot(df_portfolio.index, df_portfolio["SET_CumReturn"], label="SET Index", lw=2, ls="--")
# plt.title("Portfolio vs SET Index Benchmark", fontsize=13, fontweight="bold")
# plt.ylabel("Cumulative Return")
# plt.grid(True); plt.legend()
# plt.show()

# # ============================================================
# # 📋 Latest Holdings Summary
# # ============================================================
# latest_date = dates[-1]
# latest_prices = price_map.loc[latest_date]
# hold_df = pd.DataFrame([
#     {"Stock": s,
#      "Quantity": holdings[s],
#      "Last_Price": latest_prices[s],
#      "Market_Value": holdings[s] * latest_prices[s]}
#     for s in holdings.keys() if holdings[s] > 0
# ])
# hold_df["Weight_%"] = (hold_df["Market_Value"] / hold_df["Market_Value"].sum()) * 100
# hold_df = hold_df.sort_values("Market_Value", ascending=False).reset_index(drop=True)

# print("\n📊 Latest Portfolio Holdings (as of {})".format(latest_date.date()))
# display(hold_df.round(2))

# # ============================================================
# # 💾 Export
# # ============================================================
# with pd.ExcelWriter("Hybrid_Portfolio_Dashboard.xlsx") as writer:
#     df_portfolio.to_excel(writer, sheet_name="Portfolio_TimeSeries")
#     hold_df.to_excel(writer, sheet_name="Latest_Holdings", index=False)
# print("✅ Exported → Hybrid_Portfolio_Dashboard.xlsx")

# # ============================================================
# # 📋 Latest Holdings Summary
# # ============================================================
# latest_date = dates[-1]
# latest_prices = price_map.loc[latest_date]
# hold_df = pd.DataFrame([
#     {"Stock": s,
#      "Quantity": holdings[s],
#      "Last_Price": latest_prices[s],
#      "Market_Value": holdings[s] * latest_prices[s]}
#     for s in holdings.keys() if holdings[s] > 0
# ])
# hold_df["Weight_%"] = (hold_df["Market_Value"] / hold_df["Market_Value"].sum()) * 100
# hold_df = hold_df.sort_values("Market_Value", ascending=False).reset_index(drop=True)

# # --- 🧾 Portfolio Summary (Cash vs Equity) ---
# latest_cash = cash
# latest_equity_value = hold_df["Market_Value"].sum()
# total_portfolio_value = latest_cash + latest_equity_value
# cash_ratio = (latest_cash / total_portfolio_value) * 100
# equity_ratio = (latest_equity_value / total_portfolio_value) * 100

# print("\n📊 Latest Portfolio Holdings (as of {})".format(latest_date.date()))
# display(hold_df.round(2))

# print("\n💰 Portfolio Allocation Summary:")
# print(f"   • Total Portfolio Value : {total_portfolio_value:,.2f} THB")
# print(f"   •   ├─ Cash             : {latest_cash:,.2f} THB ({cash_ratio:.2f}%)")
# print(f"   •   └─ Equity Holdings  : {latest_equity_value:,.2f} THB ({equity_ratio:.2f}%)")

# # ============================================================
# # 💾 Export
# # ============================================================
# with pd.ExcelWriter("Hybrid_Portfolio_Dashboard.xlsx") as writer:
#     df_portfolio.to_excel(writer, sheet_name="Portfolio_TimeSeries")
#     hold_df.to_excel(writer, sheet_name="Latest_Holdings", index=False)

# summary_df = pd.DataFrame({
#     "Total_Value": [total_portfolio_value],
#     "Cash": [latest_cash],
#     "Equity_Value": [latest_equity_value],
#     "Cash_%": [cash_ratio],
#     "Equity_%": [equity_ratio]
# })
# summary_df.to_excel(writer, sheet_name="Allocation_Summary", index=False)

# print("\n✅ Exported → Hybrid_Portfolio_Dashboard.xlsx")



In [ ]:
# # ============================================================
# # 📊 BLOCK 5.4 — Regime-wise PyFolio Performance Summary (Refined)
# # ============================================================

# import numpy as np
# import pandas as pd

# # --- Ensure Regime column exists ---
# if "Regime" not in df_portfolio.columns:
#     df_reg = df_all[["Date", "Regime"]].drop_duplicates(subset="Date", keep="last")
#     df_reg["Date"] = pd.to_datetime(df_reg["Date"])
#     df_portfolio = df_portfolio.reset_index().merge(df_reg, on="Date", how="left").set_index("Date")

# # ============================================================
# # 🧮 Helper Functions
# # ============================================================
# def CAGR(series, freq=252):
#     if len(series) < 2:
#         return np.nan
#     if series.iloc[0] != 1:  # normalize
#         series = series / series.iloc[0]
#     total_ret = series.iloc[-1] - 1
#     years = len(series) / freq
#     return (1 + total_ret) ** (1 / years) - 1

# def max_drawdown(cumret):
#     roll_max = np.maximum.accumulate(cumret)
#     drawdown = (cumret - roll_max) / roll_max
#     return abs(drawdown.min())

# def win_rate(returns):
#     return (returns > 0).mean()

# def sortino_ratio(returns, rf=0.0):
#     downside = returns[returns < 0].std()
#     if downside == 0 or np.isnan(downside):
#         return np.nan
#     return (returns.mean() - rf) / downside

# def calmar_ratio(cagr, mdd):
#     if mdd == 0 or np.isnan(mdd):
#         return np.nan
#     return cagr / mdd

# # ============================================================
# # 🧾 Compute Metrics per Regime
# # ============================================================
# regime_stats = []

# for regime, df_reg in df_portfolio.groupby("Regime"):
#     if df_reg.empty:
#         continue

#     df_reg = df_reg.copy().sort_index()
#     cum = (1 + df_reg["Daily_Return"].fillna(0)).cumprod()

#     # --- Monthly compounded returns ---
#     monthly_ret = (1 + df_reg["Daily_Return"]).resample("M").prod() - 1
#     monthly_vol = df_reg["Daily_Return"].resample("M").std() * np.sqrt(21)

#     cagr = CAGR(cum)
#     mdd = max_drawdown(cum)
#     wr = win_rate(df_reg["Daily_Return"])
#     sortino = sortino_ratio(df_reg["Daily_Return"])
#     calmar = calmar_ratio(cagr, mdd)

#     regime_stats.append({
#         "Regime": regime,
#         "CAGR_%": cagr * 100,
#         "Max_Drawdown_%": mdd * 100,
#         "Win_Rate_%": wr * 100,
#         "Monthly_Return_%": np.nanmean(monthly_ret) * 100,
#         # "Monthly_Volatility_%": np.nanmean(monthly_vol) * 100,
#         "Sortino": sortino,
#         "Calmar": calmar
#     })

# # ============================================================
# # 📋 Summary Table
# # ============================================================
# df_regime_stats = pd.DataFrame(regime_stats).round(3)
# cols = ["Regime", "CAGR_%", "Max_Drawdown_%", "Win_Rate_%",
#         "Monthly_Return_%", "Sortino", "Calmar"]
# df_regime_stats = df_regime_stats[cols]

# print("\n📈 Regime-wise Performance Summary (PyFolio Style)")
# display(df_regime_stats)

# # ============================================================
# # 💾 Export
# # ============================================================
# df_regime_stats.to_excel("Hybrid_PyFolio_Regime_Summary.xlsx", index=False)
# print("✅ Exported → Hybrid_PyFolio_Regime_Summary.xlsx")


In [ ]:
# # ============================================================
# # 📈 BLOCK 5.5 — Regime-wise Equity Curve & Performance Visualization
# # ============================================================

# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# import pandas as pd

# # --- Ensure regime alignment with portfolio ---
# if "Regime" not in df_portfolio.columns:
#     df_reg = df_all[["Date", "Regime"]].drop_duplicates(subset="Date", keep="last")
#     df_reg["Date"] = pd.to_datetime(df_reg["Date"])
#     df_portfolio = df_portfolio.reset_index().merge(df_reg, on="Date", how="left").set_index("Date")

# # --- สร้าง cumulative return ในแต่ละ regime ---
# regime_curves = {}
# for regime, df_reg in df_portfolio.groupby("Regime"):
#     if df_reg.empty:
#         continue
#     cum = (1 + df_reg["Daily_Return"].fillna(0)).cumprod() - 1
#     regime_curves[regime] = cum

# # ============================================================
# # 🧾 Summary Table (รวมทุก Regime)
# # ============================================================
# regime_summary = []
# for regime, cum in regime_curves.items():
#     ann_ret = (1 + cum.iloc[-1]) ** (252 / len(cum)) - 1
#     mdd = abs(((1 + cum).cummax() - (1 + cum)) / (1 + cum).cummax()).max()
#     sharpe = df_portfolio[df_portfolio["Regime"] == regime]["Daily_Return"].mean() / df_portfolio[df_portfolio["Regime"] == regime]["Daily_Return"].std()
#     regime_summary.append({
#         "Regime": regime,
#         "Final_Return_%": cum.iloc[-1] * 100,
#         "Annualized_Return_%": ann_ret * 100,
#         "Max_Drawdown_%": mdd * 100,
#         "Sharpe": sharpe
#     })

# df_regime_equity = pd.DataFrame(regime_summary).round(3)
# print("\n📊 Regime-wise Equity Curve Summary")
# display(df_regime_equity)

# # ============================================================
# # 🎨 Visualization — Overlayed Regime Equity Curves
# # ============================================================
# palette = {"Expansion": "#2ecc71", "Neutral": "#f1c40f", "Contraction": "#e74c3c"}

# plt.figure(figsize=(12, 6))
# for regime, cum in regime_curves.items():
#     plt.plot(cum.index, cum, label=regime, lw=2, color=palette.get(regime, "gray"))

# plt.axhline(0, color="black", lw=1)
# plt.title("Regime-wise Equity Curves (Cumulative Return)", fontsize=14, fontweight="bold")
# plt.ylabel("Cumulative Return")
# plt.legend(title="Regime")
# plt.grid(True, linestyle="--", alpha=0.7)
# plt.tight_layout()
# plt.show()

# # ============================================================
# # 📦 Export
# # ============================================================
# with pd.ExcelWriter("Hybrid_Regime_Equity_Summary.xlsx") as writer:
#     df_regime_equity.to_excel(writer, sheet_name="Regime_Equity_Summary", index=False)
#     for regime, cum in regime_curves.items():
#         cum_df = cum.reset_index()
#         cum_df.columns = ["Date", f"{regime}_Cumulative_Return"]
#         cum_df.to_excel(writer, sheet_name=f"{regime}_Curve", index=False)

# print("✅ Exported → Hybrid_Regime_Equity_Summary.xlsx")


In [ ]:
# # ============================================================
# # 📊 BLOCK 5.6 — Stock-wise Profit/Loss per Regime (Linked + Ranking Table)
# # ============================================================

# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# import pandas as pd

# # --- ตรวจสอบสัญญาณ FinalSignal ---
# df_price_sig = df_all.copy()
# if "FinalSignal" not in df_price_sig.columns:
#     print("⚠️ Auto-generating FinalSignal from Score / Pred_dLogclose ...")
#     if "Score" in df_price_sig.columns:
#         df_price_sig["FinalSignal"] = np.where(abs(df_price_sig["Score"]) < 0.05, 0, np.sign(df_price_sig["Score"]))
#     elif "Pred_dLogclose" in df_price_sig.columns:
#         df_price_sig["FinalSignal"] = np.sign(df_price_sig["Pred_dLogclose"])
#     else:
#         raise KeyError("No Score or Pred_dLogclose found to create FinalSignal.")

# # --- ตรวจสอบ Regime ---
# if "Regime" not in df_price_sig.columns:
#     df_reg = df_portfolio.reset_index()[["Date", "Regime"]].drop_duplicates()
#     df_price_sig = df_price_sig.merge(df_reg, on="Date", how="left")

# # --- เพิ่ม Weight จากพอร์ตจริง (ถือหุ้นมากน้อยแค่ไหน) ---
# df_portfolio_reset = df_portfolio.reset_index()[["Date", "Total_Portfolio", "Holdings_Value", "Cash"]].copy()
# df_price_sig = df_price_sig.merge(df_portfolio_reset, on="Date", how="left")

# # สมมติให้ Weight ต่อหุ้น (position fraction) สัดส่วนจากกลยุทธ์จริง (3%)
# df_price_sig["Weight"] = 0.03 * (df_price_sig["FinalSignal"].shift(1) != 0).astype(int)

# # --- คำนวณ Return ของแต่ละหุ้นตามน้ำหนักจริง ---
# df_price_sig["Return"] = df_price_sig.groupby("Stock")["Close"].pct_change()
# df_price_sig["Strategy_Return"] = df_price_sig["Return"] * df_price_sig["Weight"] * df_price_sig["FinalSignal"].shift(1)

# # ============================================================
# # 🧮 คำนวณผลตอบแทนเฉลี่ยต่อหุ้นแยกตาม Regime
# # ============================================================
# regime_profit = (
#     df_price_sig.groupby(["Regime", "Stock"])["Strategy_Return"]
#     .mean()
#     .reset_index()
# )

# # Annualize (ประมาณ 252 วัน)
# regime_profit["Avg_Ann_Return_%"] = regime_profit["Strategy_Return"] * 252 * 100

# # ============================================================
# # 🎨 Visualization + Ranking
# # ============================================================
# regime_order = ["Expansion", "Neutral", "Contraction"]
# palette = {"Expansion": "#2ecc71", "Neutral": "#f1c40f", "Contraction": "#e74c3c"}

# ranking_tables = []

# for regime in regime_order:
#     sub = regime_profit[regime_profit["Regime"] == regime]
#     if sub.empty:
#         continue

#     sub_sorted = sub.sort_values("Avg_Ann_Return_%", ascending=False)

#     # --- Plot Bar Chart ---
#     plt.figure(figsize=(10, 5))
#     sns.barplot(
#         data=sub_sorted,
#         x="Stock", y="Avg_Ann_Return_%", color=palette[regime]
#     )
#     plt.axhline(0, color="black", linewidth=1)
#     plt.title(f"Strategy Profit/Loss by Stock — {regime} Regime", fontsize=14, fontweight='bold')
#     plt.ylabel("Average Annualized Return (%)")
#     plt.xlabel("Stock")
#     plt.xticks(rotation=45, ha="right")
#     plt.tight_layout()
#     plt.show()

#     # --- Top & Bottom 3 Ranking ---
#     top3 = sub_sorted.head(3).assign(Rank="Top 3")
#     worst3 = sub_sorted.tail(3).assign(Rank="Bottom 3")
#     ranking_tables.append(pd.concat([top3, worst3], ignore_index=True))

# # ============================================================
# # 📋 Combine Ranking Table
# # ============================================================
# df_ranking = pd.concat(ranking_tables, ignore_index=True)[
#     ["Regime", "Rank", "Stock", "Avg_Ann_Return_%"]
# ]
# df_ranking = df_ranking.sort_values(["Regime", "Rank", "Avg_Ann_Return_%"], ascending=[True, True, False])

# print("\n🏆 Top 3 / Bottom 3 Stocks by Regime (based on Avg Annualized Return %)")
# display(df_ranking.round(2))

# # ============================================================
# # 💾 Export
# # ============================================================
# with pd.ExcelWriter("Hybrid_Regime_StockPerformance_Linked.xlsx") as writer:
#     regime_profit.to_excel(writer, sheet_name="Stock_Returns", index=False)
#     df_ranking.to_excel(writer, sheet_name="Ranking_TopBottom", index=False)

# print("✅ Exported → Hybrid_Regime_StockPerformance_Linked.xlsx")


# Next period signal

In [ ]:
!pip install minisom

  Preparing metadata (setup.py) ... done
  Created wheel for minisom: filename=MiniSom-2.3.5-py3-none-any.whl size=12031 sha256=2ecc501c7fb756b6991bb3887d7a7413fe68a86a67a1969541689e7810c44151
  Stored in directory: /root/.cache/pip/wheels/0f/8c/a4/5b7aa56fa6ef11d536d45da775bcc5a2a1c163ff0f8f11990b
Successfully built minisom


In [ ]:
# # ============================================================
# # 🧭 BLOCK 5.7 — Next-Month Signal + Technical + SOM Suggestion
# # ============================================================

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# # ============================================================
# # 1️⃣ ตรวจสอบหรือสร้าง Technical Summary
# # ============================================================

# try:
#     df_technical_latest
#     print("✅ Using existing df_technical_latest.")
# except NameError:
#     print("⚠️ df_technical_latest not found — auto-generating mock version...")
#     df_technical_latest = pd.DataFrame({
#         "Stock": df_forecast_summary["Stock"].unique(),
#         "TechScore": np.random.uniform(-1, 1, len(df_forecast_summary["Stock"].unique())),
#         "MACD": np.random.uniform(-1, 1, len(df_forecast_summary["Stock"].unique())),
#         "RSI_14": np.random.uniform(30, 70, len(df_forecast_summary["Stock"].unique())),
#         "Pattern_Confidence": np.random.uniform(0, 1, len(df_forecast_summary["Stock"].unique())),
#         "Wave_Direction": np.random.choice(["Up", "Down"], len(df_forecast_summary["Stock"].unique())),
#         "Wave_Strength": np.random.uniform(0, 1, len(df_forecast_summary["Stock"].unique()))
#     })
#     print("✅ df_technical_latest mock created.")

# # ============================================================
# # 2️⃣ รวมข้อมูล Macro + Technical → Hybrid Signal
# # ============================================================

# df_signal = df_forecast_summary.copy()
# df_signal = df_signal.merge(df_technical_latest, on="Stock", how="left")

# # --- Macro bias ---
# bias = np.sign(df_signal["Pred_dLogclose"])

# # --- Technical confirmation ---
# techconfirm = (
#     0.3 * df_signal["TechScore"].fillna(0)
#     + 0.2 * df_signal["MACD"].fillna(0)
#     + 0.1 * ((60 - abs(df_signal["RSI_14"].fillna(50) - 50)) / 50)
# )

# # --- Pattern + Wave alignment ---
# wave_align = np.where(df_signal["Wave_Direction"] == np.where(bias>0, "Up", "Down"), 1, -1)
# patternwave = (
#     0.2 * df_signal["Pattern_Confidence"].fillna(0)
#     + 0.2 * df_signal["Wave_Strength"].fillna(0) * wave_align
# )

# # --- Hybrid score ---
# df_signal["Hybrid_Score"] = 0.5*bias + 0.3*techconfirm + 0.2*patternwave

# # --- Expected Return ---
# df_signal["Expected_Return_%"] = df_signal["Pred_dLogclose"] * 100

# # ============================================================
# # 3️⃣ Self-Organizing Map (SOM-style) Decision Zone
# # ============================================================

# def som_zone(row):
#     if row["Hybrid_Score"] > 0.1 and row["Pred_dLogclose"] > 0:
#         return "BUY"
#     elif -0.1 <= row["Hybrid_Score"] <= 0.1:
#         return "HOLD"
#     elif row["Hybrid_Score"] < -0.1 and row["Pred_dLogclose"] < 0:
#         return "SELL"
#     else:
#         return "HOLD"

# df_signal["SOM_Action"] = df_signal.apply(som_zone, axis=1)

# # ============================================================
# # 4️⃣ Sector Summary Table
# # ============================================================

# sector_summary = (
#     df_signal.groupby("Sector")
#     .agg(
#         N_Stocks=("Stock", "count"),
#         Avg_Hybrid_Score=("Hybrid_Score", "mean"),
#         Avg_Exp_Return=("Expected_Return_%", "mean"),
#         N_BUY=("SOM_Action", lambda x: (x == "BUY").sum()),
#         N_HOLD=("SOM_Action", lambda x: (x == "HOLD").sum()),
#         N_SELL=("SOM_Action", lambda x: (x == "SELL").sum())
#     )
#     .reset_index()
# )

# sector_summary["%BUY"] = sector_summary["N_BUY"] / sector_summary["N_Stocks"] * 100
# sector_summary["%SELL"] = sector_summary["N_SELL"] / sector_summary["N_Stocks"] * 100

# def suggest_position(row):
#     if row["%BUY"] > 60 and row["Avg_Hybrid_Score"] > 0.1:
#         return "Overweight"
#     elif row["%SELL"] > 60 and row["Avg_Hybrid_Score"] < -0.1:
#         return "Underweight"
#     else:
#         return "Neutral"

# sector_summary["Position_Suggestion"] = sector_summary.apply(suggest_position, axis=1)
# sector_summary = sector_summary.sort_values("Avg_Hybrid_Score", ascending=False).reset_index(drop=True)

# # ============================================================
# # 5️⃣ แสดงผลลัพธ์รวม
# # ============================================================

# print("\n=== 📈 Next-Month Hybrid Trading Signals (Stock Level) ===")
# display(
#     df_signal[[
#         "Stock","Sector","Pred_dLogclose","Expected_Return_%",
#         "TechScore","Hybrid_Score","SOM_Action"
#     ]].sort_values("Hybrid_Score", ascending=False).round(3)
# )

# print("\n=== 💼 Sector Position Suggestion Table ===")
# display(
#     sector_summary[[
#         "Sector","N_Stocks","N_BUY","N_HOLD","N_SELL",
#         "%BUY","%SELL","Avg_Hybrid_Score","Avg_Exp_Return","Position_Suggestion"
#     ]].round(3)
# )

# # ============================================================
# # 🎨 SOM Visualization — Stock Labels + Hybrid Score
# # ============================================================

# from minisom import MiniSom
# from sklearn.preprocessing import StandardScaler
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np

# # --- เตรียม features สำหรับ SOM ---
# features = ["Pred_dLogclose", "TechScore", "MACD", "RSI_14", "Pattern_Confidence", "Wave_Strength"]
# X = StandardScaler().fit_transform(df_signal[features].fillna(0))

# # --- Train SOM ---
# som_x, som_y = 8, 8
# som = MiniSom(x=som_x, y=som_y, input_len=X.shape[1],
#               sigma=1.0, learning_rate=0.5,
#               neighborhood_function='gaussian', random_seed=42)
# som.random_weights_init(X)
# som.train_random(X, 1000, verbose=False)

# # --- คำนวณตำแหน่ง node ของแต่ละหุ้น ---
# win_map = np.array([som.winner(x) for x in X])
# df_signal["SOM_X"] = win_map[:,0]
# df_signal["SOM_Y"] = win_map[:,1]

# # --- สีของ Action ---
# color_map = {"BUY": "#2ecc71", "HOLD": "#f1c40f", "SELL": "#e74c3c"}

# # --- สร้างค่าเฉลี่ย Hybrid Score ต่อ node ---
# node_mean = (
#     df_signal.groupby(["SOM_X","SOM_Y"])["Hybrid_Score"]
#     .mean().unstack().fillna(0)
# )

# # ============================================================
# # 🧠 BLOCK 5.7 — Enhanced SOM Cluster Map (Hybrid + Technical)
# # ============================================================

# from minisom import MiniSom
# from sklearn.preprocessing import StandardScaler
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np

# # --- ⚙️ เตรียม features สำหรับ SOM ---
# df_signal["RSI_14_scaled"] = (df_signal["RSI_14"] - 50) / 25           # RSI 25–75 → -1 ถึง +1
# df_signal["Hybrid_Score_Scaled"] = df_signal["Hybrid_Score"] * 2.5     # ขยาย dynamic range

# features = [
#     "Pred_dLogclose", "TechScore", "MACD",
#     "RSI_14_scaled", "Pattern_Confidence", "Wave_Strength"
# ]
# X = StandardScaler().fit_transform(df_signal[features].fillna(0))

# # --- 🔄 Train SOM ---
# som_x, som_y = 8, 8
# som = MiniSom(
#     x=som_x, y=som_y, input_len=X.shape[1],
#     sigma=1.0, learning_rate=0.5,
#     neighborhood_function='gaussian', random_seed=42
# )
# som.random_weights_init(X)
# som.train_random(X, 1000, verbose=False)

# # --- 📍 หา node ของแต่ละหุ้น ---
# win_map = np.array([som.winner(x) for x in X])
# df_signal["SOM_X"] = win_map[:, 0]
# df_signal["SOM_Y"] = win_map[:, 1]

# # --- 🎨 สี Action ---
# color_map = {"BUY": "#2ecc71", "HOLD": "#f1c40f", "SELL": "#e74c3c"}

# # --- 🧮 คำนวณค่าเฉลี่ย Hybrid Score ต่อ node ---
# node_mean = (
#     df_signal.groupby(["SOM_X", "SOM_Y"])["Hybrid_Score_Scaled"]
#     .mean().unstack().fillna(0)
# )

# # ============================================================
# # 🎨 Visualization — SOM Heatmap + Stock Labels + Score
# # ============================================================
# plt.figure(figsize=(9, 8))
# plt.title("🧠 SOM Cluster Map — Hybrid (Macro + Technical)", fontsize=14, fontweight="bold")

# # --- Heatmap พื้นหลัง (Dynamic scale) ---
# vmin, vmax = node_mean.min().min(), node_mean.max().max()
# sns.heatmap(
#     node_mean,
#     cmap="RdYlGn", vmin=vmin, vmax=vmax,
#     cbar_kws={"label": "Mean Hybrid Score"},
#     square=True, linewidths=0.5, alpha=0.25
# )

# # --- Plot หุ้นแต่ละตัว (ชื่อ + Score + สี Action) ---
# for _, row in df_signal.iterrows():
#     x, y = row["SOM_X"] + 0.5, row["SOM_Y"] + 0.5
#     color = color_map.get(row["SOM_Action"], "gray")
#     plt.scatter(x, y, s=200, color=color, edgecolor="black", alpha=0.9, zorder=3)
#     plt.text(
#         x, y,
#         f"{row['Stock']} ({row['Hybrid_Score']:+.2f})",
#         fontsize=8, ha="center", va="center",
#         color="black", fontweight="bold"
#     )

# plt.xlabel("SOM X (Cluster Column)")
# plt.ylabel("SOM Y (Cluster Row)")
# plt.grid(False)
# plt.tight_layout()
# plt.show()




# # ============================================================
# # 7️⃣ Export Excel
# # ============================================================

# with pd.ExcelWriter("Hybrid_SOM_Position_Suggestion.xlsx") as writer:
#     df_signal.to_excel(writer, sheet_name="Stock_Signals", index=False)
#     sector_summary.to_excel(writer, sheet_name="Sector_Suggestion", index=False)

# print("✅ Exported → Hybrid_SOM_Position_Suggestion.xlsx")


# Dynamic portfolio = Quantitative deterministic (Sharpe-weighted + Signal flip)

=> เร็วกว่า, ใช้แนว Bellman update เบื้องต้นโดยไม่ต้อง train RL model

In [ ]:
# import pandas as pd
# import numpy as np

# # ============================================================
# # ⚙️ BLOCK 6 (Revised): Long-Only RL Deterministic Allocation (All outputs in %)
# # ============================================================

# def adaptive_portfolio_rl_longonly(evaluation_results, window_corr=12, window_sharpe=6):
#     all_dates = sorted(list(set().union(*[df.index for df in evaluation_results.values()])))
#     portfolio_returns = pd.DataFrame(index=all_dates)

#     adj_signals = {}
#     sh_ratio = {}

#     # --- Step 1: Compute adjusted signal and Sharpe ---
#     for stock, df_eval in evaluation_results.items():
#         df = df_eval.copy()
#         df["Signal_raw"] = np.sign(df["Pred_dLogclose"])
#         df["Corr_Pred_Actual"] = df["Pred_dLogclose"].rolling(window_corr).corr(df["Actual_dLogclose"])
#         df["Signal_adj"] = np.where(df["Corr_Pred_Actual"] < 0, -df["Signal_raw"], df["Signal_raw"])
#         df["Return"] = df["Actual_dLogclose"]
#         df["Sharpe"] = (
#             df["Return"].rolling(window_sharpe).mean() / df["Return"].rolling(window_sharpe).std()
#         )
#         adj_signals[stock] = df["Signal_adj"]
#         sh_ratio[stock] = df["Sharpe"]
#         portfolio_returns[stock] = df["Return"]

#     weights_df = pd.DataFrame(index=all_dates, columns=list(evaluation_results.keys()))
#     port_ret_series = []

#     # --- Step 2: Compute weights (non-negative long-only) ---
#     for t, date in enumerate(all_dates):
#         w_list, r_list = [], []

#         for stock in evaluation_results.keys():
#             sig = adj_signals[stock].reindex(all_dates).iloc[t]
#             shrp = sh_ratio[stock].reindex(all_dates).iloc[t]
#             ret = portfolio_returns[stock].reindex(all_dates).iloc[t]

#             if np.isnan(sig) or np.isnan(shrp) or np.isnan(ret):
#                 continue

#             w_val = sig * abs(shrp)
#             w_list.append((stock, w_val))
#             r_list.append((stock, ret))

#         if not w_list:
#             port_ret_series.append(0.0)
#             continue

#         w_df = pd.DataFrame(w_list, columns=["Stock", "w"])
#         w_df["w"] = np.clip(w_df["w"], 0, None)

#         if w_df["w"].sum() > 0:
#             w_df["w"] /= w_df["w"].sum()
#         else:
#             w_df["w"] = 1.0 / len(w_df)

#         for s in w_df["Stock"]:
#             weights_df.loc[date, s] = w_df.loc[w_df["Stock"] == s, "w"].iloc[0]

#         r_df = pd.DataFrame(r_list, columns=["Stock", "Return"])
#         merged = pd.merge(w_df, r_df, on="Stock")
#         port_ret = np.sum(merged["w"] * merged["Return"])
#         port_ret_series.append(port_ret)

#     # --- Step 3: Portfolio performance summary ---
#     portfolio_returns["Portfolio_Return"] = port_ret_series
#     portfolio_returns["Cumulative_Portfolio"] = np.exp(portfolio_returns["Portfolio_Return"].cumsum()) - 1

#     # ✅ Convert ALL numeric columns to percentage
#     portfolio_returns_pct = portfolio_returns.copy() * 100
#     portfolio_returns_pct.columns = [col + "(%)" for col in portfolio_returns.columns]

#     # Compute summary statistics in %
#     avg_r = portfolio_returns["Portfolio_Return"].mean() * 100
#     std_r = portfolio_returns["Portfolio_Return"].std() * 100
#     sharpe_r = avg_r / std_r if std_r > 0 else np.nan
#     cum_r = portfolio_returns["Cumulative_Portfolio"].iloc[-1] * 100

#     summary = pd.DataFrame([{
#         "CumReturn_%": cum_r,
#         "AvgMonthlyReturn_%": avg_r,
#         "Volatility_%": std_r,
#         "Sharpe": sharpe_r
#     }])

#     weights_df = weights_df * 100  # export weights in %
#     return portfolio_returns_pct, weights_df, summary


# # ============================================================
# # 🚀 Run Adaptive Allocation (Long-only, % export)
# # ============================================================

# portfolio_rl, weights_rl, summary_rl = adaptive_portfolio_rl_longonly(evaluation_results)

# with pd.ExcelWriter("ECM_RL_AdaptivePortfolio_LongOnly_Percent.xlsx") as writer:
#     summary_rl.to_excel(writer, sheet_name="Portfolio_Summary", index=False)
#     weights_rl.to_excel(writer, sheet_name="Dynamic_Weights(%)")
#     portfolio_rl.to_excel(writer, sheet_name="Portfolio_Returns(%)")

# print("\n✅ Exported → ECM_RL_AdaptivePortfolio_LongOnly_Percent.xlsx")

# print("\n=== 💼 Portfolio Summary ===")
# display(summary_rl)

# print("\n=== 📊 Dynamic Weights (last 5 dates, %) ===")
# display(weights_rl.tail(5))

# print("\n=== 📈 Portfolio Return Curve (last 10, in %) ===")
# display(portfolio_rl.tail(10))


In [ ]:
# # ============================================================
# # 🎨 BLOCK 7: Visualization of Dynamic Portfolio Allocation (By Sector)
# # ============================================================
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.style.use("seaborn-v0_8-whitegrid")
# sns.set(font_scale=0.9)

# # ============================================================
# # 🔹 STEP 1: Prepare sector mapping
# # ============================================================

# # สร้าง DataFrame mapping จาก evaluation_results (หรือ summary เดิม)
# sector_map = {}
# for stock, df_eval in evaluation_results.items():
#     sector = df_eval["Sector"].iloc[0]
#     sector_map[stock] = sector

# # ตรวจสอบว่ามี mapping ครบไหม
# print("📊 Sector mapping:", sector_map)

# # ============================================================
# # 🔹 STEP 2: Convert weights_rl เป็นราย Sector
# # ============================================================

# weights_sector = pd.DataFrame(index=weights_rl.index)

# for stock in weights_rl.columns:
#     sector = sector_map.get(stock, "Unknown")
#     if sector not in weights_sector.columns:
#         weights_sector[sector] = 0.0
#     weights_sector[sector] += pd.to_numeric(weights_rl[stock], errors="coerce").fillna(0)

# # Normalize ให้ sum ต่อแถว = 100%
# weights_sector = weights_sector.div(weights_sector.sum(axis=1), axis=0) * 100

# print("\n✅ Sector-level weights created:")
# display(weights_sector.tail(3))

# # ============================================================
# # 🔹 STEP 3: Heatmap by Sector
# # ============================================================

# fig, ax = plt.subplots(figsize=(10, 5))
# sns.heatmap(weights_sector.T, cmap="YlOrRd", cbar_kws={'label': 'Weight (%)'}, ax=ax)
# ax.set_title("Dynamic Portfolio Allocation by Sector (%)", fontsize=14, weight="bold")
# ax.set_xlabel("Date")
# ax.set_ylabel("Sector")
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()

# # ============================================================
# # 🔹 STEP 4: Area Chart by Sector
# # ============================================================

# fig, ax = plt.subplots(figsize=(12, 6))
# weights_sector.plot.area(ax=ax, stacked=True, alpha=0.8)
# ax.set_title("Portfolio Allocation Composition by Sector (%)", fontsize=14, weight="bold")
# ax.set_ylabel("Weight (%)")
# ax.set_xlabel("Date")
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title="Sector")
# plt.tight_layout()
# plt.show()

# # ============================================================
# # 🔹 STEP 5: Pie Chart (ล่าสุด)
# # ============================================================

# last_date = weights_sector.index[-1]
# last_weights = weights_sector.loc[last_date].dropna()
# fig, ax = plt.subplots(figsize=(6, 6))
# ax.pie(last_weights, labels=last_weights.index, autopct='%1.1f%%', startangle=90)
# ax.set_title(f"Sector Allocation on {last_date.strftime('%Y-%m')}", fontsize=13, weight="bold")
# plt.tight_layout()
# plt.show()


# Next-Month Trading Plan after Bellman theory  => Exclude it

In [ ]:
# # ============================================================
# # 📈 BLOCK: Sharpe-weighted + Signal Flip (with Flip Info + Comments)
# # ============================================================

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# # ===== 1️⃣ Sharpe-weighted Portfolio Allocation =====
# df_perf = df_strategy_summary.copy()
# df_perf["Sharpe_Positive"] = np.where(df_perf["Sharpe_Ratio"] > 0, df_perf["Sharpe_Ratio"], 0)
# total_sharpe = df_perf["Sharpe_Positive"].sum()
# df_perf["Weight_Sharpe"] = np.where(total_sharpe > 0, df_perf["Sharpe_Positive"] / total_sharpe, 0)

# # ===== 2️⃣ สัญญาณพยากรณ์ล่าสุด =====
# df_signal_next = df_forecast_summary.copy()
# df_signal_next = df_signal_next.merge(df_perf[["Stock", "Sharpe_Ratio", "Weight_Sharpe"]], on="Stock", how="left")

# # ===== 3️⃣ สร้างสัญญาณและตรวจจับการ Flip =====
# df_signal_next["Raw_Signal"] = np.where(df_signal_next["Pred_dLogclose"] > 0, "BUY", "SELL")
# df_signal_next["Flip_Flag"] = np.where(df_signal_next["Sharpe_Ratio"] < 0, True, False)
# df_signal_next["Final_Signal"] = np.where(
#     df_signal_next["Flip_Flag"],
#     np.where(df_signal_next["Raw_Signal"] == "BUY", "SELL", "BUY"),
#     df_signal_next["Raw_Signal"]
# )

# # ===== 4️⃣ เพิ่ม Comment / Interpretation =====
# df_signal_next["Flip_Status"] = np.where(df_signal_next["Flip_Flag"], "🔁 Flipped", "✅ Normal")
# df_signal_next["Comment"] = np.where(
#     df_signal_next["Flip_Flag"],
#     "Model weak (Sharpe < 0) → Contrarian signal",
#     "Model consistent (Sharpe > 0)"
# )

# df_signal_next["Expected_Return_%"] = df_signal_next["Pred_dLogclose"] * 100
# df_signal_next["Weighted_Exp_Return_%"] = df_signal_next["Expected_Return_%"] * df_signal_next["Weight_Sharpe"]

# # ===== 5️⃣ Export & Display =====
# cols_show = [
#     "Stock", "Sector", "Sharpe_Ratio", "Weight_Sharpe",
#     "Pred_dLogclose", "Expected_Return_%",
#     "Raw_Signal", "Final_Signal", "Flip_Status", "Comment"
# ]

# # 🔹 เรียงตาม Expected_Return_% จากมากไปน้อย
# df_signal_next = df_signal_next[cols_show].sort_values("Expected_Return_%", ascending=False)

# df_signal_next.to_excel("NextMonth_SignalPlan_SharpeWeighted_FlipInfo.xlsx", index=False)
# print("✅ Exported → NextMonth_SignalPlan_SharpeWeighted_FlipInfo.xlsx")

# # ===== 6️⃣ PRINT TABLE =====
# print("\n=== 📈 Next-Month Trading Signals (Sharpe-weighted + Flip Info, Sorted by Expected Return) ===")
# display(df_signal_next)

# # ===== 7️⃣ Visualization =====
# plt.figure(figsize=(10,6))
# sns.barplot(
#     data=df_signal_next,
#     x="Stock",
#     y="Expected_Return_%",
#     hue="Final_Signal",
#     dodge=False,
#     order=df_signal_next["Stock"]  # ใช้ลำดับเดียวกับที่เรียงในตาราง
# )
# plt.axhline(0, color="black", lw=1)
# plt.title("Next-Month Expected Return by Stock (Sharpe-weighted + Flip)", weight="bold")
# plt.xticks(rotation=45, ha='right')
# plt.show()


1. สร้างโมเดล Deep Learning (Technical Side) [งานหลักของคุณ]ส่วนนี้คือการสร้าง "ตา" ข้างที่สองให้ AI (ข้างแรกคือ ARDL ของเพื่อน)สร้างโมเดล LSTM หรือ GRU: เขียน Code แยกออกมาเพื่อรับกราฟราคาและ Technical Indicators (RSI, MACD) 1กำหนดค่า Output ($b_t$): ต้องเทรนให้โมเดลทำนายออกมาเป็น ความน่าจะเป็น (Probability) ว่าเดือนถัดไปตลาดจะ Bull, Bear หรือ Neutral 2สิ่งที่ต้องระวัง: อย่าเผลอทำนายเป็น "ราคาปิด" (Price Regression) เพราะใน Proposal ระบุชัดเจนว่าต้องเป็นสัญญาณความน่าจะเป็น (Classification/Probability) เพื่อส่งต่อให้ RL2. สร้างระบบ Reinforcement Learning (The Brain) [งานหลักของคุณ]นี่คือ "สมอง" ของระบบที่ต้องเอาข้อมูลทุกอย่างมารวมกันออกแบบ Environment: เขียน Class Env ใน Python ให้รับข้อมูล 2 ทาง:Macro Signals ($m_t$): รับค่าที่เพื่อนคุณทำเสร็จแล้ว (จาก ARDL)Technical Signals ($b_t$): รับค่าจากโมเดล Deep Learning ที่คุณทำในข้อ 1รวม State ($S_t$): จับสองค่าข้างบนมารวมกันเป็น State เดียว 3Train PPO Agent: รันการเรียนรู้เพื่อให้ AI ตัดสินใจ (Buy/Hold/Sell) โดยใช้ Reward Function ที่หักลบค่า Drawdown 43. การเชื่อมต่อข้อมูลกับเพื่อน (Data Interface)เพื่อให้งานประกอบกันได้ง่าย คุณต้องตกลงกับเพื่อนให้ชัดเจนครับ:Request: บอกเพื่อนว่า "ขอไฟล์ Excel/CSV ที่มีคอลัมน์ Predicted Return หรือ Macro Signal จาก ARDL รายเดือน มาให้ฉันหน่อย"Action: คุณจะเอาคอลัมน์นั้นมาเป็น Input ส่วน $m_t$ ใน Code RL ของคุณครับ4. Backtesting & Reportนำ AI ที่เทรนเสร็จแล้ว มาเทรดจำลองกับข้อมูลปี 2025 และสรุปผลวัดค่า Sharpe Ratio, Drawdown เทียบกับตลาด 5สรุป: งานคุณตอนนี้ลดลงเหลือแค่สาย Programming & AI ล้วนๆ คือ "ทำ DL ให้เสร็จ -> เอา ARDL ของเพื่อนมายัดใส่ RL -> เทรน AI -> วัดผล" ครับ

In [ ]:
!jupyter nbconvert --to html "Semester2_V1_AQT_ARDLTrading_Macro&Technical.ipynb" --output "AQT_Semester1_Final.html"


[NbConvertApp] WARNING | pattern 'Semester2_V1_AQT_ARDLTrading_Macro&Technical.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterA